In [108]:
# -*- coding: utf-8 -*-
"""
Created on Mon Oct 10 21:05:57 2022

@author: Giorgia
"""
from scipy import interpolate
import numpy as np

from numba import njit
#@njit
class GlobalSpline2D(interpolate.interp2d):
    def __init__(self, x , y , z , kind='linear'):
        if kind == 'linear':
            if len(x) < 2 or len(y) < 2:
                raise self.get_size_error(2, kind)
        elif kind == 'cubic':
            if len(x) < 4 or len(y) < 4:
                raise self.get_size_error(4, kind)
        elif kind == 'quintic':
            if len(x) < 6 or len(y) < 6:
                raise self.get_size_error(6, kind)
        else:
            raise ValueError('unidentifiable kind of spline')

        super().__init__(x, y, z, kind=kind)
        self.extrap_fd_based_xs = self._linspace_10(self.x_min, self.x_max, -4)
        self.extrap_bd_based_xs = self._linspace_10(self.x_min, self.x_max, 4)
        self.extrap_fd_based_ys = self._linspace_10(self.y_min, self.y_max, -4)
        self.extrap_bd_based_ys = self._linspace_10(self.y_min, self.y_max, 4)

    @staticmethod
    def get_size_error(size, spline_kind):
        return ValueError('length of x and y must be larger or at least equal '
                          'to {} when applying {} spline, assign arrays with '
                          'length no less than '
                          '{}'.format(size, spline_kind, size))

    @staticmethod
    def _extrap1d(xs, ys, tar_x):
        if isinstance(xs, np.ndarray):
            xs = np.ndarray.flatten(xs)
        if isinstance(ys, np.ndarray):
            ys = np.ndarray.flatten(ys)
        assert len(xs) >= 4
        assert len(xs) == len(ys)
        f = interpolate.InterpolatedUnivariateSpline(xs, ys)
        return f(tar_x)

    @staticmethod
    def _linspace_10(p1, p2, cut=None):
        ls = list(np.linspace(p1, p2, 10))
        if cut is None:
            return ls
        assert cut <= 10
        return ls[-cut:] if cut < 0 else ls[:cut]

    def _get_extrap_based_points(self, axis, extrap_p):
        if axis == 'x':
            return (self.extrap_fd_based_xs if extrap_p > self.x_max else
                    self.extrap_bd_based_xs if extrap_p < self.x_min else [])
        elif axis == 'y':
            return (self.extrap_fd_based_ys if extrap_p > self.y_max else
                    self.extrap_bd_based_ys if extrap_p < self.y_min else [])
        assert False, 'axis unknown'
        
    def __call__(self, x_, y_, **kwargs):
        xs = np.atleast_1d(x_)
        ys = np.atleast_1d(y_)

        if xs.ndim != 1 or ys.ndim != 1:
            raise ValueError("x and y should both be 1-D arrays")

        pz_yqueue = []
        for y in ys:
            extrap_based_ys = self._get_extrap_based_points('y', y)

            pz_xqueue = []
            for x in xs:
                extrap_based_xs = self._get_extrap_based_points('x', x)

                if not extrap_based_xs and not extrap_based_ys:
                    # inbounds
                    pz = super().__call__(x, y, **kwargs)[0]

                elif extrap_based_xs and extrap_based_ys:
                    # both x, y atr outbounds
                    # allocate based_z from x, based_ys
                    extrap_based_zs = self.__call__(x,
                                                    extrap_based_ys,
                                                    **kwargs)
                    # allocate z of x, y from based_ys, based_zs
                    pz = self._extrap1d(extrap_based_ys, extrap_based_zs, y)

                elif extrap_based_xs:
                    # only x outbounds
                    extrap_based_zs = super().__call__(extrap_based_xs,
                                                       y,
                                                       **kwargs)
                    pz = self._extrap1d(extrap_based_xs, extrap_based_zs, x)

                else:
                    # only y outbounds
                    extrap_based_zs = super().__call__(x,
                                                       extrap_based_ys,
                                                       **kwargs)
                    pz = self._extrap1d(extrap_based_ys, extrap_based_zs, y)

                pz_xqueue.append(pz)

            pz_yqueue.append(pz_xqueue)

        zss = pz_yqueue
        if len(zss) == 1:
            zss = zss[0]
        return np.array(zss)    

In [18]:
using QuantEcon
using CompEcon
# function to approximate
f(x) = exp.(-x)

# Set the endpoints of approximation interval:
a =  -1                            # left endpoint
b =   1                            # right endpoint

# Choose an approximation scheme. In this case, let us use an order 10
# Chebychev approximation scheme:
n = 10                             # order of approximation
basis = fundefn(:cheb, n, a, b)      # define basis

# Compute the basis coefficients c.  There are various way to do this:
# One may use funfitf:
c = funfitf(basis, f)

# ... or one may compute the standard approximation nodes x and corresponding
# function values y and use funfitxy:
x = funnode(basis)[1]
y = f(x)
c = funfitxy(basis, x, y)[1]

# ... or one compute the standard approximation nodes x, corresponding
# function values y, and the interpolation matrix phi, and solve the
# interpolation equation directly using the backslash operator:
x = funnode(basis)[1]
y = f(x)
phi = funbase(basis)
c = phi\y

# Having computed the basis coefficients, one may now evaluate the
# approximant at any point x using funeval:
x = [0.0]
y = funeval(c, basis, x)[1]

tau0int_vec = ones(Float16, (30000,1))
zero = zeros(Float16,3)
one = ones(Float16,3)
EDUC = [2,3,4]
fspaceergeduc = fundefn(:spli,EDUC,zero,one)
#fspaceergeduc = fundef(EDUC)
Qeduc           = funbas(fspaceergeduc,tau0int_vec)

print(Qeduc)

LoadError: ArgumentError: range(0.0, stop=1.0, length=1): endpoints differ

In [11]:
typeof(EDUC)
print(EDUC)

[1, 2, 3]

In [107]:
# -*- coding: utf-8 -*-
"""
Created on Sun Oct 23 15:53:29 2022

@author: Giorgia
"""

import numpy as np
from numba import njit, boolean, int32, double, void
#from .linear_interp import binary_search

def binary_search(imin,Nx,x,xi):

    # a. checks
    if xi <= x[0]:
        return 0
    elif xi >= x[Nx-2]:
        return Nx-2

    # b. binary search
    half = Nx//2
    while half:
        imid = imin + half
        if x[imid] <= xi:
            imin = imid
        Nx -= half
        half = Nx//2

    return imin

#@njit(double(double[:],double[:],double[:,:],double,double,int32,int32),fastmath=True)
def _interp_2d(grid1,grid2,value,xi1,xi2,j1,j2):
    """ 2d interpolation for one point with known location
        
    Args:
        grid1 (numpy.ndarray): 1d grid
        grid2 (numpy.ndarray): 1d grid
        value (numpy.ndarray): value array (2d)
        xi1 (double): input point
        xi2 (double): input point
        j1 (int): location in grid 
        j2 (int): location in grid
    Returns:
        yi (double): output
    """

    # a. left/right
    nom_1_left = grid1[j1+1]-xi1
    nom_1_right = xi1-grid1[j1]

    nom_2_left = grid2[j2+1]-xi2
    nom_2_right = xi2-grid2[j2]

    # b. interpolation
    denom = (grid1[j1+1]-grid1[j1])*(grid2[j2+1]-grid2[j2])
    nom = 0
    for k1 in range(2):
        nom_1 = nom_1_left if k1 == 0 else nom_1_right
        for k2 in range(2):
            nom_2 = nom_2_left if k2 == 0 else nom_2_right                    
            nom += nom_1*nom_2*value[j1+k1,j2+k2]

    return nom/denom

#@njit(double(double[:],double[:],double[:,:],double,double),fastmath=True)
def interp_2d(grid1,grid2,value,xi1,xi2):
    """ 2d interpolation for one point
        
    Args:
        grid1 (numpy.ndarray): 1d grid
        grid2 (numpy.ndarray): 1d grid
        value (numpy.ndarray): value array (2d)
        xi1 (double): input point
        xi2 (double): input point
    Returns:
        yi (double): output
    """

    # a. search in each dimension
    j1 = binary_search(0,grid1.size,grid1,xi1)
    j2 = binary_search(0,grid2.size,grid2,xi2)

    return _interp_2d(grid1,grid2,value,xi1,xi2,j1,j2)

In [106]:
import numpy as np
from numba import njit, boolean, int32, double, void
def binary_search(imin,Nx,x,xi):

    # a. checks
    if xi <= x[0]:
        return 0
    elif xi >= x[Nx-2]:
        return Nx-2

    # b. binary search
    half = Nx//2
    while half:
        imid = imin + half
        if x[imid] <= xi:
            imin = imid
        Nx -= half
        half = Nx//2

    return imin

#@njit(double(double[:],double[:],double[:],double[:,:,:],double,double,double,int32,int32,int32),fastmath=True)
def _interp_3d(grid1,grid2,grid3,value,xi1,xi2,xi3,j1,j2,j3):
    """ 3d interpolation for one point with known location
        
    Args:
        grid1 (numpy.ndarray): 1d grid
        grid2 (numpy.ndarray): 1d grid
        grid3 (numpy.ndarray): 1d grid
        value (numpy.ndarray): value array (3d)
        xi1 (double): input point
        xi2 (double): input point
        xi3 (double): input point
        j1 (int): location in grid 
        j2 (int): location in grid
        j3 (int): location in grid
    Returns:
        yi (double): output
    """

    # a. left/right
    nom_1_left = grid1[j1+1]-xi1
    nom_1_right = xi1-grid1[j1]

    nom_2_left = grid2[j2+1]-xi2
    nom_2_right = xi2-grid2[j2]

    nom_3_left = grid3[j3+1]-xi3
    nom_3_right = xi3-grid3[j3]

    # b. interpolation
    denom = (grid1[j1+1]-grid1[j1])*(grid2[j2+1]-grid2[j2])*(grid3[j3+1]-grid3[j3])
    nom = 0
    for k1 in range(2):
        nom_1 = nom_1_left if k1 == 0 else nom_1_right
        for k2 in range(2):
            nom_2 = nom_2_left if k2 == 0 else nom_2_right       
            for k3 in range(2):
                
                nom_3 = nom_3_left if k3 == 0 else nom_3_right 
                try:
                    nom += nom_1*nom_2*nom_3*value[j1+k1,j2+k2,j3+k3]#####my modification
                except Exception:
                    nom += nom_1*nom_2*nom_3*value[j1+k1][j2+k2][j3+k3]

    return nom/denom

#@njit(double(double[:],double[:],double[:],double[:,:,:],double,double,double),fastmath=True)
def interp_3d(grid1,grid2,grid3,value,xi1,xi2,xi3):
    """ 3d interpolation for one point
        
    Args:
        grid1 (numpy.ndarray): 1d grid
        grid2 (numpy.ndarray): 1d grid
        grid3 (numpy.ndarray): 1d grid
        value (numpy.ndarray): value array (3d)
        xi1 (double): input point
        xi2 (double): input point
        xi3 (double): input point
    Returns:
        yi (double): output
    """

    # a. search in each dimension
    j1 = binary_search(0,grid1.size,grid1,xi1)
    j2 = binary_search(0,grid2.size,grid2,xi2)
    j3 = binary_search(0,grid3.size,grid3,xi3)

    return _interp_3d(grid1,grid2,grid3,value,xi1,xi2,xi3,j1,j2,j3)



# #@njit(void(double[:],double[:],double[:],double[:,:,:],double[:],double[:],double[:],double[:]),fastmath=True)
# def interp_3d_vec(grid1,grid2,grid3,value,xi1,xi2,xi3,yi):
#     """ 3d interpolation for vector of points
        
#     Args:
#         grid1 (numpy.ndarray): 1d grid
#         grid2 (numpy.ndarray): 1d grid
#         grid3 (numpy.ndarray): 1d grid
#         value (numpy.ndarray): value array (3d)
#         xi1 (numpy.ndarray): input vector
#         xi2 (numpy.ndarray): input vector
#         xi3 (numpy.ndarray): input vector
#         yi (numpy.ndarray): output vector
#     """

#     for i in range(yi.size):
#         yi[i] = interp_3d(grid1,grid2,grid3,value,xi1[i],xi2[i],xi3[i])
        
        
        
# interp_3d_vec(S_orig,FE_pos,INNO_pos,S_pol,S_orig[0],FE_pos[0],INNO_pos[0],yi)

# #@njit(int32[:](double[:],double[:],double,double,int32),fastmath=True)
# def interp_3d_prep(grid1,grid2,xi1,xi2,Nyi):
#     """ preperation for 3d interpolation of only last dimension
        
#     Args:
#         grid1 (numpy.ndarray): 1d grid
#         grid2 (numpy.ndarray): 1d grid
#         value (numpy.ndarray): value array (3d)
#         xi1 (double): input point
#         xi2 (double): input point
#         Nyi (int): number of points to be evaluated
#     Returns:
#         prep (numpy.ndarray): information for remaining operations
#     """

#     # a. search in non-last dimensions
#     j1 = binary_search(0,grid1.size,grid1,xi1)
#     j2 = binary_search(0,grid2.size,grid2,xi2)
    
#     # b. prep
#     prep = np.zeros((2+Nyi,),dtype=np.int32)
#     prep[Nyi+0] = j1
#     prep[Nyi+1] = j2

#     return prep
# interp_3d_prep(grid1,grid2,xi1,xi2,2500)
# #@njit(double(int32[:],double[:],double[:],double[:],double[:,:,:],double,double,double),fastmath=True)
# def interp_3d_only_last(prep,grid1,grid2,grid3,value,xi1,xi2,xi3):
#     """ 3d interpolation of only last dimension
        
#     Args:
#         prep (numpy.ndarray): information for remaining operations
#         grid1 (numpy.ndarray): 1d grid
#         grid2 (numpy.ndarray): 1d grid
#         grid3 (numpy.ndarray): 1d grid
#         value (numpy.ndarray): value array (3d)
#         xi1 (double): input point
#         xi2 (double): input point
#         xi3 (double): input point
#     Returns:
#         yi (double): output
#     """

#     # a. search in last dimension
#     j1 = prep[0]
#     j2 = prep[1]
#     j3 = binary_search(0,grid3.size,grid3,xi3)
    
#     return _interp_3d(grid1,grid2,grid3,value,xi1,xi2,xi3,j1,j2,j3)

# #@njit(void(int32[:],double[:],double[:],double[:],double[:,:,:],double,double,double[:],double[:],boolean,boolean),fastmath=True)
# def _interp_3d_only_last_vec(prep,grid1,grid2,grid3,value,xi1,xi2,xi3,yi,monotone,search):
#     """ 3d interpolation for vector of points
        
#     Args:
#         prep (numpy.ndarray): information for remaining operations
#         grid1 (numpy.ndarray): 1d grid
#         grid2 (numpy.ndarray): 1d grid
#         grid3 (numpy.ndarray): 1d grid
#         value (numpy.ndarray): value array (3d)
#         xi1 (double): input point
#         xi2 (double): input point
#         xi3 (numpy.ndarray): input vector
#         yi (numpy.ndarray): output vector
#         monotone (bool): indicator for whether xi3 is monotone
#         search (bool): indicator for whether search is needed at all
#     """

#     # a. search in last dimension
#     Nyi = yi.size
#     j1 = prep[Nyi + 0]
#     j2 = prep[Nyi + 1]    
#     if search:
#         for i in range(Nyi):
#             if monotone and i > 0:
#                 j3 = prep[i-1]
#                 while xi3[i] >= grid3[j3+1] and j3 < grid3.size-2:
#                     j3 += 1
#                 prep[i] = j3
#             else:
#                 prep[i] = binary_search(0,grid3.size,grid3,xi3[i])

#     # b. initialize
#     for i in range(yi.size):
#         yi[i] = 0.0

#     # c. interpolation
#     nom_1_left = grid1[j1+1]-xi1
#     nom_1_right = xi1-grid1[j1]

#     nom_2_left = grid2[j2+1]-xi2
#     nom_2_right = xi2-grid2[j2]
    
#     denom = (grid1[j1+1]-grid1[j1])*(grid2[j2+1]-grid2[j2])
#     for k1 in range(2):
#         nom_1 = nom_1_left if k1 == 0 else nom_1_right
#         for k2 in range(2):
#             nom_2 = nom_2_left if k2 == 0 else nom_2_right      
#             for i in range(yi.size):
#                 for k3 in range(2):
#                     j3 = prep[i]
#                     nom_3 = grid3[j3+1]-xi3[i] if k3 == 0 else xi3[i]-grid3[j3]            
#                     yi[i] += nom_1*nom_2*nom_3*value[j1+k1,j2+k2,j3+k3]

#     for i in range(Nyi):
#         j3 = prep[i]
#         yi[i] /= denom*(grid3[j3+1]-grid3[j3])

# #@njit(void(int32[:],double[:],double[:],double[:],double[:,:,:],double,double,double[:],double[:]),fastmath=True)
# def interp_3d_only_last_vec(prep,grid1,grid2,grid3,value,xi1,xi2,xi3,yi):
#     """ 3d interpolation for vector of points
        
#     Args:
#         prep (numpy.ndarray): information for remaining operations
#         grid1 (numpy.ndarray): 1d grid
#         grid2 (numpy.ndarray): 1d grid
#         grid3 (numpy.ndarray): 1d grid
#         value (numpy.ndarray): value array (3d)
#         xi1 (double): input point
#         xi2 (double): input point
#         xi3 (numpy.ndarray): input vector
#         yi (numpy.ndarray): output vector
#     """

#     _interp_3d_only_last_vec(prep,grid1,grid2,grid3,value,xi1,xi2,xi3,yi,False,True)

# #@njit(void(int32[:],double[:],double[:],double[:],double[:,:,:],double,double,double[:],double[:]),fastmath=True)
# def interp_3d_only_last_vec_mon(prep,grid1,grid2,grid3,value,xi1,xi2,xi3,yi):
#     """ 3d interpolation for vector of points where xi3 is monotone
        
#     Args:
#         prep (numpy.ndarray): information for remaining operations
#         grid1 (numpy.ndarray): 1d grid
#         grid2 (numpy.ndarray): 1d grid
#         grid3 (numpy.ndarray): 1d grid
#         value (numpy.ndarray): value array (3d)
#         xi1 (double): input point
#         xi2 (double): input point
#         xi3 (numpy.ndarray): input vector
#         yi (numpy.ndarray): output vector
#     """

#     _interp_3d_only_last_vec(prep,grid1,grid2,grid3,value,xi1,xi2,xi3,yi,True,True)    

# #@njit(void(int32[:],double[:],double[:],double[:],double[:,:,:],double,double,double[:],double[:]),fastmath=True)
# def interp_3d_only_last_vec_mon_rep(prep,grid1,grid2,grid3,value,xi1,xi2,xi3,yi):
#     """ 3d interpolation for vector of points where xi3 is monotone and search is not needed
        
#     Args:
#         prep (numpy.ndarray): information for remaining operations
#         grid1 (numpy.ndarray): 1d grid
#         grid2 (numpy.ndarray): 1d grid
#         grid3 (numpy.ndarray): 1d grid
#         value (numpy.ndarray): value array (3d)
#         xi1 (double): input point
#         xi2 (double): input point
#         xi3 (numpy.ndarray): input vector
#         yi (numpy.ndarray): output vector
#     """

#     _interp_3d_only_last_vec(prep,grid1,grid2,grid3,value,xi1,xi2,xi3,yi,True,False) 

In [105]:
# -*- coding: utf-8 -*-
"""
Created on Sun Oct 23 17:37:42 2022

@author: Giorgia
"""

import numpy as np
import numba
from numba import njit, boolean, int32, double, void

def binary_search(imin,Nx,x,xi):

    # a. checks
    if xi <= x[0]:
        return 0
    elif xi >= x[Nx-2]:
        return Nx-2

    # b. binary search
    half = Nx//2
    while half:
        imid = imin + half
        if x[imid] <= xi:
            imin = imid
        Nx -= half
        half = Nx//2

    return imin


#@njit(double(double[:],double[:],double[:],double[:],double[:,:,:,:],double,double,double,double,int32,int32,int32,int32),fastmath=True)
def _interp_4d(grid1,grid2,grid3,grid4,value,xi1,xi2,xi3,xi4,j1,j2,j3,j4):
    """ 4d interpolation for one point with known location
        
    Args:
        grid1 (numpy.ndarray): 1d grid
        grid2 (numpy.ndarray): 1d grid
        grid3 (numpy.ndarray): 1d grid
        grid4 (numpy.ndarray): 1d grid
        value (numpy.ndarray): value array (4d)
        xi1 (double): input point
        xi2 (double): input point
        xi3 (double): input point
        xi4 (double): input point
        j1 (int): location in grid 
        j2 (int): location in grid
        j3 (int): location in grid
        j4 (int): location in grid
    Returns:
        yi (double): output
    """

    # a. left/right
    nom_1_left = grid1[j1+1]-xi1
    nom_1_right = xi1-grid1[j1]

    nom_2_left = grid2[j2+1]-xi2
    nom_2_right = xi2-grid2[j2]

    nom_3_left = grid3[j3+1]-xi3
    nom_3_right = xi3-grid3[j3]

    nom_4_left = grid4[j4+1]-xi4
    nom_4_right = xi4-grid4[j4]

    # b. interpolation
    denom = (grid1[j1+1]-grid1[j1])*(grid2[j2+1]-grid2[j2])*(grid3[j3+1]-grid3[j3])*(grid4[j4+1]-grid4[j4])
    nom = 0
    for k1 in range(2):
        nom_1 = nom_1_left if k1 == 0 else nom_1_right
        for k2 in range(2):
            nom_2 = nom_2_left if k2 == 0 else nom_2_right       
            for k3 in range(2):
                nom_3 = nom_3_left if k3 == 0 else nom_3_right  
                for k4 in range(2):
                    nom_4 = nom_4_left if k4 == 0 else nom_4_right
                    try:
                        nom += nom_1*nom_2*nom_3*nom_4*value[j1+k1,j2+k2,j3+k3,j4+k4]
                    except Exception:
                        nom += nom_1*nom_2*nom_3*nom_4*value[j1+k1][j2+k2][j3+k3][j4+k4]

    return nom/denom

#@njit(double(double[:],double[:],double[:],double[:],double[:,:,:,:],double,double,double,double),fastmath=True)
#@numba.njit('List(int64)(ListType(float64[:, ::1]))')
def interp_4d(grid1,grid2,grid3,grid4,value,xi1,xi2,xi3,xi4):
    """ 4d interpolation for one point
        
    Args:
        grid1 (numpy.ndarray): 1d grid
        grid2 (numpy.ndarray): 1d grid
        grid3 (numpy.ndarray): 1d grid
        grid4 (numpy.ndarray): 1d grid
        value (numpy.ndarray): value array (4d)
        xi1 (double): input point
        xi2 (double): input point
        xi3 (double): input point
        xi4 (double): input point
    Returns:
        yi (double): output
    """

    # a. search in each dimension
    j1 = binary_search(0,grid1.size,grid1,xi1)
    j2 = binary_search(0,grid2.size,grid2,xi2)
    j3 = binary_search(0,grid3.size,grid3,xi3)
    j4 = binary_search(0,grid4.size,grid4,xi4)

    return _interp_4d(grid1,grid2,grid3,grid4,value,xi1,xi2,xi3,xi4,j1,j2,j3,j4)

In [104]:
# -*- coding: utf-8 -*-
"""
Created on Sun Oct 23 17:37:42 2022

@author: Giorgia
"""

import numpy as np
import numba
from numba import njit, boolean, int32, double, void

def binary_search(imin,Nx,x,xi):

    # a. checks
    if xi <= x[0]:
        return 0
    elif xi >= x[Nx-2]:
        return Nx-2

    # b. binary search
    half = Nx//2
    while half:
        imid = imin + half
        if x[imid] <= xi:
            imin = imid
        Nx -= half
        half = Nx//2

    return imin


#@njit(double(double[:],double[:],double[:],double[:],double[:,:,:,:],double,double,double,double,int32,int32,int32,int32),fastmath=True)
def _interp_4d(grid1,grid2,grid3,grid4,value,xi1,xi2,xi3,xi4,j1,j2,j3,j4):
    """ 4d interpolation for one point with known location
        
    Args:
        grid1 (numpy.ndarray): 1d grid
        grid2 (numpy.ndarray): 1d grid
        grid3 (numpy.ndarray): 1d grid
        grid4 (numpy.ndarray): 1d grid
        value (numpy.ndarray): value array (4d)
        xi1 (double): input point
        xi2 (double): input point
        xi3 (double): input point
        xi4 (double): input point
        j1 (int): location in grid 
        j2 (int): location in grid
        j3 (int): location in grid
        j4 (int): location in grid
    Returns:
        yi (double): output
    """

    # a. left/right
    nom_1_left = grid1[j1+1]-xi1
    nom_1_right = xi1-grid1[j1]

    nom_2_left = grid2[j2+1]-xi2
    nom_2_right = xi2-grid2[j2]

    nom_3_left = grid3[j3+1]-xi3
    nom_3_right = xi3-grid3[j3]

    nom_4_left = grid4[j4+1]-xi4
    nom_4_right = xi4-grid4[j4]

    # b. interpolation
    denom = (grid1[j1+1]-grid1[j1])*(grid2[j2+1]-grid2[j2])*(grid3[j3+1]-grid3[j3])*(grid4[j4+1]-grid4[j4])
    nom = 0
    for k1 in range(2):
        nom_1 = nom_1_left if k1 == 0 else nom_1_right
        for k2 in range(2):
            nom_2 = nom_2_left if k2 == 0 else nom_2_right       
            for k3 in range(2):
                nom_3 = nom_3_left if k3 == 0 else nom_3_right  
                for k4 in range(2):
                    nom_4 = nom_4_left if k4 == 0 else nom_4_right
                    try:
                        nom += nom_1*nom_2*nom_3*nom_4*value[j1+k1,j2+k2,j3+k3,j4+k4]
                    except Exception:
                        nom += nom_1*nom_2*nom_3*nom_4*value[j1+k1][j2+k2][j3+k3][j4+k4]

    return nom/denom

#@njit(double(double[:],double[:],double[:],double[:],double[:,:,:,:],double,double,double,double),fastmath=True)
#@numba.njit('List(int64)(ListType(float64[:, ::1]))')
def interp_4d(grid1,grid2,grid3,grid4,value,xi1,xi2,xi3,xi4):
    """ 4d interpolation for one point
        
    Args:
        grid1 (numpy.ndarray): 1d grid
        grid2 (numpy.ndarray): 1d grid
        grid3 (numpy.ndarray): 1d grid
        grid4 (numpy.ndarray): 1d grid
        value (numpy.ndarray): value array (4d)
        xi1 (double): input point
        xi2 (double): input point
        xi3 (double): input point
        xi4 (double): input point
    Returns:
        yi (double): output
    """

    # a. search in each dimension
    j1 = binary_search(0,grid1.size,grid1,xi1)
    j2 = binary_search(0,grid2.size,grid2,xi2)
    j3 = binary_search(0,grid3.size,grid3,xi3)
    j4 = binary_search(0,grid4.size,grid4,xi4)

    return _interp_4d(grid1,grid2,grid3,grid4,value,xi1,xi2,xi3,xi4,j1,j2,j3,j4)

In [103]:
"""
Beguinning!
"""
import time
import itertools as it
import numpy as np
from scipy import optimize
from scipy import stats
from numpy import *
from statistics import NormalDist
import scipy.interpolate as spi
from scipy.io import loadmat
import os
from types import SimpleNamespace
import numpy as np

import types, copy


model = SimpleNamespace() #model
model.par = SimpleNamespace() #param
model.sol = SimpleNamespace() 
model.options = SimpleNamespace()
model.inc = SimpleNamespace()
model.par.inc = SimpleNamespace()
model.par_temp = SimpleNamespace()
model.guess = SimpleNamespace()
model.options.guess = SimpleNamespace()
model.pol = SimpleNamespace()
model.flags = SimpleNamespace()
model.par2 = SimpleNamespace()
model.pol_dense = SimpleNamespace()

options = model.options
par = model.par
par_temp = model.par_temp
par_est = model.par
par2 = model.par2
inc = model.inc
par.inc = model.par.inc
guess = model.guess
options.guess = model.options.guess 
pol = model.pol
flags = model.flags
pol_dense = model.pol_dense 
## Options

options.exerciseSS_On   = 'N'         # Default = N; if = Y: solve SS exercise (different wage and savings options)
options.Fertility       = 'Endo'       # 'Endo': endogenous fertility; 'Exo': constant fertility
options.Fertility_Exo_N = 2.15       # 'Exo' case: number of children
options.ParentTrans     = 'Endo'      # Endo: Benchmark case with parents transfers; 'Exo': exogenously given
options.ParentTransVal  = 0.05         # Value for exogenous trasnfers
options.ExPolRetirement = 'N'         # N: benchmark case; Y: Exercise Retirement  icy
options.GridH0orig      = 'N'        # Force initial grid of H0 -- To use when doing sensitivity analysis of parameters
options.patient         = 'N'         # 'N': Baseline Discount Factor; 'Y': Higher beta (lower discounting)
options.noborrowingwedge= 'N'         # 'N': Baseline Wedge between borrowing and lending; 'Y': Smaller wedge
options.fert_transfer   = 'N'
options.init_transfer   = 'N'
options.psy_avg         = 'N'
options.AdultRisk       = 'Y' #from start
options.timer_on    	= 'N';          # display timer
options.start_Calibrate = 'N'
options.AdultRisk       = 'Y'
options.Ck              = 'Yes'
options.maxiter         = 25
options.tolV            = 1e-3
options.solve_h0        = 'Y'

#inc.age_prof = cell(length(EDUC),1); #from file par_income_process
#inc.fe_pos   = np.arange(1,N_fe,N_fe)





x = np.matrix([[0, 0.24776039834302887588],
              [1, 0.75658886515041290366],  
              [2,  0.25397448399310357248],
              [3, 25.71397930469017012456],
              [4, 1.7323110070687675055],
              [5, 3.9825431046569619298],
              [6, 0.55750684703128172703],
              [7, 0.23634828161700616178],
              [8, 0.16301011354749064819],
              [9, 0.10918570913971439862],
              [10, 0.66823638808333596373],
              [11, 0.6646077270329715514],
              [12, 0.18544353931303594885],
              [13, 0.47056974917650218337]])
                   

x_val = np.array([ 0.24776039834302887588,
             0.75658886515041290366,
             0.25397448399310357248,
             25.71397930469017012456,
             1.7323110070687675055,
             3.9825431046569619298,
             0.55750684703128172703,
             0.23634828161700616178,
             0.16301011354749064819,
             0.10918570913971439862,
             0.66823638808333596373,
             0.6646077270329715514,
             0.18544353931303594885,
             0.47056974917650218337])


par_est.gamman      = x[0,1]
par_est.ln_level    = x[1,1]
par_est.mu          = 0.5
par_est.sigmah0     = x[2,1]
par_est.psy_max     = x[3,1]
par_est.psy_cor     = x[4,1]
par_est.psy_hs      = x[5,1]
par_est.R_hs_curv   = x[6,1]
par_est.R_col_curv  = x[7,1]
par_est.R_hs_level  = x[8,1]
par_est.R_col_level = x[9,1]
par_est.Nanny_oppC  = 1
par_est.Child_Ccurv = 0.6445
# par_est.Child_Ccurv = 1.000
par_est.Child_C_Tot = x[10,1] #Multiply by (1-tax) in child cost function to obtain parameter from paper
par_est.child_cost_inc_curv = x[11,1]
par_est.beta_inc    = x[12,1]
#if options.solve_h0 = 'N'
par_est.meanh0      = x[13,1]
#par.meanh0 = 2
#par.sigmah0 = 0.5

##############  PROBNLEMS:
    #parameters:  par.PG becasue of lognormcdf
                # options.init_trans_size
    #prob_workT:  C[inno,educ,not_feasible,ife] = 0
                # V[inno,educ,not_feasible,ife] = (-(10**(5/gammac))**(1-gammac))/(1-gammac),
                # S[inno,educ,not_feasible,ife] seems to work but would also work if Sp[inno,educ,not_feasible,ife]= Spgrid[0] was not commented


In [102]:
# -*- coding: utf-8 -*-
"""
Created on Sat Aug  6 14:06:12 2022

@author: Giorgia
"""
from types import SimpleNamespace
import time
import itertools as it
import numpy as np
from scipy import optimize
from scipy import stats
from __future__ import print_function
from numpy import *
from statistics import NormalDist



options.AdultRisk       = 'Y';



def discretize_nonstationary_ar_rouwen(rho, sigma_z_l,sigma_z_t, n):
 #Rouwen method to discretize AR(1), non-stationary, process w symmetric innovations
 # Based on WP Fella, Gallipoli and Pan (2015)
 # It matches persistence of the original process exactly
    
    q = 1/2 * (1+rho*sigma_z_l/sigma_z_t)
    nu = sqrt(n-1) * sigma_z_t

    P = [[q, 1-q],[1-q, q]]

    for i in range (2,n):
        P = np.dot(q,[np.append(np.append(np.reshape(P,[i,i]), np.zeros([i,1]),axis = 1), np.zeros([1,i+1]),  axis = 0)]) + np.dot((1-q),[np.append(np.append(np.zeros([i,1]), np.reshape(P,[i,i]), axis = 1),np.zeros([1,i+1]), axis = 0)]) + np.dot((1-q),[np.append(np.zeros([1,i+1]), np.append(np.reshape(P,[i,i]), np.zeros([i,1]), axis = 1), axis = 0)]) + np.dot(q,[np.append(np.zeros([1,i+1]), np.append(np.zeros([i,1]), np.reshape(P,[i,i]), axis = 1), axis = 0)])
        P = np.reshape(P, [i+1,i+1])
        P[1:i,:] = P[1:i,:]/2

    zgrid = linspace(-nu, nu, n).T
    
    return   P, zgrid



In [101]:
# -*- coding: utf-8 -*-
"""
Created on Tue Aug  9 17:28:23 2022

@author: Giorgia
"""

# -*- coding: utf-8 -*-
"""
Created on Tue Aug  2 18:23:35 2022

@author: Giorgia
"""
from types import SimpleNamespace
import time
import itertools as it
import numpy as np
from scipy import optimize
from scipy import stats
from __future__ import print_function
from numpy import *
from statistics import NormalDist



options.AdultRisk       = 'Y' #from start


def par_income_process(par,options):
## Income Process
# Based on Abbott, Gallipoli, Meghir, Violante (2013). Updated.
# agent i, education e = 1,2,3, age a:
# Earnings_i,e,a = h_e,a gamma_e,i eta_e,a,i
# h_e,a:     age profile of efficency units of labor by education group
# gamma_e,i: fixed effect
# eta_e,a,i: idiosyncratic shock, AR(1)
# clear; clc; close all; 



############################################################################### MAY BE UNNECESSARY
    # par.time_period     = 2
    # par.age             = np.linspace(0,80,41)
    # par.NJ              = len(par.age);
    # par.Je1             = 16                                    # HS age
    # par.Je1_pos         = list(par.age).index(par.Je1)+1        #np.where(par.age == par.Je1)
    # par.Je2             = 18                                    #College age
    # par.Je2_pos         = list(par.age).index(par.Je2)+1 
    # par.Jc              = 28                                    #Fertility
    # par.Jc_pos          = list(par.age).index(par.Jc)+1 
    # par.Jt              = 40                                    #OAS
    # par.Jt_pos          = list(par.age).index(par.Jt)+1
    # par.Jr              = 66                                    #Retirement
    # par.Jr_pos          =  list(par.age).index(par.Jr)+1
    # par.Jd              = 80                                    #Die, source: WB - life expectancy = 79
    # par.Jd_pos          =  list(par.age).index(par.Jd)+1
    
    par.educ = [1,2,3]
    EDUC         = par.educ
    par.time_period     = 2
    par.age             = np.linspace(0,80,41)
    age          = par.age[0:par.Jr_pos]
    par.N_markov = 5
    N_markov     = par.N_markov
    N_fe         = par.N_fe = 5
    inc.inno_pos = list(range(0,N_markov))
    inc.fe_pos   = range(0,N_fe)
    
    age_start       = [par.Je1_pos-1, par.Je2_pos-1, par.Je2_pos+1]
########################################################################################
    ## 1. age profile of efficency units of labor by education group: h_e,a
    # Source: PSID
    def age_prof(e,a):
        if e==0:
            return 1*(0.0507705*a-0.0005522*a**2)
        elif e==1:
            return 1*(0.0668012*a-0.0007312*a**2)
        elif e==2:
            return 1*(0.1221627*a-0.0013147*a**2)



    inc.age_prof = {}
    for i in range(0,len(EDUC)):
        inc.age_prof[i] = [[]]
        

    for ie in range(0,len(EDUC)):
        inc.age_prof[ie][0]      = np.zeros(par.Jr_pos)
        for ia in range (par.Je1_pos-1,par.Jr_pos):
            inc.age_prof[ie][0][ia] = age_prof(ie,age[ia])
        
        inc.age_prof[ie][0][range(par.Je1_pos-1,par.Jr_pos)]     = inc.age_prof[ie][0][range(par.Je1_pos-1,par.Jr_pos)] - inc.age_prof[ie][0][par.Je1_pos-1]


                 
       ## 2. Idiosyncratic shock: eta_e,a,i 
       # Source: NLSY79
    inc.inno_rho   = [0.875153, 0.961399, 0.966375] # autocorrelation of innovation for education groups
    inc.z0_var      = [0.22832, 0.10134, 0.0660616] # Variance of iid shocks to innovation for education groups
    inc.inno_var    = [0.0624742, 0.02341, 0.0291241]   # Variance of iid shocks to innovation for education groups
    if options.AdultRisk == 'N':
       inc.inno_var = 1e-6*inc.inno_var
       inc.z0_var   = 1e-6*inc.z0_var

    inc.z_var       = np.zeros([len(EDUC),len(age)])
    inc.z_var = reshape(inc.z_var,[len(EDUC),len(age)])
    
    for ie in range (0,len(EDUC)):
        rho      = inc.inno_rho[ie]
        z0_var   = inc.z0_var[ie]
        inno_var = inc.inno_var[ie]
        inc.z_var[ie,age_start[ie]] = z0_var
        for ia in range (age_start[ie]+1,par.Jr_pos):
            inc.z_var[ie,ia] = rho**2 * inc.z_var[ie, ia-1] + inno_var


    # aux_inc.inno_var     = zeros(length(inc.educ),length(age));
    inc.z_val        = {}
    for i in range(0,len(EDUC)):
        inc.z_val[i] = [[]]
    
    inc.z_prob    = {}
    for i in range(0,len(EDUC)):
         
         inc.z_prob[i] = [[]]
    # for i in range(0,1):
    #     a = [0]
    #     inc.z_prob[i] = [[a]*len(EDUC)]
    
    def normcdf(x,mu,sigma):
        result = np.zeros(len(x))
        for i in range(0,len(x)):
            result[i] = NormalDist(mu,sigma).cdf(x[i])
        return result
    
    
    
    # Z0 PROBABILITIES (AND VALUES) 
    for ie in range (0,len(EDUC)): 
        inc.z_val[ie]        = np.zeros([par.Jr_pos,N_markov])
        inc.z_prob[ie]     = np.repeat(np.zeros([par.Jr_pos,N_markov]),N_markov)
        inc.z_prob[ie] = np.reshape(inc.z_prob[ie],[N_markov,par.Jr_pos,N_markov])
                                 
        # Initial draw of z
        ia                      = age_start[ie]
        nu                      = sqrt(N_markov-1) * inc.z_var[ie,ia]**0.5
        etagrid                 = np.linspace(-nu, nu, N_markov).T
        inc.z_val[ie][ia,:]     = etagrid
   
        grid_med                 = 0.5*(etagrid[1:]+etagrid[0:-1])
        aux_cdf                  = normcdf(grid_med,0,inc.z_var[ie,ia]**0.5)
        aux_pdf                  = np.append(aux_cdf[0], [aux_cdf[1:]-aux_cdf[0:-1]])
        aux_pdf                  = np.append(aux_pdf, 1-sum(aux_pdf))
        inc.z_prob[ie][:,ia,:] = np.reshape(np.repeat(aux_pdf,N_markov),[5,5])
        
        for ia in range (age_start[ie]+1,par.Jr_pos):
            ############################################################
            P, etagrid  =  discretize_nonstationary_ar_rouwen(inc.inno_rho[ie],inc.z_var[ie,ia-1]**0.5,inc.z_var[ie,ia]**0.5, N_markov)           
            print(P)
            #inc_z_prob[ie][:,ia,:],inc_z_val[ie][ia,:] = discretize_nonstationary_ar_rouwen(inc_inno_rho[ie],inc_z_var[ie,ia-1]**0.5,inc_z_var[ie,ia]**0.5, N_markov)
            inc.z_prob[ie][:,ia,:] = P.T
            inc.z_val[ie][ia,:] = etagrid
            
    return inc
##need to create a SimpleNameSpace first



In [111]:
# -*- coding: utf-8 -*-
"""
Created on Sun Aug  7 13:52:12 2022

@author: Giorgia
"""

# -*- coding: utf-8 -*-
"""
Created on Tue Jul 26 12:07:27 2022

@author: Giorgia
"""
from types import SimpleNamespace
import time
import itertools as it
import numpy as np
from scipy import optimize
from scipy import stats
from __future__ import print_function
from numpy import *




def parameters(par_est,options):
    par = model.par
    par_est = model.par
## 1. Mean household income (unconditional of educ groups) in 2000 dollars
# * Source PSID, 2000 % 
    par.mean_inc        = 70179.45
    mean_inc_age42      = 75630                        # mean income at age 40-43 (to compute OAS transfers)

    options.exerciseSS_On  = 'N'         # Default = N; if = Y: solve SS exercise (different wage and savings options)
# Option to change w for SS exercises
    if options.exerciseSS_On == 'N':
        par.w       = 1
    else:
        par.w       = options.exerciseSS.w_ss

###############
    ## 2. Age structuretime_period     = 2;
    par.time_period     = 2
    par.age             = np.linspace(0,80,41)
    par.NJ              = len(par.age);
    par.Je1             = 16                                    # HS age
    par.Je1_pos         = list(par.age).index(par.Je1)+1        #np.where(par.age == par.Je1)
    par.Je2             = 18                                    #College age
    par.Je2_pos         = list(par.age).index(par.Je2)+1 
    par.Jc              = 28                                    #Fertility
    par.Jc_pos          = list(par.age).index(par.Jc)+1 
    par.Jt              = 40                                    #OAS
    par.Jt_pos          = list(par.age).index(par.Jt)+1
    par.Jr              = 66                                    #Retirement
    par.Jr_pos          =  list(par.age).index(par.Jr)+1
    par.Jd              = 80                                    #Die, source: WB - life expectancy = 79
    par.Jd_pos          =  list(par.age).index(par.Jd)+1

    

    ## 3. Income process
    par.educ     = np.array([1, 2, 3])
    par.dist_educ = np.array([0.094, 0.601, (1-.601-.094)])
    par.N_markov = 5
    par.N_fe     = 5
    par.sigmah0  = par_est.sigmah0 ##par_est.sigmah0     = x(3), x from function "calibrate"
    par.inc      = par_income_process(par,options) ##par_income_process fucntion defined in file par_income process
    if 1 == 0:
    # To calculate the persistence of the income process: XXX Need to Fix XXX
        solve_corr_model()### must define

    ##Education Returns
    par.Reducs      = np.zeros([2,2]) #Education stage, [constant, curvature]: h_(t+1) = h_t^curv * constant
    par.Reducs[0,:] = [par_est.R_hs_level, par_est.R_hs_curv] ## from calibratewith input x: par_est.R_hs_curv   = x(7); par_est.R_col_curv  = x(8); par_est.R_hs_level  = x(9); par_est.R_col_level = x(10); 
    par.Reducs[1,:] = [par_est.R_col_level, par_est.R_col_curv]
    
## 4. Initial H0 
########################################################################
# Distribution of parents: By Education 
###################################################################
    par.beta_inc        = par_est.beta_inc ## = x(13) from calibrate
    par.sigmah0         = par_est.sigmah0 ## x(3)
    par.meanh0          = par_est.meanh0  ##output of a loop in calibrate file and in par_grid_h0_old

    pp                  = np.linspace(0.025,0.975,par.N_fe)   
    
    def lognorminv(x,mu,sigma):
        lognorm = stats.lognorm(sigma, scale=math.exp(mu))
        lognorm.ppf(x)
        return lognorm.ppf(x)
    
    if options.GridH0orig == 'N':
        sigma_grid          = 1.5*par_est.sigmah0 #Larger grid of H0 so that it includes more points if beta_inc is large.
    else:
        sigma_grid          = 1.5*options.SigmaH0orig # Larger grid of H0 so that it includes more points if beta_inc is large.

    par.H0                  = lognorminv(pp,log(par.meanh0),sigma_grid) # eturns the inverse of the lognormal cdf with the distribution parameters mu (mean of logarithmic values) and sigma (standard deviation of logarithmic values), evaluated at the probability values in p.


#     def lognormcdf(x,mu,sigma):
#         for i in range(0,len(x_val)):
#             #for j in range(0,len(mu)):
#             lognorm = stats.lognorm(sigma, scale=math.exp(mu))
#             res = []
#             res = res.append(lognorm.cdf(x_val[i]))
#             i += 1
#             res_final = []
#             res_final = res_final.append(lognorm.cdf(x_val[i]))
#             return res_final
        
        
#     mat = np.zeros([len(mu),len(x)])  
#     def lognormcdf(x,mu,sigma):
#         for i in range(0,len(mu)):
#             mat[i,:] = mu[i]
#             for j in range (0, len(x)):
#                 mat[:,j] = x[j]
#                 return mat
            
# ####need to translate into function


            
        #rr = stats.lognorm(1, scale=math.exp(2)).cdf(x_val)    
        
        

    # def lognormcdf(x_val,mu,sigma):
    #     #x_val = np.reshape(x_val,[1,len(x_val)])
    #     mat = np.zeros([len(mu),len(x_val)])
    #     for j in range(0,len(mu)):
    #             for i in range(0,len(x_val)):
    #                 lognorm = stats.lognorm(sigma, scale=math.exp(mu[j]))
    #                 mat[j,i] = lognorm.cdf(x_val[i])
    #                 return mat

######must be fixed
 #in the same code in  matlab they transpose the array of x so it is difficult to compare

###################################################################################
    #par.PG =  @(x) np.array([logncdf(np.reshape(np.append(np.array(par.H0[0:-1]),inf),[1,len(np.append(np.array(par.H0[0:-1]),inf))]),log(par.meanh0) + par_est.beta_inc * log(x_val),par_est.sigmah0) - lognormcdf(np.append(0, np.array(par.H0[0:-1])),log(par.meanh0) + par_est.beta_inc * log(x),par_est.sigmah0)]).T ##need to understand x()
    def PG(x):
        par.PG = np.squeeze(np.array([logncdf(np.append(np.array(par.H0[0:-1]),inf),log(par.meanh0) + par_est.beta_inc * log(x),par_est.sigmah0) - logncdf(np.append(0, np.array(par.H0[0:-1])),log(par.meanh0) + par_est.beta_inc * log(x),par_est.sigmah0)])).T
        return par.PG
    
    ##need to understand x()
        
########################################################################################                          
    # Transformed by education##
    ########### WARNING!!  par.inc.fe is a cell array...how to translate it?
    par.H0               = par.H0.T ### is it actually the right shape?
    H0                   = par.H0
    
    EDUC = par.educ
    par.inc.fe           = {}
    for i in range (0,len(EDUC)):
        par.inc.fe [i] = []
    par.inc.fe [0]      = log(H0) 
    par.inc.fe [0][3]   = -0.511650093072249179115829065267                                           #HS Drop
    par.inc.fe [1]      = log(   H0   + par.Reducs[0,0] * H0**par.Reducs[0,1])   #HS Grad
    par.inc.fe [2]      = log(   H0   + par.Reducs[1,0] * H0**par.Reducs[1,1])   #Col Grad

    ## Model-Data exchange rate: use mean income at age 40-43 for HS graduates, from average initial h0
    mean_inc_data = par.time_period*mean_inc_age42
    mean_inc_model = par.time_period # Solve to get h0 at age 42 = 2, 1 per year
    par.p_model_data = mean_inc_model / mean_inc_data
    
    ## 6. Preferences:
    # 6.1 Estimated from outside the model
    beta_annual = 0.975
    #options.patient = 'Y'
    if options.patient == 'Y':
        beta_annual = 0.99
    
    par.beta = beta_annual**par.time_period
    par.gammac = 0.5
    
    # 6.2 Estimated internally, altruism: b(n) = par.ln_level * n^(par.gamman)
    par.gamman = par_est.gamman # grid_gamman = [0.01, 0.8]
    par.lambdan = par_est.ln_level # grid_in = [0.01, 0.8]
    
    # 6.3 Education costs
    # 6.3.1 College: externally
    # Source: Delta Cost Project, average cost 2000 = 6588 per year.
    # In the model: 4 years of college and household ( two agents)
    par.pe2 = par.time_period * 2* 6588 * par.p_model_data
    # 6.3.2 High School: internally
    par.pe1 = par.pe2*0.09
    
    
    #options.exerciseSS_On = 'Y'
    options.exerciseSS_change_pe = 'Y'
    if options.exerciseSS_On == 'Y':
        if options.exerciseSS_change_pe == 'Y':
            par.pe2 = par.pe2*options.exerciseSS_p_mult
            par.pe1 = par.pe1*options.exerciseSS_p_mult
            
    pe2_total = (4/par.time_period)*par.pe2
    pe1_total = (4/par.time_period)*par.pe1
    
    # 6.4 Psychic cost: relative cost of education
    # par_est.psy         = 5;
    # par_est.psy         = 20;
    gridpsy = 100 # multiple of 5
    # par.psy_val_hs      = (linspace(0,par_est.psy_max,gridpsy).*par.pe1*par_est.psy_hs)';
    
    if options.exerciseSS_On == 'Y':
        if options.exercizeSS_change_psy == 'Y':
            par_est.psy_hs = par_est.psy_hs * options.exerciseSS_w_ss**options.psy_adj
            par_est.psy_max = par_est.psy_max * options.exerciseSS_w_ss**options.psy_adj
   
    par.psy_val_hs      = (np.linspace(0,par_est.psy_hs,gridpsy)).T
    par.psy_val_col     = (np.linspace(0,par_est.psy_max,gridpsy)).T
    # par.psy_val         = (linspace(0,par_est.sigmapsy,gridpsy).*par.pe2)';
    # ini                 = 100/(gridpsy) *.5;
    # psy_cdf             = linspace(ini,100-ini,gridpsy)';
    # psy_cdf             = psy_cdf/100;
    # par.psy_val         = norminv(psy_cdf,0,par_est.sigmapsy).*par.pe2;

    par.psy_prob        = np.zeros([3,gridpsy])
    Ngrid               = gridpsy+1
    # HS drop: more prob in higher values
    curv_psy            = par_est.psy_cor
    grid_cdf            = np.linspace(0**(1/curv_psy),1**(1/curv_psy),Ngrid)**curv_psy
    par.psy_prob[0,:]   = grid_cdf[1:]-grid_cdf[0:-1]
    
     # HS grad
    curv_psy            = 1
    grid_cdf            = np.linspace(0**(1/curv_psy),1**(1/curv_psy),Ngrid)**curv_psy
    par.psy_prob[1,:]   = grid_cdf[1:]-grid_cdf[0:-1]

    # CO grad: more prob in lower values
    curv_psy            = par_est.psy_cor
    grid_cdf            = np.linspace(0**(1/curv_psy),1**(1/curv_psy),Ngrid)**curv_psy
    grid_cdf_inv        = grid_cdf[1:]-grid_cdf[0:-1]
    par.psy_prob[2,:]   = grid_cdf_inv[::-1]

    prob_educ_data      = np.array([9.4, 60.1, 30.5])/100
    prob_psy            = np.dot(prob_educ_data,par.psy_prob)
    avg_psy             = np.array([np.dot(prob_psy,par.psy_val_hs), np.dot(prob_psy,par.psy_val_col)])
    
    if options.psy_avg == 'Y':
        gridpsy         = 2
        par.psy_val_hs  = np.array([avg_psy[0]*0.995,avg_psy[0]])
        par.psy_val_col = np.array([avg_psy[1]*0.995,avg_psy[1]])
            
        par.psy_prob    = np.ones([3,gridpsy])/2
            



       
    #6.5 Child cost parameters
    par.Nanny_oppC = par_est.Nanny_oppC
    par.Child_Ccurv = par_est.Child_Ccurv
    par.Child_C_Tot = par_est.Child_C_Tot
    par.child_cost_inc_curv = par_est.child_cost_inc_curv
    
    ##7. Partial Equilibrium prices and taxes
    ## 7. Partial Equilibrium prices and taxes
    ## 7.1 Wages: internally, to match mean income and education shares
    ## 7.2 Interest rate
    ## prime        = 1/beta_annual - 1;
    par.time_period     = 2 ## assumed
    prime        = 3/100; # Smets and Wouters AER 2007
    par.r        = (1+prime)**par.time_period-1
    par.r_sav     = (1+prime)**par.time_period-1            # Roys, Seshadri - 2014 XXX CONSIDER ADDING TAXES
    #par_est.int_iota = 0.1; # XXX Add to estimation? XXX
    par_est.int_iota = 0.1 ###### my modeification from par_est which is part of a more complicated function
    
    #options.noborrowingwedge = 'Y'
    if options.noborrowingwedge == 'Y':
        par_est.int_iota = 0.01
        
    par.r_debt   =  ((1+prime+par_est.int_iota)**par.time_period - 1)
    par.r_col    = ((1+prime+0.00925)**par.time_period - 1); #See /DK/data/Data_US/Education/Loans/College Loans.xls
    par.col_fact = par.r_col/(1-(1+par.r_col)**(-10))*(1-(1+par.r_debt)**(-10))/par.r_debt # Assume debt is repaid in 20 years, i.e. 10 periods.
    
    
    #Borrowing constraint: by education
    borr = par.w * (np.array([10000, 24000, 34000])*par.p_model_data) # self-reported limits on unsecured credit by family type from the SCF. Based on Abbot et al (2016).
    borr_sch = par.w * np.array([0, 0, 2*par.col_fact*23000])* par.p_model_data # Less borrowing if not finished college
    
    # 7.3 Government social security taxes
    if options.ExPolRetirement == 'N':
        par.Lambda = 0.124
    elif options.ExPolRetirement == 'Y':           # Social Security Ppayroll Tax, Krueger, Ludwig - 2015
        par.Lambda = options.PolRetTaxes
      
    par.pn = 0.54 * par.time_period* par.mean_inc*par.p_model_data #price of childcare, source: Folbre 2008, page 129. Wage of child care in 2000 = $7.43, mean wage = 13.74, 7.43/13.74=.54
    par.w_college = 0.56 * par.w # wage at college, eg inc = w_college * w * h; source: IPUMS
    
    if options.exerciseSS_On == 'Y':
        par.pn = par.pn * options.exerciseSS_p_mult ## actually commented in Matlab
        
    gdp_pc = 44308
    par.gov_debt = 0.2*gdp_pc*par.p_model_data
     
    # Source: AGMV 2013
    par.prod_alpha = 0.350
    par.prod_delta = 1-(1-0.0650)**par.time_period
    par.prod_s = np.array([0.160, 0.390, 0.450])
    par.prod_rho = 0.680
     
     ## 9. Grids for savings and discrete choices
     # 8.1 Number of children
     
    par.fam_size = 2
    if options.Fertility == 'Endo': #Endogenous fertility
        par.N = np.arange(0,4)
    elif options.Fertility == 'Exco': #Exogenous fertility
        par.N = options.Fertility_Exo_N/par.fam_size ####### from start_calibrate: options.Fertility_Exo_N = 2.15;         #'Exo' case: number of children
     
     # N = 0: no child
     # N = 1: fam_size children
     # N = 2: 2*fam_size children
     # etc.
     
     # 8.4 Savings
    par.Ls_0 = 20 #Length of savings grid for j = 12
    par.Ls_1 = 80 ##Length of savings grid for j = 16,20,24
     #% par.Ls              = [1 1 1 par.Ls_0 repmat(par.Ls_1,1,3) repmat(par.Ls_2,1,11) repmat(par.Ls_3,1,3)]; 
    par.Ls = np.append(np.append(np.ones(par.Je1_pos-1), par.Ls_0), np.repeat(par.Ls_1,par.Jd_pos - par.Je1_pos))

    curv = 3
        
    par.grids =  {}
    for i in range (0, len(EDUC)):
        a = [0]
        par.grids[i]= [a]*par.Jd_pos
    
        
    
    if options.ParentTrans == 'Endo':
        PHI_max = (pe1_total + pe2_total)*3
        par.PHI = np.append(np.array([0,pe1_total/3, 2/3*pe1_total]), (np.linspace((pe1_total*1.1)**(1/curv),PHI_max**(1/curv),int(par.Ls[par.Je1_pos-1])-3))**curv)        
        
    elif options.ParentTrans == 'Exo':
        # Computation of factor: 
        # 1. solve with factor = 1. 
        # 2. Solve for factor such that (res_MOM(4,100)*factor-res_MOM(1,100))/res_MOM(1,100) = 1
        if options.parentTransExo == 'MeanTrans':
            options.ParentTransVal = 2*30566*par.p_model_data
        elif options.parentTransExo == 'MeanHSCost':
            options.ParentTransVal = pe1_total*1.5
        elif options.parentTransExco == 'MeanColCost':
            options.ParentTransVal = (pe1_total + pe2_total)*1.5
        elif options.parentTransExo == 'search_val':
            options.ParentTransVal = 2*options.ParentTransVal*par.p_model_data
        par.PHI = options.ParentTransVal * np.array([0.999, 1.001])
               
    for ie in range (0,len(par.educ)):
        borr_limits = np.append(np.append(np.append(np.zeros(par.Je2_pos),borr_sch[ie]),np.append(borr_sch[ie],borr[ie]*np.ones(par.Jr_pos-1-(par.Je2_pos + 2)))), np.zeros(par.Jd_pos - (par.Jr_pos-1)))                      
        # j = Je1
        par.grids[ie][par.Je1_pos-1] = par.PHI ##### par.phi is a sequence
        #j = Je2
        maxE = par.PHI[-1]*2;

        for j in range(par.Je1_pos, par.Je2_pos+1):### fits only for j = 8
            #par.grids[j][ie]    = np.linspace(int(-borr_limits[j]**(1/curv)),int(maxE**(1/curv)),int(par.Ls[j]))**curv
            par.grids[ie][j] = np.linspace(-borr_limits[j]**(1/curv),maxE**(1/curv),int(par.Ls[j]))**curv

    # j = Je2+1 : Jc -1 CHECK! IT'S IMPORTANT THAT GRIDS STARTS CHANGING BY EDUCATION ONLY AFTER JE2+1
    
        for j in range(par.Je2_pos+1,par.Jc_pos-1):
            maxWY           = par.PHI[-1]*par.N[-1]*0.25

            max_inc         = par.w*exp(inc.age_prof[ie][0][j-1] + np.max(par.inc.fe[ie][:]) + np.max(inc.z_val[ie][j-1,:]))*2 # par.inc.age_prof to be changed in par_income_process to get rid of [0] as it is defined wrong and we should not need [0]
            maxS            = max(par.grids[ie][j-1][-1]*1.3,max(max_inc,maxWY))
            par.grids[ie][j] = np.linspace(-borr_limits[j]**(1/curv),maxS**(1/curv),int(par.Ls[j]))**curv
    
    # j = Jc: Jr
        for j in range (par.Jc_pos-1,par.Jr_pos):
            max_inc         = par.w*exp(inc.age_prof[ie][0][j-1] + np.max(par.inc.fe[ie][:]) + max(inc.z_val[ie][j-1,:]))
            maxS            = max(par.grids[ie][j-1][-1]*1.3,max_inc*3)
            par.grids[ie][j] = np.linspace(-borr_limits[j]**(1/curv),maxS**(1/curv),int(par.Ls[j]))**curv

    # j = Jr+1 : Jd
        maxR                = par.grids[ie][j][-1]*2
        for j in range (par.Jr_pos,par.Jd_pos):
            par.grids[ie][j] = np.linspace(-borr_limits[j]**(1/curv),maxR**(1/curv),int(par.Ls[j]), dtype='float64')**curv


## Fertility Policies

    if options.fert_transfer == 'N':
        par.fert_trans = np.zeros(len(par.N))
    elif options.fert_transfer == 'only2':
        par.fert_trans = np.zeros(len(par.N))
        ind            = logical(par.N == 1);
        par.fert_trans[ind] = 2*options.fert_trans_size*par.p_model_data


    if options.init_transfer == 'N':
        par.init_trans = np.zeros([len(par.educ),par.Jd_pos])
        par.init_trans = np.reshape(par.init_trans,[len(par.educ),par.Jd_pos])
    elif options.init_transfer == 'Y':
        par.init_trans = np.zeros([len(par.educ),par.Jd_pos])
        par.init_trans = np.reshape(par.init_trans,[len(par.educ),par.Jd_pos])
###########################################################################################     missing options.init_trans_size
        par.init_trans[:,par.Je1_pos]   = 2*options.init_trans_size*par.p_model_data
    elif options.init_transfer == 'OnlyCollege':
        par.init_trans = np.zeros([len(par.educ),par.Jd_pos])
        par.init_trans = np.reshape(par.init_trans,[len(par.educ),par.Jd_pos])
        par.init_trans[2,(par.Je2_pos-1):(par.Je2_pos+1)]    = 2*options.init_trans_size*par.p_model_data
############################################################################################
#still need options.init_trans_size 
    return par
    

parameters(par_est, options)

def logncdf(x,mu,sigma):  
    #mat = np.zeros([len(mu),len(x)])
    x = np.array(x)
    mu = np.array(mu)
    if size(mu)>1:
        mat = np.empty([size(mu),size(x)])
        for j in range(0,size(x)):
            for i in range(0,size(mu)):
                mat[i,j] = stats.lognorm(sigma, scale=math.exp(mu[i])).cdf(x[j])
        result = mat
    else:
        result = stats.lognorm(sigma, scale=math.exp(mu)).cdf(x)
    return result

def PG(x):
    par.PG = np.squeeze(np.array([logncdf(np.append(np.array(par.H0[0:-1]),inf),log(par.meanh0) + par_est.beta_inc * log(x),par_est.sigmah0) - logncdf(np.append(0, np.array(par.H0[0:-1])),log(par.meanh0) + par_est.beta_inc * log(x),par_est.sigmah0)])).T
    return par.PG

par_income_process(par,options)

#format(sigma_grid, '.19g')
#num2str(sigma_grid, 17)
# for i in range(0,len(par.inc.fe[0])):
#     print(format(par.inc.fe [0][i], '.19g'))
    
    
    #-0.51165009307224917912



[[7.45412218e-01 2.27259908e-01 2.59824687e-02 1.32024789e-03
  2.51571719e-05]
 [5.68149769e-02 7.58403453e-01 1.71435117e-01 1.30163915e-02
  3.30061972e-04]
 [4.33041145e-03 1.14290078e-01 7.62759021e-01 1.14290078e-01
  4.33041145e-03]
 [3.30061972e-04 1.30163915e-02 1.71435117e-01 7.58403453e-01
  5.68149769e-02]
 [2.51571719e-05 1.32024789e-03 2.59824687e-02 2.27259908e-01
  7.45412218e-01]]
[[7.52379252e-01 2.21859768e-01 2.45330512e-02 1.20570777e-03
  2.22210113e-05]
 [5.54649420e-02 7.64645778e-01 1.67299107e-01 1.22887466e-02
  3.01426942e-04]
 [4.08884187e-03 1.11532738e-01 7.68756841e-01 1.11532738e-01
  4.08884187e-03]
 [3.01426942e-04 1.22887466e-02 1.67299107e-01 7.64645778e-01
  5.54649420e-02]
 [2.22210113e-05 1.20570777e-03 2.45330512e-02 2.21859768e-01
  7.52379252e-01]]
[[7.57462866e-01 2.17886646e-01 2.35034246e-02 1.12680603e-03
  2.02580877e-05]
 [5.44716614e-02 7.69214578e-01 1.64260089e-01 1.17719704e-02
  2.81701508e-04]
 [3.91723744e-03 1.09506726e-01 7.7315

namespace(inno_pos=[0, 1, 2, 3, 4],
          fe_pos=range(0, 5),
          age_prof={0: [array([0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
                            0.       , 0.       , 0.       , 0.0639914, 0.1235652, 0.1787214,
                            0.22946  , 0.275781 , 0.3176844, 0.3551702, 0.3882384, 0.416889 ,
                            0.441122 , 0.4609374, 0.4763352, 0.4873154, 0.493878 , 0.496023 ,
                            0.4937504, 0.4870602, 0.4759524, 0.460427 , 0.440484 , 0.4161234,
                            0.3873452, 0.3541494, 0.316536 , 0.274505 ])],
                    1: [array([0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
                            0.       , 0.       , 0.       , 0.0838808, 0.161912 , 0.2340936,
                            0.3004256, 0.360908 , 0.4155408, 0.464324 , 0.5072576, 0.5443416,
                            0.575576 , 0.6009608, 0.620496 , 0.6341816, 0.6420176, 0.644004 ,
       

In [75]:
par.r_sav

0.060899999999999954

In [99]:
# -*- coding: utf-8 -*-
"""
Created on Sun Jul 24 09:59:08 2022

@author: Giorgia
"""

# -*- coding: utf-8 -*-
"""
Created on Thu Jul 21 11:58:53 2022

@author: Giorgia
"""


#approx_2d with arrays
from types import SimpleNamespace
import time
import itertools as it
import numpy as np
from scipy import optimize

#approx_2d
def approx_2d(X,Y,x):
    Nx = len(X)
    #Mx = round(Nx/2)
    j = 1
    y = np.empty(len(x))
    y[:] = np.nan
    for i in range(0,len(x)):
        #while (j < Nx-1 and x[i] > X[j]):
        while (j<Nx-1 and x[i]>X[j]):
            j = j+1

    #First order aproximation
        dy = (Y[j]-Y[j-1])/(X[j]-X[j-1])############### problem, in matlab index j is different, each element of a matrix has a single number
        dx = x[i]-X[j-1]
        y[i] = Y[j-1] + dy*dx

    #approximation outside bounds: linear extrapolation (OLS)
        Nextrap = 10
        Nextrap = min(Nextrap,Nx)
        if x[i] < X[0]:#CHAGED SIGN FOR ATTEMPT
            X_reg = np.array([np.ones(Nextrap), np.reshape(X[0:Nextrap],Nextrap)]).T #I transpose it to get same result as Matlab, but I X is a marix then it does not work well
            Y_reg = np.reshape(Y[0:Nextrap],Nextrap).T
            beta_nom = np.dot(X_reg.T,Y_reg)
            beta_denom = np.dot(X_reg.T, X_reg)
            beta = np.linalg.inv(beta_denom).dot(beta_nom)##### equivalent of blacklash operator to solve linear system in matlab
            new_array = np.array([1,x[i]])
            y[i] = np.dot(new_array,beta) #####I get reversed order from if: (instead of y = [2.71, 2.71, 5] I get y = [5,5,2.71])
        elif x[i] > X[Nx-1]:
            X_reg = np.array([np.ones(Nextrap), np.reshape(X[Nx-Nextrap:], Nextrap)]).T #I transpose it to get same result as Matlab
            Y_reg = np.reshape(Y[Nx - Nextrap:],Nextrap).T
            beta_nom = np.dot(X_reg.T,Y_reg)
            beta_denom = np.dot(X_reg.T, X_reg)
            beta = np.linalg.inv(beta_denom).dot(beta_nom)
            y[i] = np.dot(np.array([1,x[i]]),beta)########bad things happening PROBLEM WITH IF STATEMENT, BOTH RESULTS ARE THE SAME BECAUSE IT OPERATES ONLY ON x[i]>X[i]
    return y






In [98]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jul 21 11:58:53 2022

@author: Giorgia
"""


from types import SimpleNamespace
import time
import itertools as it
import numpy as np
from scipy import optimize




def EGM(par, ucp, dispinc, Spgrid, S):
    r_sav = par.r_sav
    r_debt = par.r_debt
    gammac = par.gammac
    
##check if ucp is decreasing
#Lag = (ucp(1:end-1)<=1e+10)*(ucp(2:end)-ucp(1:end-1))
#Lagmax = max(Lag)
#if Lagmax > 0:
    #fprint('ucp is not decreasing\n')
    
#solve for an endogenous
    c = (ucp)**(-1/gammac)# vector-- ucp, c, Spgrid and dipinc must have same dimension
    ################################################################
    # a_endo = np.zeros([len(c),len(Spgrid)])
    # for i in range(0,len(c)):
    #     for j in range(0,len(Spgrid)):
    #         a_endo[i,j] = c[i]+Spgrid[j] -dispinc
    # a_endo = a_endo.T
    a_endo = (c + Spgrid.T - dispinc )############# careful here, it needs to be a one dimensional vector. c is (80,1) and Spgrid (1,80) which is why we use the transposed
    a_endo = a_endo/(1+r_sav)*(a_endo>=0) + a_endo/(1+r_debt)*(a_endo<0)
    #########################################################    
    #check borrowing limits
    #here we get scalars
    a_bc = a_endo[0] 
    S_bc = []
    
    #a_endo = c + Spgrid.T- dispinc
    for i in range(0,len(S)):
        if np.any(S[i]<a_bc):
            S_bc.append(S[i])      
    S_bc = np.array(S_bc)

    c_bc = (1 + r_sav)*S_bc * (S_bc >=0) + (1 +r_debt)*(S_bc)*(S_bc < 0) + dispinc - Spgrid[0] ######what is S_bc was not (2x1)?
    
    #Interpolate in original grid S (Since the borrowing limit is defined on strict inequality, all a_endo should be included in interpolation)
   
    c_final = approx_2d(np.append(S_bc,a_endo), np.append(c_bc,c), S)#### problem with dimensions
    #c_final = approx_2d(np.concatenate(np.reshape(S_bc,(len(S_bc),1)), a_endo), np.concatenate(np.reshape(c_bc,(len(c_bc),1)),c), S)#### problem with dimensions
    
    #c_final = approx_2d(np.concatenate( [S_bc,a_endo]), np.append(np.array([np.append(c_bc,c)]),np.array([np.append(c_bc,c)])), S)## works as in matlab for X, Y vector with same lenght and S vector even with different lenght (but while in Matlab it returns a value fro c_final, in python it gives an error even if thye deliver the same output y)
    
    sp_final = (1 +r_sav)*S*(S>=0) + (1 + r_debt)*S.T*(S.T<0) + dispinc - c_final #WORKS as in MATLAB!!! but same result as if I did not transpose S
    
    #sp_final = (1 +r_sav)*np.reshape(S,(len(S),1))*(np.reshape(S,(len(S),1))>=0) + (1 + r_debt)*np.reshape(S,(len(S),1))*(np.reshape(Spgrid,(len(Spgrid),1))<0) + dispinc #- c_final

    sp_final = sp_final * ((sp_final >= Spgrid[0]) + (sp_final<Spgrid[0]-1e-6)) + Spgrid[0] * (sp_final < Spgrid[0])*(sp_final >= Spgrid[0])*(sp_final>=Spgrid[0]-1e-6)### same result as matlab
    
    if sum(sp_final < Spgrid[0]) > 0:
         print('EGM error: savings are below debt limit - wrong extrapolation, min(s''): {sp_final} \n')### must be fixed
    
    boundgrid = sum(sp_final >1.025*Spgrid[len(Spgrid)-1]) #should be ok, same result as Matlab

    
    #check for bad extrapolation
    c_max = (1 + r_sav)*S.T*(S.T >= 0) + (1 + r_debt)*S.T*(S.T<0) + dispinc-Spgrid[0] #working as in Matlab
    c_final = (c_final <= c_max)*c_final + (c_final > c_max) * c_max
    
    # check for bad extrapolation    
    sp_final = (1 +r_sav)*S.T*(S.T>=0) + (1 + r_debt)*S.T*(S.T<0) + dispinc - c_final
    
    return c_final, sp_final, boundgrid
        

                                                                                                          
                                                                                                 

    
    

In [97]:

from types import SimpleNamespace
import time
import itertools as it
import numpy as np
from scipy import optimize
from scipy import stats
from __future__ import print_function
from numpy import *
from statistics import NormalDist

    
def ret_rep(par,fe,educ,options):
# keyboard
# Pension replacement rate: 
# Source: Krueger, Ludwig 2015
# Parameters
    
    FE                  = par.inc.fe
    AGE_PROF            = inc.age_prof

    
## 1. Average lifetime income (AIME)
    age_ret             = range(par.Je2_pos,par.Jr_pos-1)
    totinc              = w * np.sum(exp(np.reshape(np.repeat([FE[educ][:len(fe)][:]],len(age_ret),axis = 0),[len(age_ret),len(fe)]) + np.reshape(np.repeat([AGE_PROF[educ][0][age_ret]],len(fe)),[len(age_ret),len(fe)])), axis = 0)##### get rid of rrays in first element of the sum

    prob_educ           = par.dist_educ
    den                 = 0
    h0_pos              = math.ceil(par.N_fe/2) # To find individual that starts with h0 = average
    
    for ie in range (0,len(par.educ)):
        den             =  den + prob_educ[ie] * w *np.sum(exp(FE[educ][h0_pos-1].T + AGE_PROF[ie][0][age_ret]))
    y                   = totinc/den

## 2. Marginal replacement rates
    if options.ExPolRetirement == 'Y':
        tau1 = options.PolRetMult[0]
        tau2 = options.PolRetMult[1]
        tau3 = options.PolRetMult[2]
    elif options.ExPolRetirement == 'N':
        tau1   = 0.9
        tau2   = 0.32
        tau3   = 0.15    

##3. Bendpoints
    b1                  = 0.24
    b2                  = 1.35
    b3                  = 1.99

## 4. Replacement rate
    rep_rate            = (tau1*y)*(y<= b1) + (tau1*b1 + tau2*(y-b1))*(y>b1)*( y<= b2) + (tau1*b1 + tau2*(b2-b1) + tau3*(y- b2))*(y>b2)*( y<= b3) + (tau1*b1 + tau2*(b2-b1) + tau3*(b3-b2))*(y>b3);
                
## Replacement Benefits:
    avg_inc             = den/len(age_ret)
    rep                 = avg_inc * rep_rate
    return rep
    #         fprintf('educ = %i, fe = %i, y=%3.3f,ret_rep = %3.3f , rep = %3.3f, avg inc = %3.3f \n',educ,fe,y,rep_rate,rep,totinc/length(age_ret));


In [95]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jul 25 16:56:07 2022

@author: Giorgia
"""
from types import SimpleNamespace
import time
import itertools as it
import numpy as np
from scipy import optimize
from scipy.interpolate import RegularGridInterpolator
import scipy.interpolate as spi


##all coming from file parameters



def prob_ret(par,options,Vp,Cp,j_pos):
                ##########    r_sav = par.r_sav
    r_sav = par.r_sav
    r_debt = par.r_debt

    beta = par.beta
    gammac = par.gammac
    EDUC = par.educ
    
    par.N_fe = 5
    N_fe         = par.N_fe # from parameters par.N_fe = 5
    par.inc.fe_pos   = np.linspace(0,N_fe-1,N_fe) ## again from parameters
    inc.fe_pos = par.inc.fe_pos
    FE_pos = inc.fe_pos #form par_income _process we have inc_fe_pos = np.arange(0,N_fe,N_fe) (still need N_fe)
    
##########################indexing using j_pos are right only if we use j_pos = j in solve_model!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

    if j_pos == par.Jd_pos-2: #consume all, next period is dead
        S = par.grids[0][j_pos]## matlab uses {} because par.grids is a cell array which I do not manage to convert in python
        Sp = np.zeros([len(EDUC),len(S),len(FE_pos)])
        C = np.zeros([len(EDUC),len(S),len(FE_pos)])
        V = np.zeros([len(EDUC),len(S),len(FE_pos)])
        
        for educ in range(0,len(EDUC)):
            S = par.grids[educ][j_pos]
            C[educ,:,:,] = ((1+r_sav)*np.repeat([S],len(FE_pos),axis =0) + np.reshape(np.repeat(ret_rep(par,FE_pos,educ,options),len(S)),[len(FE_pos),len(S)])).T
            V[educ,:,:,]= (C[educ,:,:,]**(1-gammac))/(1-gammac)
    else:# solve Euler equation
        S = par.grids[0][j_pos-1]## matlab uses {} because par.grids is a cell array which I do not manage to convert in python
        Sp = np.zeros([len(EDUC),len(S),len(FE_pos)])
        C = np.zeros([len(EDUC),len(S),len(FE_pos)])
        V = np.zeros([len(EDUC),len(S),len(FE_pos)])

        boundgrid = np.zeros([len(FE_pos),len(EDUC)])

        for educ in range (0,len(EDUC)):
            S       = par.grids[educ][j_pos]
            Spgrid  = par.grids[educ][j_pos+1]
        
            for ife in range (0,len(FE_pos)):
                ##############################################
                #splVp = RegularGridInterpolator((Spgrid,),Vp[educ][ife][:]) 
                splVp = spi.interp1d(Spgrid,Vp[educ,:,ife],axis=0, fill_value="extrapolate")
                #splVp = scipy.interpolate.interp1d(Spgrid,Vp[educ][ife][:])
                ##############################################
                Cpp   = Cp[educ,:,ife] #actually Cp, not C

                ucp     = beta*(1+r_sav)*Cpp**(-gammac)

                dispinc = ret_rep(par,FE_pos,educ,options)
                dispinc = dispinc[ife]

                C[educ,:,ife],Sp[educ,:,ife],boundgrid[ife,educ] = EGM(par,ucp,dispinc,Spgrid,S)

                V[educ,:,ife] = C[educ,:,ife]**(1-gammac)/(1-gammac) + beta*splVp(Sp[educ,:,ife])
                
                errors = sum(boundgrid[:])/np.size(C[:])
    
###################################################################    
        if options.timer_on == 'Y':
             if sum(boundgrid[:]) >= len(EDUC)*len(FE_pos):
                 print(f"j : {par.age[j_pos-1]}, Share of errors (increase grid) = {errors}")
##################################################################################################            
    return V, C, Sp


In [94]:
# -*- coding: utf-8 -*-
"""
Created on Sat Aug 13 16:49:46 2022

@author: Giorgia
"""

from types import SimpleNamespace
import time
import itertools as it
import numpy as np
from scipy import optimize
#from scipy.interpolate import RegularGridInterpolator
from scipy.io import loadmat
import os
import scipy.interpolate as spi

def prob_workT(par,options,Vp,Cp,j_pos):
    r_sav     = par.r_sav
    r_debt    = par.r_debt
    beta      = par.beta
    gammac    = par.gammac
    w         = par.w
    Lambda    = par.Lambda
    
    S = par.grids[0][j_pos]## matlab uses {} because par.grids is a cell array which I do not manage to convert in python
    EDUC      = par.educ
    AGE_PROF  = par.inc.age_prof
    INNO_pos  = par.inc.inno_pos
    FE_pos    = par.inc.fe_pos
    
    Sp = np.zeros([len(INNO_pos),len(EDUC),len(S),len(FE_pos)])
    C = np.zeros([len(INNO_pos),len(EDUC),len(S),len(FE_pos)])
    V = np.zeros([len(INNO_pos),len(EDUC),len(S),len(FE_pos)])
    boundgrid = np.zeros([len(INNO_pos),len(FE_pos),len(EDUC)])
    
    for educ in range (0,len(EDUC)):
        S       = par.grids[educ][j_pos]
        Spgrid  = par.grids[educ][j_pos+1]
        
        FE        = par.inc.fe[educ]
        INNO      = par.inc.z_val[educ][j_pos,:]
        
        r         = (r_sav * (Spgrid>=0) + r_debt * (Spgrid<0)).T
        rs        = (r_sav * (S>=0) + r_debt * (S<0))
        
        for ife in range (0,len(FE_pos)):
            splVp = spi.interp1d(Spgrid,Vp[educ,:,ife],axis=0, fill_value="extrapolate")
            # People don't work next period so only disutility of consumption. 
            Cpp = Cp[educ,:,ife]
            ucp = beta*(1+r)*Cpp**(-gammac)
            
            for inno  in range (0,len(INNO_pos)):
                dispinc = (1-Lambda) * exp(AGE_PROF[educ][0][j_pos]) * exp(FE[ife]) * exp(INNO[inno]) #AGE_PROF[educ][0][j_pos], i THINK THE [0] IS NEEDED ONLY BECAUSE IT IS A 1 DIMENSION DICTIONARY(FLEXIBLE) 
    
                feasible = (dispinc + (1+rs)*S - Spgrid[0]>0)
                not_feasible = (dispinc + (1+rs)*S - Spgrid[0]<=0)
    
                C[inno,educ,feasible,ife],Sp[inno,educ,feasible,ife],boundgrid[inno,ife,educ] = EGM(par,ucp,dispinc,Spgrid,S[feasible])
    
                vp = splVp(Sp[inno,educ,feasible,ife])
                V[inno,educ,feasible,ife] = (C[inno,educ,feasible,ife]**(1-gammac))/(1-gammac) + beta*vp
                C[inno,educ,not_feasible,ife] = 0
                V[inno,educ,not_feasible,ife] = (-(10**(5/gammac))**(1-gammac))/(1-gammac)
                Sp[inno,educ,not_feasible,ife]= Spgrid[0]

    
    errors = sum(boundgrid[:])/np.size(C[:])
    if options.timer_on == 'Y':
            if sum(boundgrid[:]) >= len(EDUC)*len(FE_pos)*len(INNO_pos):
                 print(f"j : {par.age[j_pos-1]}, Share of errors (increase grid) = {errors}")
##################################################################################################            
    return V, C, Sp
    
#########################THE PROBLEM IS THAT i AM MIZING TYPES IN A LIST
#compare with matlab calling C[32][4][2] or any other combination of [inno][educ], first row [0] is 00000 for C and S, -2 for V. if we comment out " C[inno,educ,not_feasible,ife] = 0", everything is fine
#it works great except C[32][:][:][0],
#C(not_feasible,ife,educ,inno)

In [93]:
# -*- coding: utf-8 -*-
"""
Created on Sun Aug 14 15:48:14 2022

@author: Giorgia
"""

# -*- coding: utf-8 -*-
"""
Created on Sat Aug 13 16:49:46 2022

@author: Giorgia
"""

from types import SimpleNamespace
import time
import itertools as it
import numpy as np
from scipy import optimize
from scipy.interpolate import RegularGridInterpolator
from scipy.io import loadmat
import os
import scipy.interpolate as spi
from scipy.interpolate import griddata
from scipy.interpolate import RegularGridInterpolator
from scipy.interpolate import LinearNDInterpolator
from scipy.interpolate import interpn
from scipy.interpolate import interp2d
from scipy.interpolate import NearestNDInterpolator

def prob_work_old(par,options,Vp,Cp,j_pos):
    r_sav     = par.r_sav
    r_debt    = par.r_debt
    beta      = par.beta
    gammac    = par.gammac
    w         = par.w
    Lambda    = par.Lambda
    #parameters(par_est, options)
    
    S = par.grids[0][j_pos]## matlab uses {} because par.grids is a cell array which I do not manage to convert in python
    EDUC      = par.educ
    AGE_PROF  = par.inc.age_prof
    INNO_pos  = par.inc.inno_pos
    FE_pos    = par.inc.fe_pos
    
    Sp = np.zeros([len(INNO_pos),len(EDUC),len(S),len(FE_pos)])
    C = np.zeros([len(INNO_pos),len(EDUC),len(S),len(FE_pos)])
    V = np.zeros([len(INNO_pos),len(EDUC),len(S),len(FE_pos)])
    boundgrid = np.zeros([len(INNO_pos),len(FE_pos),len(EDUC)])
    INNO_posT = np.reshape(INNO_pos,[1,5])
    inno3 = np.repeat(INNO_posT,len(S),axis=0)

    
    for educ in range (0,len(EDUC)):
        S       = par.grids[educ][j_pos]
        FE        = par.inc.fe[educ]
        INNO      = par.inc.z_val[educ][j_pos,:]
        Spgrid  = par.grids[educ][j_pos+1]
        r         = (r_sav * (Spgrid>=0) + r_debt * (Spgrid<0)).T
        rs        = (r_sav * (S>=0) + r_debt * (S<0))
        
        INNOp_prob = par.inc.z_prob[educ][:,j_pos+1,:].T # Note I need j_pos + 1 here!
        """it would be good to understand why I need to transpose par.inc.z_prob all the times, as I had to transpose P when I filled the matrix in "par_income_process"     
        """
        Spgrid     = par.grids[educ][j_pos+1]
        
        FE        = par.inc.fe[educ]
        INNO      = par.inc.z_val[educ][j_pos,:]
        
        r         = (r_sav * (Spgrid>=0) + r_debt * (Spgrid<0)).T
        rs        = (r_sav * (S>=0) + r_debt * (S<0))
        
        INNO2,S2  = np.meshgrid(INNO_pos,Spgrid)

    
        for ife in range (0,len(FE_pos)):
            ##points in grid given by INNO2
            ##values in Vp[]
            # splVp = RegularGridInterpolator(S2,Vp[:,educ,:,ife].T,method = 'linear')#########may need to invert S2 and INNO2 ????
            # splVp = LinearNDInterpolator((S2,INNO2), Vp[:,educ,:,ife].T)
            ######
            splVp = GlobalSpline2D(Spgrid, INNO_pos,Vp[:,educ,:,ife])
            
            # for i in range(0,len(S)):
            #     if Vp[inno,educ,i,ife] != 0:
            #         Vp[inno,educ,i,ife] = Vp[inno,educ,i,ife] 
            #         splVp = interp2d(Spgrid, INNO_pos,Vp[:,educ,:,ife],  'linear' )
            #     else:
            #         Vp[inno,educ,i,ife] = (-(10**(5/gammac))**(1-gammac))/(1-gammac)
            #         splVp = interp2d(Spgrid, INNO_pos,Vp[:,educ,:,ife],  'linear' )
            
            #splVp = NearestNDInterpolator(S2,Vp[:,educ,:,ife].T,  'linear' )
            #matplotlib.pyplot.contourf(S2, INNO2, Vp[:,educ,:,ife].T, cmap = 'jet')
            #splVp = interpn(INNO2,Vp[:,educ,:,ife],S2, )
            ## People don't work next period so only disutility of consumption. 
            for inno  in range (0,len(INNO_pos)):
                
                innop_prob = INNOp_prob[inno,:]
                Cpp = Cp[:,educ,:,ife].T
                if min(np.ravel(Cpp.any())<0 and abs(min(np.ravel(Cpp.any()))))>1e-6:
                    Cpp = max(Cpp,0)
                ucp = beta*np.reshape(((1+r)),[80,])*np.dot(Cpp**(-gammac), np.reshape(innop_prob,[5,]))
            
            
                dispinc = (1-Lambda) * exp(AGE_PROF[educ][0][j_pos]) * exp(FE[ife]) * exp(INNO[inno]) #AGE_PROF[educ][0][j_pos], i THINK THE [0] IS NEEDED ONLY BECAUSE IT IS A 1 DIMENSION DICTIONARY(FLEXIBLE) 
    
                feasible = (dispinc + (1+rs)*S - Spgrid[0]>0)
                not_feasible = (dispinc + (1+rs)*S - Spgrid[0]<=0)
                

                C[inno,educ,feasible,ife],Sp[inno,educ,feasible,ife],boundgrid[inno,ife,educ] = EGM(par,ucp,dispinc,Spgrid,S[feasible])
                
                #c_final, sp_final, boundgrid = EGM_old_worker(par,ucp,dispinc,Spgrid,S[feasible])
                Sp3 = np.repeat(np.reshape(Sp[inno,educ,:,ife],[80,1]),len(INNO_pos),axis = 1)

                vp = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)
                #vp = splVp(Sp3)
                
                V[inno,educ,feasible,ife] = C[inno,educ,feasible,ife]**(1-gammac)/(1-gammac) + beta*vp[feasible]
                C[inno,educ,not_feasible,ife] = 0
                V[inno,educ,not_feasible,ife] = (-(10**(5/gammac))**(1-gammac))/(1-gammac)
                Sp[inno,educ,not_feasible,ife] = Spgrid[0]
    
    errors = sum(boundgrid[:])/size(C[:])
    if options.timer_on == 'Y':
        if sum(boundgrid[:]) >= len(EDUC)*len(FE_pos)*len(INNO_pos):
            print(f"j : {par.age[j_pos-1]}, Share of errors (increase grid) = {errors}")
##################################################################################################            
    return V, C, Sp
    
#########################THE PROBLEM IS THAT i AM MIZING TYPES IN A LIST
#compare with matlab calling C[32][4][2] or any other combination of [inno][educ], first row [0] is 00000 for C and S, -2 for V. if we comment out " C[inno,educ,not_feasible,ife] = 0", everything is fine
#it works great except C[32][:][:][0],
#C(not_feasible,ife,educ,inno)

In [92]:
# -*- coding: utf-8 -*-
"""
Created on Wed Aug 24 15:55:00 2022

@author: Giorgia
"""
#https://github.com/maitlahcen/CompEcon-python/blob/master/compecon/tools.py
    
from functools import reduce
import numpy as np
from scipy.linalg import qz
import time

from scipy.sparse import identity


def gridmake(*arrays):
    """
    Forms grid points
    USAGE
    X = gridmake(x1,x2,...,xn)
    X1,..., Xn = gridmake(x1,x2,...,xn)
    Expands matrices into the associated grid points.
    If N is the 2xd array that indexes the size of the inputs, GRIDMAKE returns a sum(N[0]) by prod(N[1]) array.
    The output can also be returned as either
      d matrices or
      sum(N(:,2)) matrices
    Note: the grid is expanded so the last variable change most quickly.
    Example:
    X = gridmake([1, 2, 3], [4, 5])
    array([[1, 1, 2, 2, 3, 3],
           [4, 5, 4, 5, 4, 5]])
    Also the inputs need not be vectors.
    Y = gridmake(X, [10, 20])
    array([[ 1,  1,  1,  1,  2,  2,  2,  2,  3,  3,  3,  3],
           [ 4,  4,  5,  5,  4,  4,  5,  5,  4,  4,  5,  5],
           [10, 20, 10, 20, 10, 20, 10, 20, 10, 20, 10, 20]])
    """
    if len(arrays) == 1:
        return arrays[0]

    arrays = np.atleast_2d(*arrays)
    n = len(arrays)
    idx = np.indices([a.shape[1] for a in arrays]).reshape([n, -1])
    return np.vstack(arrays[k][:, idx[k]] for k in range(n))


In [91]:
# -*- coding: utf-8 -*-
"""
Created on Tue Aug 23 17:27:21 2022

@author: Giorgia
"""

def ChildCost(par,labor_inc,n,T,options): ##ok<INUSD>
    ## Child cost function - Source: Angrist and Evans 1998. Assume 3rd child cost equal to any child + linear cost (see table 10)
    # 1/3 is loss of hours when raising kids oneself, 1/8 when hiring nannys. Difference must be covered by nannies.
    # C  = @(h,n,T) T * n^0.7 *(par.w*h*(1-par.lambda) * 1/8  + (1/3-1/8) * pn ) ...
    # 		  + (1-T) * n^0.7 *  (par.w*h*(1-par.lambda) * 1/3);
    # Source: Angrist and Evans 1998. Assume 3rd child cost equal to any child + linear cost (see table 10)
    
    n                   = n*2/par.fam_size
    h                   = labor_inc/(par.w*(1-par.Lambda))
    
    
    #lost_wnanny         = par.Child_C_Tot*par.Nanny_oppC
    l_ret               = par.Child_Ccurv
    child_cost_inc_curv = par.child_cost_inc_curv
    # lost_total          = par.Child_C_Tot * (1-par.lambda)**(child_cost_inc_curv-1)
    lost_total          = par.Child_C_Tot
    C                   = n**l_ret * par.w*(1-par.Lambda) *  h**child_cost_inc_curv * lost_total
    
    return C
# lost_total          = par.Child_C_Tot
    # C                   = n**l_ret * labor_inc**child_cost_inc_curv * lost_total
    
    # l_ret               = 0.4757
    # l_ret               = 0.7;
    # if options.ChildCost == 'OppC_Const':
    #     
    ##         C           = T *       (n*par.fam_size)**l_ret * (labor_inc* lost_wnanny   + (lost_total-lost_wnanny) * par.pn ) + (1-T) * (n*par.fam_size)^l_ret * (labor_inc* lost_total);
    ##             If estimating lost_total, lost_wnanny, l_ret: use below
    #           C           = T *       n**l_ret * (labor_inc* lost_wnanny   + (lost_total-lost_wnanny) * par.pn ) + (1-T) * n**l_ret * (labor_inc* lost_total);
    # elif options.ChildCost == 'Constant':
    ##        C           = (n*par.fam_size)**l_ret * lost_total * par.pn
    #           C           = n**l_ret * lost_total * par.pn
    # elif options.ChildCost == 'OppC':
    ##        C           = (n*par.fam_size)**l_ret * (labor_inc* lost_total);
   ##        C           = min(n**l_ret * (labor_inc* lost_total),labor_inc)
    #         C           = n^l_ret .* labor_inc.^child_cost_inc_curv .* lost_total;

    

    
    
    ### Economies of scale:
    ## Source: Folbre 2008, table 6.4, also used in Cordoba, Liu, Ripoll 2015
    # Y                   = [150 2*100 3*85]';
    # l_ret               = 0.4757;
    #% Y                   = [55.2 82.6 113]';
    # N                   = (1:3)';
    #lY                  = log(Y);
    # lN                  = log(N);
    # X                   = [ones(3,1) lN];
    # b                   = (X.T*X)\X.T*lY;
    # l_ret               = b(2)
    # l_ret               = 0.6445;


In [90]:
# -*- coding: utf-8 -*-
"""
Created on Wed Aug 24 16:43:00 2022

@author: Giorgia
"""

def EGM_withchild( par,ucp, dispinc,Spgrid,S,n ):
    # EGM
    #dispinc= -4.227620999831343
    r_sav  = par.r_sav
    r_debt = par.r_debt
    gammac = par.gammac
    
    ## Check ucp decreasing
    # Lag =(ucp(1:end-1)<=1e+10) .* (ucp(2:end) - ucp(1:end-1));
    # Lagmax = max(Lag);
    # if Lagmax > 0 
    #     fprintf('ucp is not decreasing\n')
    # end
    
    # Solve for a endogenous
    c = ( ucp)**(-1/gammac)
    
    lambdan = par.lambdan
    gamman  = par.gamman
    f_n     = (lambdan/n**(1-gamman))**(1/gammac)
    c_k     = f_n * c
    a_endo  = (c + n*c_k + Spgrid.T - dispinc )
    a_endo = a_endo/(1+r_sav)*(a_endo>=0) + a_endo/(1+r_debt)*(a_endo<0)
    
    # Check borrowing limits
    a_bc    = a_endo[0] # Any current level of savings below this should be constrained
    S_bc    = S[S < a_bc] # Borrowing constrained
    c_bc    = ((1+r_sav) * S_bc * (S_bc >= 0) + (1+r_debt) * S_bc * (S_bc < 0) + dispinc - Spgrid[0])/(1+n*f_n)
    
    c_k_bc  = f_n * c_bc
    
    # Interpolate in originial grid S
    # (Since the borrowing limit is defined on strict inequality, all a_endo
    # should be included in interpolation)
    
    c_final = approx_2d(np.append(S_bc,a_endo), np.append(c_bc,c), S)
    c_k_final  = approx_2d(np.append(S_bc, a_endo), np.append(c_k_bc, c_k), S)
############# MAKE Approx2s more precise or at least aligned with Matlab
    sp_final = (1 +r_sav)*S.T*(S>=0) + (1 + r_debt)*S.T*(S.T<0) + dispinc - c_final - n*c_k_final

    ########spfinal is not precize enough, but spfinal[0] witll not go over 15 decimals
    sp_final = sp_final * ((sp_final >= Spgrid[0]) + (sp_final<Spgrid[0]-1e-6)) + Spgrid[0] * (sp_final < Spgrid[0])*(sp_final >= Spgrid[0]-1e-6)### same result as matlab
                        ##############################requires a level of precision which is extreeme
    if sum(sp_final < Spgrid[0]) > 0:
         print('EGM error: savings are below debt limit - wrong extrapolation, min(s''): {sp_final} \n')### must be fixed

    
    boundgrid = sum(sp_final >1.025*Spgrid[len(Spgrid)-1])
    
    # check for bad extrapolation
    c_max = ((1 + r_sav)*S.T*(S.T >= 0) + (1 + r_debt)*S.T*(S.T<0) + dispinc-Spgrid[0])/(1+n*f_n)

    c_k_max   = f_n * c_max
    c_final = (c_final <= c_max)*c_final + (c_final > c_max) * c_max

    c_k_final = ( c_k_final<= c_k_max)*c_k_final+         (c_k_final > c_k_max)*c_k_max
    sp_final = (1 +r_sav)*S.T*(S.T>=0) + (1 + r_debt)*S.T*(S.T<0) + dispinc - c_final - n*c_k_final


    
    return c_final,sp_final, boundgrid 
###############################


In [87]:
# -*- coding: utf-8 -*-
"""
Created on Sat Oct 22 14:25:49 2022

@author: Giorgia
"""

# -*- coding: utf-8 -*-
"""
Created on Fri Aug 19 21:34:08 2022

@author: Giorgia
"""

from types import SimpleNamespace
import time
import itertools as it
import numpy as np
from scipy import optimize
from scipy.interpolate import RegularGridInterpolator
from scipy.io import loadmat
import os
import scipy.interpolate as spi
import math 

def prob_work_trans(par,options,Vp,Cp,Vc0,j_pos):
    
    r_sav     = par.r_sav
    r_debt    = par.r_debt
    beta      = par.beta
    gammac    = par.gammac
    w         = par.w
    Lambda    = par.Lambda
    lambdan   = par.lambdan
    gamman    = par.gamman
    
    S         = par.grids[0][j_pos]
    EDUC      = par.educ
    
    AGE_PROF  = par.inc.age_prof
    INNO_pos  = par.inc.inno_pos
    INNO_posT = np.reshape(INNO_pos,[1,5])
    FE_pos    = par.inc.fe_pos
    
    N         = np.arange(0,4,1)
    PHI_pos   = np.arange(0,len(par.PHI))
    
    Sp = np.zeros([len(INNO_pos),len(EDUC),len(N),len(PHI_pos),len(S),len(FE_pos)])
    Sp[:] = np.nan
    C  = np.zeros([len(INNO_pos),len(EDUC),len(N),len(PHI_pos),len(S),len(FE_pos)])
    C[:] = np.nan
    V  = np.zeros([len(INNO_pos),len(EDUC),len(N),len(PHI_pos),len(S),len(FE_pos)])
    V[:] = np.nan
    Ck = np.zeros([len(INNO_pos),len(EDUC),len(N),len(PHI_pos),len(S),len(FE_pos)])
    Ck[:] = np.nan
    Tp = np.zeros([len(INNO_pos),len(EDUC),len(N),len(PHI_pos),len(S),len(FE_pos)])
    Tp[:] = np.nan
    boundgrid = np.zeros([len(INNO_pos),len(N),len(PHI_pos),len(FE_pos),len(EDUC)])
    
    ## Without children and phi = 1 (phi is transfer to each child)
    if options.Fertility == "Endo": # Only need to solve case without children in benchmark case where we allow them to choose endogenous fertility
        i_n   = 0
        iphi =0
        ig   = 0 #Group does not matter in value function tomorrow if no children
        
        for educ in range (0,len(EDUC)):
            
            S          = par.grids[educ][j_pos]
            Spgrid  = par.grids[educ][j_pos+1]
            
            FE        = par.inc.fe[educ]
            INNO      = par.inc.z_val[educ][j_pos,:]
            
            
            r         = (r_sav * (Spgrid>=0) + r_debt * (Spgrid<0)).T
            rs        = (r_sav * (S>=0) + r_debt * (S<0))
            
            inno3 = np.repeat(INNO_posT,len(S),axis=0)
            INNOp_prob = par.inc.z_prob[educ][:,j_pos+1,:].T # Note I need j_pos + 1 here!

            INNO2,S2  = np.meshgrid(INNO_pos,Spgrid)
            
            for ife in range (0,len(FE_pos)):

                splVp = GlobalSpline2D(Spgrid, INNO_pos,Vp[:,educ,:,ife] )
           
                for inno  in range (0,len(INNO_pos)):
                
                    innop_prob = INNOp_prob[inno,:]
                    Cpp = Cp[:,educ,:,ife].T
                    
                    if min(np.ravel(Cpp.any())<0 and abs(min(np.ravel(Cpp.any()))))>1e-6:
                        Cpp = max(Cpp,0)
                    ucp = beta*np.reshape(((1+r)),[80,])*np.dot(Cpp**(-gammac), np.reshape(innop_prob,[5,]))
            
                    
                    h   = exp(AGE_PROF[educ][0][j_pos]) * exp(FE[ife])* exp(INNO[inno])
                    dispinc = w*h*(1-Lambda)
                    
                    feasible = (dispinc + (1+rs)*S - Spgrid[0]>0)
                    #not_feasible = (1-feasible)
                    not_feasible = (dispinc + (1+rs)*S - Spgrid[0]<=0)

                    C[inno,educ,i_n,iphi,feasible,ife],Sp[inno,educ,i_n,iphi,feasible,ife],boundgrid[inno,i_n,iphi,ife,educ] = EGM(par,ucp,dispinc,Spgrid,S[feasible])
                    Ck[inno,educ,i_n,iphi,feasible,ife] = np.zeros(len(C[inno,educ,i_n,iphi,feasible,ife]))                    
                    
                    Sp3 = np.repeat(np.reshape(Sp[inno,educ,i_n,iphi,feasible,ife],[80,1]),len(INNO_pos),axis = 1)
                    
                    vp = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)
                    # vp = np.dot((splVp(Sp3[:,0],inno3[0,:]).T),innop_prob)
                    # for j in range(0,len(vp)):
                    #     if math.isnan(vp[j]):
                    #         vp[j] = -inf
                    # vp = np.sort(vp)
                    # for j in range(0,len(vp)):
                    #     if math.isinf(vp[j]):
                    #         vp[j] = nan
                    
                    V[inno,educ,i_n,iphi,feasible,ife] = C[inno,educ,i_n,iphi,feasible,ife]**(1-gammac)/(1-gammac) + beta*vp[feasible]
                    
                    C[inno,educ,i_n,iphi,not_feasible,ife] = 0;
                    V[inno,educ,i_n,iphi,not_feasible,ife] = -(10**(5/gammac))**(1-gammac)/(1-gammac)
                    Sp[inno,educ,i_n,iphi,not_feasible,ife]= Spgrid[0]
                    
                    Tp[inno,educ,i_n,iphi,:,ife] = np.zeros(len(C[inno,educ,i_n,iphi,:,ife]))
        
        # Fill cell for all PHI;
        for iphi in range (1,len(PHI_pos)):
            for educ in range (0,len(EDUC)):
                for ife in range (0,len(FE_pos)):
                    for inno in range (0,len(FE_pos)):
                        C[inno,educ,i_n,iphi,:,ife]  = C[inno,educ,i_n,0,:,ife]
                        Ck[inno,educ,i_n,iphi,:,ife] = Ck[inno,educ,i_n,0,:,ife]
                        Sp[inno,educ,i_n,iphi,:,ife] = Sp[inno,educ,i_n,0,:,ife]
                        V[inno,educ,i_n,iphi,:,ife]  = V[inno,educ,i_n,0,:,ife]
                        Tp[inno,educ,i_n,iphi,:,ife] = Tp[inno,educ,i_n,0,:,ife]
                        boundgrid[inno,i_n,0,ife,educ] = boundgrid[inno,i_n,0,ife,educ]

#############
#PROBLEMS: 
# vp is worng as consequence of wrong Vp which is wrong from wrong V wrong itself because splVp is not correct and gives a shitty aproximantion

## Case with Children
    if options.Fertility == 'Endo': # Only need to solve case without children in benchmark case where we allow them to choose endogenous fertility
            in_1    = 1
    elif options.Fertility == 'Exo':
            in_1    = 0
            
    for educ in range (0,len(EDUC)):
    
        INNOp_prob  = par.inc.z_prob[educ][:,j_pos+1,:].T  # Note I need j_pos + 1 here!
        
        S          = par.grids[educ][j_pos]
        Spgrid  = par.grids[educ][j_pos+1]
    
        r         = (r_sav * (Spgrid>=0) + r_debt * (Spgrid<0)).T
        rs        = (r_sav * (S>=0) + r_debt * (S<0))
        
        inno3 = np.repeat(INNO_posT,len(S),axis=0)
        
        FE        = par.inc.fe[educ]
        INNO      = par.inc.z_val[educ][j_pos,:]
        
        INNO2,S2  = np.meshgrid(INNO_pos,Spgrid)
        
        for i_n in range(in_1,len(N)):
        #for i_n in range(in_1,in_1+1):
            #i_n = 1
            n             = par.N[i_n]##### careful here that par.N is [0,1,2,3] also in Matlab
            for iphi in range (0,len(PHI_pos)):
                phi        = par.PHI[iphi]
                for ife in range (0,len(FE_pos)):
                    splVp = GlobalSpline2D(Spgrid, INNO_pos,Vp[:,educ,:,ife])
                    for inno in range (0,len(INNO_pos)):
                        innop_prob      = INNOp_prob[inno,:]
                        h   = exp(AGE_PROF[educ][0][j_pos]) * exp(FE[ife])* exp(INNO[inno])*(1-Lambda)
                        
                        Cpp = Cp[:,educ,:,ife].T
                        if min(np.ravel(Cpp.any())<0 and abs(min(np.ravel(Cpp.any()))))>1e-6:
                            Cpp = max(Cpp,0)
                        ucp = beta*np.reshape(((1+r)),[80,])*np.dot(Cpp**(-gammac), np.reshape(innop_prob,[5,]))
                        
                        
                        labor_inc = w*h*(1-Lambda)
                        
                        n_final = n* (par.fam_size/2)# 3 as in matlab
                        ChildCost_0 = ChildCost(par,labor_inc,n_final,0,options)
                        ChildCost_1 = ChildCost(par,labor_inc,n_final,1,options)
                        ChildCost_opt = min(ChildCost_0,ChildCost_1)
                        Tp[inno,educ,i_n,iphi,:,ife]   = (ChildCost_1>ChildCost_0)
                        
                        dispinc = labor_inc- ChildCost_opt -  n_final  *phi
                        feasible = (dispinc + (1+rs)*S - Spgrid[0]>0)
                        not_feasible = (dispinc + (1+rs)*S - Spgrid[0]<=0)
                    
         ########################### Altruism
                        hp              = exp(AGE_PROF[educ][0][j_pos]) * exp(FE[ife])* exp(INNO[inno])
                        Vc0_aux     = np.reshape(Vc0[:,iphi,:],[par.N_fe*len(par.psy_val_hs)])
                        
                        
                        Vc0_prob_h0 = PG(hp)
                        Vc0_prob_psy= par.psy_prob[educ,:].T
                        Vc0_prob    = gridmake(Vc0_prob_psy,Vc0_prob_h0).T
                        Vc0_prob    = Vc0_prob[:,1] * Vc0_prob[:,0]
                        
                        Gn = beta * lambdan * ( n_final )**(gamman) * np.dot(Vc0_aux.T, Vc0_prob)
                        
                        if options.Ck == 'Yes':
                                C[inno,educ,i_n,iphi,feasible,ife],Sp[inno,educ,i_n,iphi,feasible,ife],boundgrid[inno,i_n,iphi,ife,educ] = EGM_withchild(par,ucp,dispinc,Spgrid,S[feasible],n_final) 

                                f_n     = (lambdan/n_final**(1-gamman))**(1/gammac)
                                Ck[inno,educ,i_n,iphi,feasible,ife]     = f_n * C[inno,educ,i_n,iphi,feasible,ife]
                                                          
                                Sp3 = np.repeat(np.reshape(Sp[inno,educ,i_n,iphi,:,ife],[len(feasible),1]),len(INNO_pos),axis = 1)   
                                
                                vp = np.dot((splVp(Sp3[:,0],inno3[0,:]).T),innop_prob)
                                # for j in range(0,len(vp)):
                                #     if math.isnan(vp[j]):
                                #         vp[j] = -inf
                                # vp = np.sort(vp)
                                # for j in range(0,len(vp)):
                                #     if math.isinf(vp[j]):
                                #         vp[j] = nan

                                  
                    
   
                                V[inno,educ,i_n,iphi,feasible,ife] = (C[inno,educ,i_n,iphi,feasible,ife]**(1-gammac))/(1-gammac)*(C[inno,educ,i_n,iphi,feasible,ife]>=0) + lambdan* n_final**(gamman) * (Ck[inno,educ,i_n,iphi,feasible,ife]**(1-gammac))/(1-gammac)*(C[inno,educ,i_n,iphi,feasible,ife]>=0) -((10**(5/gammac))**(1-gammac))/(1-gammac)*(C[inno,educ,i_n,iphi,feasible,ife]<0) + beta*vp[feasible]*(C[inno,educ,i_n,iphi,feasible,ife]>=0) + Gn*(C[inno,educ,i_n,iphi,feasible,ife]>=0)
                                # Fix bad extrapolation:
                                Sp[inno,educ,i_n,iphi,feasible,ife]  = Sp[inno,educ,i_n,iphi,feasible,ife]*(C[inno,educ,i_n,iphi,feasible,ife] >=0) + Spgrid[0]*(C[inno,educ,i_n,iphi,feasible,ife]<0)
                                C[inno,educ,i_n,iphi,feasible,ife]  = C[inno,educ,i_n,iphi,feasible,ife]*(C[inno,educ,i_n,iphi,feasible,ife] >= 0) + 0*(C[inno,educ,i_n,iphi,feasible,ife] <0)
                                Ck[inno,educ,i_n,iphi,feasible,ife] = Ck[inno,educ,i_n,iphi,feasible,ife]*(Ck[inno,educ,i_n,iphi,feasible,ife]>= 0) + 0*(Ck[inno,educ,i_n,iphi,feasible,ife]<0)

                                C[inno,educ,i_n,iphi,not_feasible,ife] = 0
                                Ck[inno,educ,i_n,iphi,not_feasible,ife] = 0
                                V[inno,educ,i_n,iphi,not_feasible,ife] = -(10**(5/gammac))**(1-gammac)/(1-gammac)
                                Sp[inno,educ,i_n,iphi,not_feasible,ife]= Spgrid[0]
                                
                        elif options.Ck == 'No':
                                C[inno,educ,i_n,iphi,feasible,ife],Sp[inno,educ,i_n,iphi,feasible,ife],boundgrid[inno,i_n,iphi,ife,educ] = EGM(par,ucp,dispinc,Spgrid,S[feasible])
                                Ck[inno,educ,i_n,iphi,feasible,ife] =np.zeros(len(C[inno,educ,i_n,iphi,feasible,ife]))
                                Sp3         = np.repeat(np.reshape(Sp[inno,educ,i_n,iphi,:,ife],[len(feasible),1]),len(INNO_pos),axis = 1)    
                                vp          = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)### wrong interpolation
                                vp = np.dot((splVp(Sp3[:,0],inno3[0,:]).T),innop_prob)
                                # for j in range(0,len(vp)):
                                #     if math.isnan(vp[j]):
                                #         vp[j] = -inf
                                # vp = np.sort(vp)
                                # for j in range(0,len(vp)):
                                #     if math.isinf(vp[j]):
                                #         vp[j] = nan
                                        
                                        
                                V[inno,educ,i_n,iphi,feasible,ife] = C[inno,educ,i_n,iphi,feasible,ife]**(1-gammac)/(1-gammac)*(C[inno,educ,i_n,iphi,feasible,ife]>=0) -(10**(5/gammac))**(1-gammac)/(1-gammac)*(C[inno,educ,i_n,iphi,feasible,ife]<0) + beta*vp[feasible]*(C[inno,educ,i_n,iphi,feasible,ife]>=0) + Gn*(C[inno,educ,i_n,iphi,feasible,ife]>=0)
                                
                                # Fix bad extrapolation:
                                Sp[inno,educ,i_n,iphi,feasible,ife] = Sp[inno,educ,i_n,iphi,feasible,ife]*(C[inno,educ,i_n,iphi,feasible,ife] >=0) + Spgrid[0]*(C[inno,educ,i_n,iphi,feasible,ife]<0)
                                C[inno,educ,i_n,iphi,feasible,ife] = C[inno,educ,i_n,iphi,feasible,ife]*(C[inno,educ,i_n,iphi,feasible,ife] >=0) + 0*(C[inno,educ,i_n,iphi,feasible,ife]<0)
                                C[inno,educ,i_n,iphi,not_feasible,ife] = 0
                                Ck[inno,educ,i_n,iphi,not_feasible,ife] = 0
                                V[inno,educ,i_n,iphi,not_feasible,ife] = -(10**(5/gammac))**(1-gammac)/(1-gammac)
                                Sp[inno,educ,i_n,iphi,not_feasible,ife]= Spgrid[0]


##Search Grid:
    V_2    = np.zeros([len(INNO_pos),len(EDUC),len(N),len(S),len(FE_pos)])
    V_2[:] = np.nan
    C_2    = np.zeros([len(INNO_pos),len(EDUC),len(N),len(S),len(FE_pos)])
    C_2[:] = np.nan
    Ck_2   = np.zeros([len(INNO_pos),len(EDUC),len(N),len(S),len(FE_pos)])
    Ck_2[:] = np.nan
    Sp_2   = np.zeros([len(INNO_pos),len(EDUC),len(N),len(S),len(FE_pos)])
    Sp_2[:] = np.nan
    PHIp_2 = np.zeros([len(INNO_pos),len(EDUC),len(N),len(S),len(FE_pos)])
    PHIp_2[:] = np.nan
    Tp_2   = np.zeros([len(INNO_pos),len(EDUC),len(N),len(S),len(FE_pos)])
    Tp_2[:] = np.nan
    
    posP = np.zeros([len(INNO_pos),len(EDUC),len(N),len(S),len(FE_pos)])
    for i_s in range (0,len(S)):
        
        for ife in range (0,len(FE_pos)):
                
            for inno in range (0,len(INNO_pos)):
                for educ in range (0,len(EDUC)):
                    for i_n in range (0,len(N)):
                        Vaux                           = np.squeeze(V[inno,educ,i_n,:,i_s,ife])     
                        posauxP                        = np.argmax(Vaux)### otherwise it would be zero and in Matlab it is 1
                        posP[inno,educ,i_n,i_s,ife]    = int64(posauxP)
                        V_2[inno,educ,i_n,i_s,ife]     = V[inno,educ,i_n,int64(posP[inno,educ,i_n,i_s,ife]),i_s,ife]################understand!!!!!!!!!!!!!!!!!!!!!!!!!!
                        C_2[inno,educ,i_n,i_s,ife]     = C[inno,educ,i_n,int64(posP[inno,educ,i_n,i_s,ife]),i_s,ife]
                        Ck_2[inno,educ,i_n,i_s,ife]    = Ck[inno,educ,i_n,int64(posP[inno,educ,i_n,i_s,ife]),i_s,ife]
                        Sp_2[inno,educ,i_n,i_s,ife]    = Sp[inno,educ,i_n,int64(posP[inno,educ,i_n,i_s,ife]),i_s,ife]
                        Tp_2[inno,educ,i_n,i_s,ife]    = Tp[inno,educ,i_n,int64(posP[inno,educ,i_n,i_s,ife]),i_s,ife]
                        PHIp_2[inno,educ,i_n,i_s,ife]  = posP[inno,educ,i_n,i_s,ife]
                        

    
    errors = sum(boundgrid[:])/len(C[:])
    if options.timer_on == 'Y':
            if sum(boundgrid[:]) >= len(EDUC)*len(FE_pos)*len(INNO_pos)*len(PHI_pos)*len(N):
                printf('j : {par.age(j_pos)}, Share of errors (increase grid) ={errors} \n')

    return V_2, C_2, Ck_2, Sp_2, PHIp_2, Tp_2
####
#Problems:
    
#in search Grid we use Vaux = np.squeeze(V[inno,educ,i_n,:,i_s,ife]) which determines the grid. Since my V[] is output of a bad approximation, numbers are messed up: all V_2, SP_2, Ck_2, PHIp_2

#########I am using C instead of V in Vaux but need to change that

In [86]:
# -*- coding: utf-8 -*-
"""
Created on Mon Sep 19 14:24:53 2022

@author: Giorgia
"""

# -*- coding: utf-8 -*-
"""
Created on Wed Sep 14 19:28:30 2022

@author: Giorgia
"""

# -*- coding: utf-8 -*-
"""
Created on Mon Sep  5 21:02:14 2022

@author: Giorgia
"""

#import matplotlib.pyplot as plt

import types, copy

def GEGM_withchild(par,ucp, dispinc, Spgrid, S, splVp, innop_prob,n):# inputs: ucp (1,80) (ucp.T compared to the ucp in prob_work_withchild), so c is (80,1)
# EGM
    #options = optimset('Display','off') #options = optimset(Name,Value) returns options with specified parameters set using one or more name-value pair arguments.
    #https://www.mathworks.com/help/matlab/ref/optimset.html
    #ucp = np.reshape(ucp,[1,len(ucp)])
    r_sav  = par.r_sav
    r_debt = par.r_debt
    gammac = par.gammac
    beta   = par.beta
    INNO_pos   = range(0,size(innop_prob)) #Because only in last period need to take expectations
    
    ## Find no-concave region
    difmin = np.append(ucp[:,0:-1]- ucp[:,1::],0) #wrong
    indmin = (difmin<0)
    
    difmax = np.append(0, ucp[:,1::] - ucp[:,0:-1]) #wrong
    indmax = (difmax>0)
    if sum(indmin)>0:
        #ucp = np.reshape(ucp,[len(ucp),1])
        vmin = np.min(ucp[:,indmin])
        vmax = np.max(ucp[:,indmax])
        #     vmin = min(ucp.*indmin+1e20.*(1-indmin));
        #     [vmax,~] = max(ucp.*indmax);
        
        imin = np.argmax(range(0,size(ucp))*(ucp>vmax))
        
        if vmin > np.min(ucp):
            imax = np.min(range(0,size(ucp))*(ucp<vmin)+1e20*(ucp>=vmin))
        elif vmin == np.min(ucp):
            imax = size(ucp)-1# hopefully ok as it is for GEGM
        
            #code.interact(local=locals())#https://docs.python.org/3/library/pdb.html

        
    else:
        imin = 0
        imax = 1

    
    # non concave region: [imin+1, imax-1]
    # figure
    # plot(Spgrid,ucp,Spgrid,vmin.*ones(size(Spgrid)),Spgrid,vmax.*ones(size(Spgrid)))
    
    
    ## Solve for a endogenous
    c = ( ucp.T)**(-1/gammac)
    lambdan = par.lambdan
    gamman  = par.gamman
    f_n     = (lambdan/n**(1-gamman))**(1/gammac)
    c_k     = f_n * c
    
    a_endo = (c + n*c_k + np.reshape(Spgrid,[len(Spgrid),1]) - dispinc )
    a_endo = a_endo/(1+r_sav)*(a_endo>=0) + a_endo/(1+r_debt)*(a_endo<0)
    
    # 
    # figure(2)
    ##############################################################
    # plot(a_endo,Spgrid)
    # fig, ax = plt.subplots()
    # ax.scatter(x=a_endo, y=Spgrid, marker='o', c='r', edgecolor='b')
    
    ## Find global solution:
    a_opt  = copy.deepcopy(a_endo)
    sp_opt = copy.deepcopy(Spgrid)
    c_opt  = copy.deepcopy(c)
    c_k_opt = copy.deepcopy(c_k)
    
    # Find global solution in non concave region
    if imin == 0:
        for i in range(int(imin),int(imax)):
            if imax != 1:
                
                if i == imin:
                    inno3,Sp3  = np.meshgrid(INNO_pos,Spgrid)
                       #Vpaux          = splVp(Sp3,inno3)*innop_prob.T
                    Vpaux          = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)
                    
                    curv         = 2
                    Spgrid_dense_neg = -np.linspace(0,(-Spgrid[0])**(1/curv),300)**curv
                    Spgrid_dense = np.append(Spgrid_dense_neg[::-1], np.linspace(1e-6**(1/curv),np.max(Spgrid[:])**(1/curv),1000)**curv)
            
                    inno3, Sp3  = np.meshgrid(INNO_pos,Spgrid_dense)
                    Vpaux_dense = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)
                    
                a_candidate = a_endo[i]
           
                c = ((1+r_sav)*a_candidate*(a_candidate>=0) + (1+r_debt)*a_candidate*(a_candidate<0)  +dispinc - Spgrid.T)/(1+n*f_n)
                c_k     = f_n * c
                
                objfunc = np.zeros(len(c))
                for j in range(0,len(c)):
                    if c[j]>=0:
                        objfunc[j] = c[j]**(1-gammac)/(1-gammac)+ lambdan*n**gamman *c_k[j]**(1-gammac)/(1-gammac) + beta*Vpaux[j]
                    else: 
                        objfunc[j] = (-(10**(5/gammac))**(1-gammac)/(1-gammac)) + beta*Vpaux[j]
                #objfunc = (c**(1-gammac)/(1-gammac))*(c>=0) + (-(10**(5/gammac))**(1-gammac)/(1-gammac)*(c<0)) + beta*Vpaux
               
                amax = np.argmax(objfunc)
               
                if amax != i: # Discard solution     
            #       fprintf('z:%3.2f,a''_candidate:%3.2f,a''_global:%3.2f, \n',a_candidate,Spgrid(i),Spgrid(amax))
            #       a_opt(i)  = nan;
            #       sp_opt(i) = nan;
            #       c_opt(i)  = nan;
                    c = ((1+r_sav)*a_candidate*(a_candidate>=0) + (1+r_debt)*a_candidate*(a_candidate<0)  +dispinc - Spgrid_dense.T)/(1+n*f_n)
                    c_k     = f_n * c
                    
                    objfunc = np.zeros(len(c))
                    for j in range(0,len(c)):
                        if c[j]>=0:
                            objfunc[j] = c[j]**(1-gammac)/(1-gammac)+ lambdan*n**gamman *c_k[j]**(1-gammac)/(1-gammac) + beta*Vpaux_dense[j]
                        else: 
                            objfunc[j] = (-(10**(5/gammac))**(1-gammac)/(1-gammac)) + beta*Vpaux_dense[j]
        
                    amax = np.argmax(objfunc)
                  
                    a_opt[i]  = a_candidate
                    sp_opt[i] = Spgrid_dense[amax]
                    c_opt[i]  = c[amax]
                    c_k_opt[i]  = f_n * c[amax]
    
    else:
        for i in range(int(imin+1),int(imax)):   
            if i == imin + 1:
                inno3,Sp3  = np.meshgrid(INNO_pos,Spgrid)
                   #Vpaux          = splVp(Sp3,inno3)*innop_prob.T
                Vpaux          = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)
                
                curv         = 2
                Spgrid_dense_neg = -np.linspace(0,(-Spgrid[0])**(1/curv),300)**curv
                Spgrid_dense = np.append(Spgrid_dense_neg[::-1], np.linspace(1e-6**(1/curv),np.max(Spgrid[:])**(1/curv),1000)**curv)
        
                inno3, Sp3  = np.meshgrid(INNO_pos,Spgrid_dense)
                Vpaux_dense = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)
                
            a_candidate = a_endo[i]
       
            c = ((1+r_sav)*a_candidate*(a_candidate>=0) + (1+r_debt)*a_candidate*(a_candidate<0)  +dispinc - Spgrid.T)/(1+n*f_n)
            c_k     = f_n * c
            
            objfunc = np.zeros(len(c))
            for j in range(0,len(c)):
                if c[j]>=0:
                    objfunc[j] = c[j]**(1-gammac)/(1-gammac)+ lambdan*n**gamman *c_k[j]**(1-gammac)/(1-gammac) + beta*Vpaux[j]
                else: 
                    objfunc[j] = (-(10**(5/gammac))**(1-gammac)/(1-gammac)) + beta*Vpaux[j]
            #objfunc = (c**(1-gammac)/(1-gammac))*(c>=0) + (-(10**(5/gammac))**(1-gammac)/(1-gammac)*(c<0)) + beta*Vpaux
           
            amax = np.argmax(objfunc)
           
            if amax != i: # Discard solution     
        #       fprintf('z:%3.2f,a''_candidate:%3.2f,a''_global:%3.2f, \n',a_candidate,Spgrid(i),Spgrid(amax))
        #       a_opt(i)  = nan;
        #       sp_opt(i) = nan;
        #       c_opt(i)  = nan;
                c = ((1+r_sav)*a_candidate*(a_candidate>=0) + (1+r_debt)*a_candidate*(a_candidate<0)  +dispinc - Spgrid_dense.T)/(1+n*f_n)
                c_k     = f_n * c
                
                objfunc = np.zeros(len(c))
                for j in range(0,len(c)):
                    if c[j]>=0:
                        objfunc[j] = c[j]**(1-gammac)/(1-gammac)+ lambdan*n**gamman *c_k[j]**(1-gammac)/(1-gammac) + beta*Vpaux_dense[j]
                    else: 
                        objfunc[j] = (-(10**(5/gammac))**(1-gammac)/(1-gammac)) + beta*Vpaux_dense[j]
    
                amax = np.argmax(objfunc)
              
                a_opt[i]  = a_candidate
                sp_opt[i] = Spgrid_dense[amax]
                c_opt[i]  = c[amax]
                c_k_opt[i]  = f_n * c[amax]

    # figure(3)
    # plot(a_endo,Spgrid,a_opt,sp_opt)
    
    ## Borrowing constraint
    # Case 1: Spgrid(1) is global solution
    if isnan(sp_opt[1]) == 0:
    #     fprintf('Borrowing constraint - global \n')
        a_bc    = a_opt[0] # Any current level of savings below this should be constrained 
        
        S_bc = []
        
        #a_endo = c + Spgrid.T- dispinc
        for i in range(0,len(S)):
            if np.any(S[i]<a_bc):
                S_bc.append(S[i])      
        S_bc = np.array(S_bc)# Borrowing constrained
         
        c_bc    = ((1+r_sav) * S_bc * (S_bc >= 0) + (1+r_debt) * S_bc * (S_bc < 0) + dispinc - Spgrid[0])/(1+n*f_n)
        c_k_bc  = f_n * c_bc
    else: # Approximate a_bc
        pos = np.argmin(sp_opt)
        a_opt1    = a_opt[pos]
        
        if a_opt1<S[0]:
    #         fprintf('Borrowing constraint - local, but never binding \n')
            a_bc    = a_opt[pos]
            S_bc = []
            
            #a_endo = c + Spgrid.T- dispinc
            for i in range(0,len(S)):
                if np.any(S[i]<a_bc):
                    S_bc.append(S[i])      
            S_bc = np.array(S_bc)# Borrowing constrained
            
            c_bc    = ((1+r_sav) * S_bc * (S_bc >= 0) + (1+r_debt) * S_bc * (S_bc < 0) + dispinc - Spgrid[0])/(1+n*f_n)
            c_k_bc  = f_n * c_bc
        else:
    #         fprintf('Borrowing constraint - local & potentially binding \n')
            S_bc = S[S<a_opt1]
            
            inno3,Sp3  = np.meshgrid(INNO_pos,Spgrid)
            
            Vpaux          = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)
            
            c = (np.repeat((1+r_sav) * S_bc * (S_bc >= 0) + (1+r_debt) * S_bc * (S_bc < 0),size(Spgrid.T)) +dispinc - np.repeat(Spgrid.T,size(S_bc)))/(1+n*f_n)
            c_k  = f_n * c
            
            objfunc = np.zeros(len(c))
            for j in range(0,len(c)):
                if c[j]>=0:
                    objfunc[j] = c[j]**(1-gammac)/(1-gammac)+ lambdan*n**gamman *c_k[j]**(1-gammac)/(1-gammac) + beta*np.repeat(Vpaux,size(S_bc))[j]
                else: 
                    objfunc[j] = (-(10**(5/gammac))**(1-gammac)/(1-gammac)) + beta*np.repeat(Vpaux,size(S_bc))[j]
            
            
            amax = np.argmax(objfunc, axis =0)
            Sp_vfi   = Spgrid[amax]
            c_bc    = (1+r_sav) * S_bc * (S_bc >= 0) + (1+r_debt) * S_bc * (S_bc < 0) + dispinc - Sp_vfi/(1+n*f_n)
            c_k_bc  = f_n * c_bc

 ##########################################################################   
 ##################################################################  ##so fa exactly like matlab 
    # Discard local solutions and sort grid
    a_opt = a_opt[(isnan(a_opt)==0)]
    c_opt = c_opt[(isnan(c_opt)==0)]
    c_k_opt = c_k_opt[(isnan(c_k_opt)==0)]
    ord = np.argsort(a_opt)
    a_opt = np.sort(a_opt)
    c_opt = c_opt[ord]
    c_k_opt = c_k_opt[ord]
    # Interpolate in originial grid S
    # (Since the borrowing limit is defined on strict inequality, all a_endo
    # should be included in interpolation)
    
    c_final  = approx_2d(np.append(S_bc, a_opt), np.append(c_bc, c_opt), S)
    c_k_final  = approx_2d(np.append(S_bc, a_opt), np.append(c_k_bc, c_k_opt), S)
    sp_final = (1+r_sav)*S.T*(S.T>=0) + (1+r_debt)*S.T*(S.T<0) + dispinc - c_final - n*c_k_final
    sp_final = sp_final * ((sp_final>=Spgrid[0]) +(sp_final<Spgrid[0]-1e-6)) + Spgrid[0] *(sp_final<Spgrid[0])*(sp_final>=Spgrid[0]-1e-6)
    
    
    inno3,Sp3  = np.meshgrid(INNO_pos,sp_final)
    Vpaux0          = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)

###################################################CAREFUL THAT objfunc is defined differently for c-final >00 or c_final <0!!!!!!!!!!!!!!!!!
    objfunc0 = np.zeros(len(c_final))

    for j in range(0,len(c_final)):
        if c_final[j]>=0:
            objfunc0[j] = c_final[j]**(1-gammac)/(1-gammac)+ lambdan*n**gamman *c_k_final[j]**(1-gammac)/(1-gammac)+ beta*Vpaux0[j]
            
        else: 
            objfunc0[j] = (-(10**(5/gammac))**(1-gammac)/(1-gammac)) + beta*Vpaux0[j]
    
    if sum(sp_final<Spgrid[0])>0:

        print('EGM error: savings are below debt limit - wrong extrapolation, min(s''): {min(sp_final)} \n')### must be fixed

    
    boundgrid = sum(sp_final>1.025*Spgrid[-1])
    ##############################################c_final still as good as in matlab
    # Check if there are slow decreases in C, i.e., not jumps
    difc = np.append(0, c_final[1::] - c_final[0:-1])
    inddecc = (difc<0)
    inddecc = (inddecc*((np.append(1, inddecc[0:-1])==1) + (np.append(inddecc[1::], 1) == 1))>0)
    inddecc[0:-2] = np.max([inddecc[0:-2],inddecc[2::]],axis =0)
    inddecc[2::] = np.max([inddecc[2::],  inddecc[0:-2]],axis = 0)
    inddecc[objfunc0<0] = 1
    inddecc[np.append(0, objfunc0[0:-1])<0] = 1 #inddedd 78 but objfunc is 80
        
    grid_prob = np.arange(0,len(c_final))
    do_interp = 1
    for i in (grid_prob[inddecc]):
        if do_interp == 1:
            curv         = 2
            Spgrid_dense_neg = -np.linspace(0,(-Spgrid[0])**(1/curv),300)**curv
            Spgrid_dense = np.append(Spgrid_dense_neg[::-1], np.linspace(1e-6**(1/curv),max(Spgrid[:])**(1/curv),1000)**curv)
    
            
            inno3, Sp3  = np.meshgrid(INNO_pos,Spgrid_dense)
            Vpaux_dense = np.dot(splVp(Spgrid_dense,INNO_pos).T,innop_prob)
            
            do_interp    = 0
        
        c = ((1+r_sav)*S[i]*(S[i]>=0) + (1+r_debt)*S[i]*(S[i]<0)  +dispinc - Spgrid_dense.T)/(1+n*f_n)
        c_k     = f_n * c
        
        objfunc = np.zeros(len(c))

        for j in range(0,len(c)):
            if c[j]>=0:
                objfunc[j] = c[j]**(1-gammac)/(1-gammac)+ lambdan*n**gamman *c_k[j]**(1-gammac)/(1-gammac)+ beta*Vpaux_dense[j]
                
            else: 
                objfunc[j] = (-(10**(5/gammac))**(1-gammac)/(1-gammac)) + beta*Vpaux_dense[j]

       
        #objfunc0 = c**(1-gammac)/(1-gammac) * (c>=0) + -(10**(5/gammac))**(1-gammac)/(1-gammac)*(c<0) + beta*Vpaux_dense
    
        amax = np.argmax(objfunc)
        sp_final[i] = Spgrid_dense[amax]
        c_final[i]  = c[amax] #######HERE cfinal[5:6] is wrong!!!!!!!!!!!!! becasue objfunc is  different and so in argmax (objfunc)
        c_k_final[i]  = f_n * c[amax]
    # check for bad extrapolation
    c_max   = ((1+r_sav)*S.T*(S.T>=0) + (1+r_debt)*S.T*(S.T<0)+dispinc-Spgrid[0])/(1+n*f_n)
    c_k_max   = f_n * c_max
    c_final = ( c_final<= c_max) * c_final + (c_final > c_max) * c_max
    c_k_final = ( c_k_final<= c_k_max) * c_k_final + (c_k_final > c_k_max) * c_k_max
    

    #code.interact(local=locals())#https://docs.python.org/3/library/pdb.html


    
    sp_final = (1+r_sav)*S.T*(S.T>=0) + (1+r_debt)*S.T*(S.T<0) + dispinc - c_final - n*c_k_final
    

    return c_final, sp_final, boundgrid

In [85]:
def GEGM(par,ucp, dispinc, Spgrid, S, splVp, innop_prob):
# EGM
    #options = optimset('Display','off') #options = optimset(Name,Value) returns options with specified parameters set using one or more name-value pair arguments.
    #https://www.mathworks.com/help/matlab/ref/optimset.html
    #ucp = ucp.T
    r_sav  = par.r_sav
    r_debt = par.r_debt
    gammac = par.gammac
    beta   = par.beta
    INNO_pos   = range(0,size(innop_prob)) #Because only in last period need to take expectations
    
    ## Find no-concave region
    difmin = np.append(ucp[:,0:-1]- ucp[:,1::],0)
    indmin = (difmin<0)
    
    difmax = np.append(0, ucp[:,1::] - ucp[:,0:-1])
    indmax = (difmax>0);
    if sum(indmin)>0:
        #ucp = np.reshape(ucp,[len(ucp),1])
        vmin = np.min(ucp[:,indmin])
        vmax = np.max(ucp[:,indmax])
        #     vmin = min(ucp.*indmin+1e20.*(1-indmin));
        #     [vmax,~] = max(ucp.*indmax);
        
        imin = np.argmax(range(0,size(ucp))*(ucp>vmax))
        
        if vmin > np.min(ucp):
            imax = np.min(range(0,size(ucp))*(ucp<vmin)+1e20*(ucp>=vmin))
        elif vmin == np.min(ucp):
            imax = size(ucp)-1
        else:
            pass
            #code.interact(local=locals())#https://docs.python.org/3/library/pdb.html

        
    else:
        imin = 0
        imax = 1

    
    # non concave region: [imin+1, imax-1]
    # figure
    # plot(Spgrid,ucp,Spgrid,vmin.*ones(size(Spgrid)),Spgrid,vmax.*ones(size(Spgrid)))
    

    ## Solve for a endogenous
    c = ( ucp.T)**(-1/gammac)
    a_endo = (c + np.reshape(Spgrid,[len(Spgrid),1]) - dispinc )####################huge mess with Spgrid to get it the right shape without messing up othe functions (e.g. splVp)
    a_endo = a_endo/(1+r_sav)*(a_endo>=0) + a_endo/(1+r_debt)*(a_endo<0)
    
    # 
    # figure(2)
    ##############################################################
    # plot(a_endo,Spgrid)
    # fig, ax = plt.subplots()
    # ax.scatter(x=a_endo, y=Spgrid, marker='o', c='r', edgecolor='b')
    
    ## Find global solution:
    a_opt  = copy.deepcopy(a_endo)
    sp_opt = copy.deepcopy(Spgrid)
    c_opt  = copy.deepcopy(c)
    
    # Find global solution in non concave region
    if imin == 0:
        for i in range(int(imin),int(imax)):
            if imax != 1:
                if i == imin:
                    if size(innop_prob)>1:
                        inno3,Sp3  = np.meshgrid(INNO_pos,Spgrid)
                       #Vpaux          = splVp(Sp3,inno3)*innop_prob.T
                        Vpaux          = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)
                    else:
                        Vpaux          = splVp(Spgrid).T######################problem
                    
                    curv         = 2
                    #curv = 3
                    Spgrid_dense_neg = -np.linspace(0,(-Spgrid[0])**(1/curv),300)**curv
                    Spgrid_dense = np.append(Spgrid_dense_neg[::-1], np.linspace(1e-6**(1/curv),np.max(Spgrid[:])**(1/curv),1000)**curv)
            
                    if size(innop_prob)>1:
                        inno3,Sp3  = np.meshgrid(INNO_pos,Spgrid_dense)
                        Vpaux_dense = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)
                    else:
                        Vpaux_dense = splVp(Spgrid_dense).T
                a_candidate = a_endo[i]
               
                c = (1+r_sav)*a_candidate*(a_candidate>=0) + (1+r_debt)*a_candidate*(a_candidate<0)  +dispinc - Spgrid.T
                
                objfunc = np.zeros(len(c))
                for j in range(0,len(c)):
                    if c[j]>=0:
                        objfunc[j] = c[j]**(1-gammac)/(1-gammac) + beta*Vpaux[j]
                    else: 
                        objfunc[j] = (-(10**(5/gammac))**(1-gammac)/(1-gammac)) + beta*Vpaux[j]
                #objfunc = (c**(1-gammac)/(1-gammac))*(c>=0) + (-(10**(5/gammac))**(1-gammac)/(1-gammac)*(c<0)) + beta*Vpaux
               
                amax = np.argmax(objfunc)
           
                if amax != i: # Discard solution
                 
            #       fprintf('z:%3.2f,a''_candidate:%3.2f,a''_global:%3.2f, \n',a_candidate,Spgrid(i),Spgrid(amax))
            #       a_opt(i)  = nan;
            #       sp_opt(i) = nan;
            #       c_opt(i)  = nan;
                  
                    c = (1+r_sav)*a_candidate*(a_candidate>=0) + (1+r_debt)*a_candidate*(a_candidate<0)  +dispinc - Spgrid_dense.T
                    objfunc = np.zeros(len(c))
                    for j in range(0,len(c)):
                        if c[j]>=0:
                            objfunc[j] = c[j]**(1-gammac)/(1-gammac) + beta*Vpaux_dense[j]
                        else: 
                            objfunc[j] = (-(10**(5/gammac))**(1-gammac)/(1-gammac)) + beta*Vpaux_dense[j]
    
                    amax = np.argmax(objfunc)
                  
                    a_opt[i]  = a_candidate
                    sp_opt[i] = Spgrid_dense[amax]
                    c_opt[i]  = c[amax]

    else:
        
        for i in range(int(imin+1),int(imax)):
            if i == imin + 1:
                if size(innop_prob)>1:
                    inno3,Sp3  = np.meshgrid(INNO_pos,Spgrid)
                   #Vpaux          = splVp(Sp3,inno3)*innop_prob.T
                    Vpaux          = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)
                else:
                    Vpaux          = splVp(Spgrid).T######################problem
                
                curv         = 2
                #curv = 3
                Spgrid_dense_neg = -np.linspace(0,(-Spgrid[0])**(1/curv),300)**curv
                Spgrid_dense = np.append(Spgrid_dense_neg[::-1], np.linspace(1e-6**(1/curv),np.max(Spgrid[:])**(1/curv),1000)**curv)
        
                if size(innop_prob)>1:
                    inno3,Sp3  = np.meshgrid(INNO_pos,Spgrid_dense)
                    Vpaux_dense = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)
                else:
                    Vpaux_dense = splVp(Spgrid_dense).T
            a_candidate = a_endo[i]
           
            c = (1+r_sav)*a_candidate*(a_candidate>=0) + (1+r_debt)*a_candidate*(a_candidate<0)  +dispinc - Spgrid.T
            
            objfunc = np.zeros(len(c))
            for j in range(0,len(c)):
                if c[j]>=0:
                    objfunc[j] = c[j]**(1-gammac)/(1-gammac) + beta*Vpaux[j]
                else: 
                    objfunc[j] = (-(10**(5/gammac))**(1-gammac)/(1-gammac)) + beta*Vpaux[j]
            #objfunc = (c**(1-gammac)/(1-gammac))*(c>=0) + (-(10**(5/gammac))**(1-gammac)/(1-gammac)*(c<0)) + beta*Vpaux
           
            amax = np.argmax(objfunc)
           
            if amax != i: # Discard solution
             
        #       fprintf('z:%3.2f,a''_candidate:%3.2f,a''_global:%3.2f, \n',a_candidate,Spgrid(i),Spgrid(amax))
        #       a_opt(i)  = nan;
        #       sp_opt(i) = nan;
        #       c_opt(i)  = nan;
              
                c = (1+r_sav)*a_candidate*(a_candidate>=0) + (1+r_debt)*a_candidate*(a_candidate<0)  +dispinc - Spgrid_dense.T
                objfunc = np.zeros(len(c))
                for j in range(0,len(c)):
                    if c[j]>=0:
                        objfunc[j] = c[j]**(1-gammac)/(1-gammac) + beta*Vpaux_dense[j]
                    else: 
                        objfunc[j] = (-(10**(5/gammac))**(1-gammac)/(1-gammac)) + beta*Vpaux_dense[j]
    
                amax = np.argmax(objfunc)
              
                a_opt[i]  = a_candidate
                sp_opt[i] = Spgrid_dense[amax]
                c_opt[i]  = c[amax]
    
    # figure(3)
    # plot(a_endo,Spgrid,a_opt,sp_opt)
    
    ## Borrowing constraint
    # Case 1: Spgrid(1) is global solution
    if isnan(sp_opt[1]) == 0:
    #     fprintf('Borrowing constraint - global \n')
        a_bc    = a_opt[0] # Any current level of savings below this should be constrained 
        
        S_bc = []
        
        #a_endo = c + Spgrid.T- dispinc
        for i in range(0,len(S)):
            if np.any(S[i]<a_bc):
                S_bc.append(S[i])      
        S_bc = np.array(S_bc)# Borrowing constrained
         
        c_bc    = (1+r_sav) * S_bc * (S_bc >= 0) + (1+r_debt) * S_bc * (S_bc < 0) + dispinc - Spgrid[0]
    #     fprintf('Borrowing constraint - global \n')

    else: # Approximate a_bc
        pos = np.argmin(sp_opt)
        a_opt1    = a_opt[pos]
        
        if a_opt1<S[0]:
    #         fprintf('Borrowing constraint - local, but never binding \n')
            a_bc    = a_opt[pos]
            
            S_bc = []
            
            #a_endo = c + Spgrid.T- dispinc
            for i in range(0,len(S)):
                if np.any(S[i]<a_bc):
                    S_bc.append(S[i])      
            S_bc = np.array(S_bc)# Borrowing constrained
            
            c_bc    = (1+r_sav) * S_bc * (S_bc >= 0) + (1+r_debt) * S_bc * (S_bc < 0) + dispinc - Spgrid[0]
        else:
    #         fprintf('Borrowing constraint - local & potentially binding \n')
            S_bc = S[S<a_opt1]
            
            if size(innop_prob)>1:
                inno3,Sp3  = np.meshgrid(INNO_pos,Spgrid)
               #Vpaux          = splVp(Sp3,inno3)*innop_prob.T
                Vpaux          = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)
            else:
                Vpaux          = splVp(Spgrid).T
          
            
            c = np.repeat((1+r_sav) * S_bc * (S_bc >= 0) + (1+r_debt) * S_bc * (S_bc < 0),size(Spgrid.T)) +dispinc - np.repeat(Spgrid.T,size(S_bc))
            
            objfunc = np.zeros(len(c))
            for j in range(0,len(c)):
                if c[j]>=0:
                    objfunc[j] = c[j]**(1-gammac)/(1-gammac) + beta*np.repeat(Vpaux,size(S_bc))[j]
                else: 
                    objfunc[j] = (-(10**(5/gammac))**(1-gammac)/(1-gammac)) + beta*np.repeat(Vpaux,size(S_bc))[j]
            
            
            amax = np.argmax(objfunc, axis =0)
            Sp_vfi   = Spgrid[amax]
            c_bc    = (1+r_sav) * S_bc * (S_bc >= 0) + (1+r_debt) * S_bc * (S_bc < 0) + dispinc - Sp_vfi

 ##########################################################################   
 ##################################################################  ##so fa exactly like matlab 
    # Discard local solutions and sort grid
    a_opt = a_opt[(isnan(a_opt)==0)]
    c_opt = c_opt[(isnan(c_opt)==0)]
    ord = np.argsort(a_opt)
    a_opt = np.sort(a_opt)
    c_opt = c_opt[ord]
    
    # Interpolate in originial grid S
    # (Since the borrowing limit is defined on strict inequality, all a_endo
    # should be included in interpolation)
    
    c_final  = approx_2d(np.append(S_bc, a_opt), np.append(c_bc, c_opt), S)
    sp_final = (1+r_sav)*S.T*(S.T>=0) + (1+r_debt)*S.T*(S.T<0) + dispinc - c_final 
    sp_final = sp_final * ((sp_final>=Spgrid[0]) +(sp_final<Spgrid[0]-1e-6)) + Spgrid[0] *(sp_final<Spgrid[0])*(sp_final>=Spgrid[0]-1e-6)
    
    if size(innop_prob)>1:
        inno3,Sp3  = np.meshgrid(INNO_pos,sp_final)
        Vpaux0          = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)
    else:
        Vpaux0          = splVp(sp_final)
###################################################CAREFUL THAT objfunc is defined differently for c-final >00 or c_final <0!!!!!!!!!!!!!!!!!
    objfunc0 = np.zeros(len(c_final))

    for j in range(0,len(c_final)):
        if c_final[j]>=0:
            objfunc0[j] = c_final[j]**(1-gammac)/(1-gammac)+ beta*Vpaux0[j]
            
        else: 
            objfunc0[j] = (-(10**(5/gammac))**(1-gammac)/(1-gammac)) + beta*Vpaux0[j]
    
    if sum(sp_final<Spgrid[0])>0:

        print('EGM error: savings are below debt limit - wrong extrapolation, min(s''): {min(sp_final)} \n')### must be fixed

    
    boundgrid = sum(sp_final>1.025*Spgrid[-1])
    ##############################################c_final still as good as in matlab
    # Check if there are slow decreases in C, i.e., not jumps
    difc = np.append(0, c_final[1::] - c_final[0:-1])
    inddecc = (difc<0)
    inddecc = (inddecc*((np.append(1, inddecc[0:-1])==1) + (np.append(inddecc[1::], 1) == 1))>0)
    inddecc[0:-2] = np.max([inddecc[0:-2],inddecc[2::]],axis =0)
    inddecc[2::] = np.max([inddecc[2::],  inddecc[0:-2]],axis = 0)
    inddecc[objfunc0<0] = 1
    inddecc[np.append(0, objfunc0[0:-1])<0] = 1 #inddedd 78 but objfunc is 80
######################################################################################here mistake for i = 61, amax should be 1 + amax given by the code        
    grid_prob = np.arange(0,len(c_final))
    do_interp = 1
    for i in (grid_prob[inddecc]):
        if do_interp == 1:
            curv         = 2
            Spgrid_dense_neg = -np.linspace(0,(-Spgrid[0])**(1/curv),300)**curv
            Spgrid_dense = np.append(Spgrid_dense_neg[::-1], np.linspace(1e-6**(1/curv),max(Spgrid[:])**(1/curv),1000)**curv)
    
            if size(innop_prob)>1:
                inno3, Sp3  = np.meshgrid(INNO_pos,Spgrid_dense)
                Vpaux_dense = np.dot(splVp(Spgrid_dense,INNO_pos).T,innop_prob)
            else:
                Vpaux_dense          = splVp(Spgrid_dense)
            
            do_interp    = 0
        
        c = (1+r_sav)*S[i]*(S[i]>=0) + (1+r_debt)*S[i]*(S[i]<0)  +dispinc - Spgrid_dense.T
        
        objfunc = np.zeros(len(c))

        for j in range(0,len(c)):
            if c[j]>=0:
                objfunc[j] = c[j]**(1-gammac)/(1-gammac)+ beta*Vpaux_dense[j]
                
            else: 
                objfunc[j] = (-(10**(5/gammac))**(1-gammac)/(1-gammac)) + beta*Vpaux_dense[j]

       
        #objfunc0 = c**(1-gammac)/(1-gammac) * (c>=0) + -(10**(5/gammac))**(1-gammac)/(1-gammac)*(c<0) + beta*Vpaux_dense
    
        amax = np.argmax(objfunc)
        sp_final[i] = Spgrid_dense[amax]
        c_final[i]  = c[amax] #######HERE cfinal[5:6] is wrong!!!!!!!!!!!!! becasue objfunc is  different and so in argmax (objfunc)
        
    # check for bad extrapolation
    c_max   = (1+r_sav)*S.T*(S.T>=0) + (1+r_debt)*S.T*(S.T<0)+dispinc-Spgrid[0]
    c_final = ( c_final<= c_max) * c_final + (c_final > c_max) * c_max
    
    if min(c_final)<0:
        pass
    
    sp_final = (1+r_sav)*S.T*(S.T>=0) + (1+r_debt)*S.T*(S.T<0) + dispinc - c_final 
    

    return c_final, sp_final, boundgrid

In [84]:
# -*- coding: utf-8 -*-
"""
Created on Mon Sep  5 17:45:09 2022

@author: Giorgia
"""

        
def prob_work_with_child(par,options,Vp,Cp,j_pos):
# Solve household problem when j = Jc+1: children consume at home
# keyboard
    r_sav     = par.r_sav
    r_debt    = par.r_debt
    beta      = par.beta
    gammac    = par.gammac
    w         = par.w
    Lambda    = par.Lambda
    lambdan   = par.lambdan
    gamman    = par.gamman
    
    S         = par.grids[0][j_pos]
    EDUC      = par.educ
    
    AGE_PROF   = par.inc.age_prof
    INNO_pos   = par.inc.inno_pos
    INNO_posT = np.reshape(INNO_pos,[1,5])
    FE_pos     = par.inc.fe_pos
    
    N         = np.arange(0,4,1)
    
    Sp = np.zeros([len(INNO_pos),len(EDUC),len(N),len(S),len(FE_pos)])
    C  = np.zeros([len(INNO_pos),len(EDUC),len(N),len(S),len(FE_pos)])
    V  = np.zeros([len(INNO_pos),len(EDUC),len(N),len(S),len(FE_pos)])
    Ck = np.zeros([len(INNO_pos),len(EDUC),len(N),len(S),len(FE_pos)])
    Tp = np.zeros([len(INNO_pos),len(EDUC),len(N),len(S),len(FE_pos)])
    boundgrid = np.zeros([len(INNO_pos),len(N),len(FE_pos),len(EDUC)])
    
    ## Without children and phi = 1
    if options.Fertility == 'Endo': # Only need to solve case without children in benchmark case where we allow them to choose endogenous fertility
            i_n   = 0
            
            for educ in range (0,len(EDUC)):
                #parameters(par_est, options)
                S          = par.grids[educ][j_pos]
                Spgrid  = par.grids[educ][j_pos+1]
                
                FE        = par.inc.fe[educ]
                INNO      = par.inc.z_val[educ][j_pos,:]
                
                
                r         = (r_sav * (Spgrid>=0) + r_debt * (Spgrid<0)).T
                rs        = (r_sav * (S>=0) + r_debt * (S<0))
                
                inno3 = np.repeat(INNO_posT,len(S),axis=0)
                INNOp_prob = par.inc.z_prob[educ][:,j_pos+1,:].T # Note I need j_pos + 1 here!

                INNO2,S2  = np.meshgrid(INNO_pos,Spgrid)
                
                
                for ife in range (0,len(FE_pos)):
                    splVp = GlobalSpline2D(Spgrid, INNO_pos,Vp[:,educ,i_n,:,ife] )
                    
                    
                    for inno  in range (0,len(INNO_pos)):
                        
                   
                        innop_prob = INNOp_prob[inno,:]
                        
                        
                        h   = exp(AGE_PROF[educ][0][j_pos]) * exp(FE[ife])* exp(INNO[inno])
                        
                        Cpp = Cp[:,educ,i_n,:,ife].T
                        
                        if min(np.ravel(Cpp.any())<0 and abs(min(np.ravel(Cpp.any()))))>1e-6:
                            Cpp = max(Cpp,0)
                        ucp = beta*np.reshape(((1+r)),[80,])*np.dot(Cpp**(-gammac), np.reshape(innop_prob,[5,]))

                        dispinc = w*h*(1-Lambda)
                        feasible = ((dispinc + (1+rs)*S - Spgrid[0])>0)
                        
                        not_feasible = (dispinc + (1+rs)*S - Spgrid[0]<=0)
                        
                        C[inno,educ,i_n,feasible,ife],Sp[inno,educ,i_n,feasible,ife],boundgrid[inno,i_n,ife,educ] = EGM(par,ucp.T,dispinc,Spgrid,S[feasible])
                        Ck[inno,educ,i_n,feasible,ife] = np.zeros(len(C[inno,educ,i_n,feasible,ife]))
                        
                        
                        Sp3 = np.repeat(np.reshape(Sp[inno,educ,i_n,feasible,ife],[80,1]),len(INNO_pos),axis = 1)
                        
                        vp = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)
                        
                        V[inno,educ,i_n,feasible,ife] = C[inno,educ,i_n,feasible,ife]**(1-gammac)/(1-gammac) + beta*vp[feasible]
                        
                        C[inno,educ,i_n,not_feasible,ife] = 0;
                        V[inno,educ,i_n,not_feasible,ife] = -(10**(5/gammac))**(1-gammac)/(1-gammac)
                        Sp[inno,educ,i_n,not_feasible,ife]= Spgrid[0]
                        
                        
                        Tp[inno,educ,i_n,:,ife] = np.zeros(len(C[inno,educ,i_n,:,ife]))

    
    ## Case with Children
    if options.Fertility == 'Endo': #Only need to solve case without children in benchmark case where we allow them to choose endogenous fertility
            in_1    = 1
    elif options.Fertility == 'Exo':
            in_1    = 0

    
    for educ in range (0,len(EDUC)):
        
        INNOp_prob  = par.inc.z_prob[educ][:,j_pos+1,:].T  # Note I need j_pos + 1 here!
        
        S          = par.grids[educ][j_pos]
        Spgrid     = par.grids[educ][j_pos+1]
    
        r         = (r_sav * (Spgrid>=0) + r_debt * (Spgrid<0)).T
        rs        = (r_sav * (S>=0) + r_debt * (S<0))
        
        inno3 = np.repeat(INNO_posT,len(S),axis=0)
        
        FE        = par.inc.fe[educ]
        INNO      = par.inc.z_val[educ][j_pos,:]
        
        INNO2,S2  = np.meshgrid(INNO_pos,Spgrid)
        
        for i_n in range(in_1,len(N)):
            n             = par.N[i_n]
            for ife in range (0,len(FE_pos)):
                splVp = GlobalSpline2D(Spgrid, INNO_pos,Vp[:,educ,i_n,:,ife])
                for inno in range (0,len(INNO_pos)):
                    innop_prob      = INNOp_prob[inno,:]
                    h   = exp(AGE_PROF[educ][0][j_pos]) * exp(FE[ife])* exp(INNO[inno])
                    
                    Cpp = np.squeeze(Cp[:,educ,i_n,:,ife].T)
                    if min(np.ravel(Cpp.any())<0 and abs(min(np.ravel(Cpp.any()))))>1e-6:
                        Cpp = max(Cpp,0)
                    ucp = beta*np.reshape(((1+r)),[80,1])*np.dot(Cpp**(-gammac), np.reshape(innop_prob,[5,1]))
                    
                    labor_inc = w*h*(1-Lambda)
                    
                    n_final = n* (par.fam_size/2)
                    
                    ChildCost_0 = ChildCost(par,labor_inc,n_final,0,options)
                    ChildCost_1 = ChildCost(par,labor_inc,n_final,1,options)
                    ChildCost_opt = min(ChildCost_0,ChildCost_1)
                    Tp[inno,educ,i_n,:,ife]   = (ChildCost_1>ChildCost_0)
                    
                    dispinc = labor_inc- ChildCost_opt
                    feasible = (dispinc + (1+rs)*S - Spgrid[0]>0)
                    not_feasible = (dispinc + (1+rs)*S - Spgrid[0]<=0)
                    
                    
                    if options.Ck == 'Yes':
                        
                        C[inno,educ,i_n,feasible,ife],Sp[inno,educ,i_n,feasible,ife],boundgrid[inno,i_n,ife,educ] = GEGM_withchild(par,ucp.T,dispinc,Spgrid,S[feasible],splVp,innop_prob,n_final) 
                       # parameters(par_est, options)
                        Spgrid     = par.grids[educ][j_pos+1]#                        EGM_withchild(par,ucp,dispinc,Spgrid,S,n_final); %%%% To do: Generalized EGM with child
                        
                        f_n     = (lambdan/n_final**(1-gamman))**(1/gammac)
                        Ck[inno,educ,i_n,feasible,ife]     = f_n * C[inno,educ,i_n,feasible,ife]
                        
                        
                        Sp3 = np.repeat(np.reshape(Sp[inno,educ,i_n,:,ife],[len(Spgrid),1]),len(INNO_pos),axis = 1)   #######wrong!! but Sp[] ok                 
                        vp = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)
                        
                        
                        V[inno,educ,i_n,feasible,ife] = (C[inno,educ,i_n,feasible,ife]**(1-gammac))/(1-gammac)*(C[inno,educ,i_n,feasible,ife]>=0) + lambdan* n_final**(gamman) * (Ck[inno,educ,i_n,feasible,ife]**(1-gammac))/(1-gammac)*(C[inno,educ,i_n,feasible,ife]>=0) -((10**(5/gammac))**(1-gammac))/(1-gammac)*(C[inno,educ,i_n,feasible,ife]<0) + beta*vp[feasible]
                        # Fix bad extrapolation:
                        Sp[inno,educ,i_n,feasible,ife]  = Sp[inno,educ,i_n,feasible,ife]*(C[inno,educ,i_n,feasible,ife] >=0) + Spgrid[0]*(C[inno,educ,i_n,feasible,ife]<0)
                        C[inno,educ,i_n,feasible,ife]  = C[inno,educ,i_n,feasible,ife]*(C[inno,educ,i_n,feasible,ife] >= 0) + 0*(C[inno,educ,i_n,feasible,ife] <0)
                        Ck[inno,educ,i_n,feasible,ife] = Ck[inno,educ,i_n,feasible,ife]*(Ck[inno,educ,i_n,feasible,ife]>= 0) + 0*(Ck[inno,educ,i_n,feasible,ife]<0)

                        C[inno,educ,i_n,not_feasible,ife] = 0
                        Ck[inno,educ,i_n,not_feasible,ife] = 0
                        V[inno,educ,i_n,not_feasible,ife] = -(10**(5/gammac))**(1-gammac)/(1-gammac)
                        Sp[inno,educ,i_n,not_feasible,ife]= Spgrid[0]


                        
                    elif options.Ck == 'No':
                        C[inno,educ,i_n,feasible,ife],Sp[inno,educ,i_n,feasible,ife],boundgrid[inno,i_n,ife,educ]= GEGM(par,ucp.T,dispinc,Spgrid,S[feasible],splVp,innop_prob)
                        
                        Ck[inno,educ,i_n,feasible,ife] = np.zeros(len(C[inno,educ,i_n,feasible,ife]))
                        
                        Sp3         = np.repeat(np.reshape(Sp[inno,educ,i_n,:,ife],[len(feasible),1]),len(INNO_pos),axis = 1)    
                        vp          = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)### wrong interpolation
                        

                        V[inno,educ,i_n,feasible,ife] = C[inno,educ,i_n,feasible,ife]**(1-gammac)/(1-gammac)*(C[inno,educ,i_n,feasible,ife]>=0) -(10**(5/gammac))**(1-gammac)/(1-gammac)*(C[inno,educ,i_n,feasible,ife]<0) + beta*vp[feasible]

                        
                        C[inno,educ,i_n,not_feasible,ife] = 0
                        Ck[inno,educ,i_n,not_feasible,ife] = 0
                        V[inno,educ,i_n,not_feasible,ife] = -(10**(5/gammac))**(1-gammac)/(1-gammac)
                        Sp[inno,educ,i_n,not_feasible,ife]= Spgrid[0]
                        


    errors = sum(boundgrid[:])/len(C[:])
    if options.timer_on == 'Y':
        if sum(boundgrid[:]) >= len(EDUC)*len(FE_pos)*len(INNO_pos)*len(N):
            print('j : {par.age(j_pos)}, Share of errors (increase grid) ={errors} \n')
    return V,C,Ck,Sp,Tp

####To Do:
    #outcome of V[] is messed up for small or negative numbers in first rows. OFTEN IT SKIPS NUMBERS!!


In [83]:
# -*- coding: utf-8 -*-
"""
GEGM COLLEGE

@author: Giorgia
"""
import numpy as np

import code
#import matplotlib.pyplot as plt


def GEGM_college(par,ucp, dispinc, Spgrid, S, splVp, innop_prob):# inputs: ucp (1,80) (ucp.T compared to the ucp in prob_work_withchild), so c is (80,1)
# EGM
    #options = optimset('Display','off') #options = optimset(Name,Value) returns options with specified parameters set using one or more name-value pair arguments.
    #https://www.mathworks.com/help/matlab/ref/optimset.html
    r_sav  = par.r_sav
    r_debt = par.r_debt
    gammac = par.gammac
    beta   = par.beta
    INNO_pos   = range(0,size(innop_prob)) #Because only in last period need to take expectations
    
    ## Find no-concave region
    difmin = np.append(ucp[:,0:-1]- ucp[:,1::],0)
    indmin = (difmin<0)
    
    difmax = np.append(0, ucp[:,1::] - ucp[:,0:-1])
    indmax = (difmax>0);
    if sum(indmin)>0:
        #ucp = np.reshape(ucp,[len(ucp),1])
        vmin = np.min(ucp[:,indmin])
        vmax = np.max(ucp[:,indmax])
        #     vmin = min(ucp.*indmin+1e20.*(1-indmin));
        #     [vmax,~] = max(ucp.*indmax);
        
        imin = np.argmax(range(0,size(ucp))*(ucp>vmax))
        
        if vmin > np.min(ucp):
            imax = np.min(range(0,size(ucp))*(ucp<vmin)+1e20*(ucp>=vmin))
        elif vmin == np.min(ucp):
            imax = size(ucp)-1## hopefully -1 is ok, it was for GEGM
        
            #code.interact(local=locals())#https://docs.python.org/3/library/pdb.html

        
    else:
        imin = 0
        imax = 1

    
    # non concave region: [imin+1, imax-1]
    # figure
    # plot(Spgrid,ucp,Spgrid,vmin.*ones(size(Spgrid)),Spgrid,vmax.*ones(size(Spgrid)))
    
    
    ## Solve for a endogenous
    c = ( ucp.T)**(-1/gammac)
##    
    col_fact = (par.col_fact * (Spgrid<0) + 1 * (Spgrid>=0))
    col_fact = np.reshape(col_fact,[size(col_fact),1])
    lambdan = par.lambdan
    gamman  = par.gamman
    
    
    a_endo = (c +  np.reshape(Spgrid,[1,len(Spgrid)]).T*(1/col_fact) - dispinc )
    a_endo = a_endo/(1+r_sav)*(a_endo>=0) + a_endo/(1+r_debt)*(a_endo<0)
    
    # 
    # figure(2)
    ##############################################################
    # plot(a_endo,Spgrid)
    # fig, ax = plt.subplots()
    # ax.scatter(x=a_endo, y=Spgrid, marker='o', c='r', edgecolor='b')
    
    ## Find global solution:
    a_opt  = copy.deepcopy(a_endo)
    sp_opt = copy.deepcopy(Spgrid)
    c_opt  = copy.deepcopy(c)
    
    
    # Find global solution in non concave region
    if imin == 0:
        if imax != 1:
            for i in range(int(imin),int(imax)): #for i in range(int(imin+1),int(imax)):#################################IPERCAREFUL!!!!!!! i = 0 was excluded from the loop but chnaging loop starting from imin insteado of imin+1 it may mess tigs up!
                if i == imin: #if i == imin + 1:#################################IPERCAREFUL!!!!!!! i = 0 was excluded from the loop but it may mess tigs up!
                    if size(innop_prob)>1:
                        inno3,Sp3  = np.meshgrid(INNO_pos,Spgrid)
                       #Vpaux          = splVp(Sp3,inno3)*innop_prob.T
                        Vpaux          = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)
                    else:
                        Vpaux          = splVp(Spgrid).T######################problem
                    
                    curv         = 2
                    Spgrid_dense_neg = -np.linspace(0,(-Spgrid[0])**(1/curv),300)**curv
                    Spgrid_dense = np.append(Spgrid_dense_neg[::-1], np.linspace(1e-6**(1/curv),np.max(Spgrid[:])**(1/curv),1000)**curv)
        ####            
                    col_fact_dense = (par.col_fact * (Spgrid_dense<0) + 1 * (Spgrid_dense>=0))
                    col_fact_dense = np.reshape(col_fact_dense,[size(col_fact_dense),1])
                    
                    
                    if size(innop_prob)>1:
                        inno3,Sp3  = np.meshgrid(INNO_pos,Spgrid_dense)
                        Vpaux_dense = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)
                    else:
                        Vpaux_dense = splVp(Spgrid_dense).T
                a_candidate = a_endo[i]
               
                c = (1+r_sav)*a_candidate*(a_candidate>=0) + (1+r_debt)*a_candidate*(a_candidate<0)  +dispinc - np.reshape(Spgrid,[size(Spgrid),1])*(1./col_fact)
                
                objfunc = np.zeros(size(c))
                for j in range(0,size(c)):
                    if c[j]>=0:
                        objfunc[j] = c[j]**(1-gammac)/(1-gammac)+ beta*Vpaux[j]
                    else: 
                        objfunc[j] = (-(10**(5/gammac))**(1-gammac)/(1-gammac)) + beta*Vpaux[j]
                #objfunc = (c**(1-gammac)/(1-gammac))*(c>=0) + (-(10**(5/gammac))**(1-gammac)/(1-gammac)*(c<0)) + beta*Vpaux
               
                amax = np.argmax(objfunc)
               
                if amax != i: # Discard solution
                 
            #       fprintf('z:%3.2f,a''_candidate:%3.2f,a''_global:%3.2f, \n',a_candidate,Spgrid(i),Spgrid(amax))
            #       a_opt(i)  = nan;
            #       sp_opt(i) = nan;
            #       c_opt(i)  = nan;
                  
                    c = (1+r_sav)*a_candidate*(a_candidate>=0) + (1+r_debt)*a_candidate*(a_candidate<0)  +dispinc - np.reshape(Spgrid_dense,[size(Spgrid_dense),1])*(1./col_fact_dense)
                    
                    
                    objfunc = np.zeros(size(c))
                    for j in range(0,size(c)):
                        if c[j]>=0:
                            objfunc[j] = c[j]**(1-gammac)/(1-gammac)+ beta*Vpaux_dense[j]
                        else: 
                            objfunc[j] = (-(10**(5/gammac))**(1-gammac)/(1-gammac)) + beta*Vpaux_dense[j]
        
                    amax = np.argmax(objfunc)
                  
                    a_opt[i]  = a_candidate
                    sp_opt[i] = Spgrid_dense[amax]
                    c_opt[i]  = c[amax]
    else:
        for i in range(int(imin+1),int(imax)): # it may mess tigs up!
            if i == imin+1: 
                if size(innop_prob)>1:
                    inno3,Sp3  = np.meshgrid(INNO_pos,Spgrid)
                   #Vpaux          = splVp(Sp3,inno3)*innop_prob.T
                    Vpaux          = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)
                else:
                    Vpaux          = splVp(Spgrid).T######################problem
                
                curv         = 2
                Spgrid_dense_neg = -np.linspace(0,(-Spgrid[0])**(1/curv),300)**curv
                Spgrid_dense = np.append(Spgrid_dense_neg[::-1], np.linspace(1e-6**(1/curv),np.max(Spgrid[:])**(1/curv),1000)**curv)
    ####            
                col_fact_dense = (par.col_fact * (Spgrid_dense<0) + 1 * (Spgrid_dense>=0))
                col_fact_dense = np.reshape(col_fact_dense,[size(col_fact_dense),1])
                
                
                if size(innop_prob)>1:
                    inno3,Sp3  = np.meshgrid(INNO_pos,Spgrid_dense)
                    Vpaux_dense = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)
                else:
                    Vpaux_dense = splVp(Spgrid_dense).T
            a_candidate = a_endo[i]
           
            c = (1+r_sav)*a_candidate*(a_candidate>=0) + (1+r_debt)*a_candidate*(a_candidate<0)  +dispinc - np.reshape(Spgrid,[size(Spgrid),1])*(1./col_fact)
            
            objfunc = np.zeros(size(c))
            for j in range(0,size(c)):
                if c[j]>=0:
                    objfunc[j] = c[j]**(1-gammac)/(1-gammac)+ beta*Vpaux[j]
                else: 
                    objfunc[j] = (-(10**(5/gammac))**(1-gammac)/(1-gammac)) + beta*Vpaux[j]
            #objfunc = (c**(1-gammac)/(1-gammac))*(c>=0) + (-(10**(5/gammac))**(1-gammac)/(1-gammac)*(c<0)) + beta*Vpaux
           
            amax = np.argmax(objfunc)
           
            if amax != i: # Discard solution
             
        #       fprintf('z:%3.2f,a''_candidate:%3.2f,a''_global:%3.2f, \n',a_candidate,Spgrid(i),Spgrid(amax))
        #       a_opt(i)  = nan;
        #       sp_opt(i) = nan;
        #       c_opt(i)  = nan;
              
                c = (1+r_sav)*a_candidate*(a_candidate>=0) + (1+r_debt)*a_candidate*(a_candidate<0)  +dispinc - np.reshape(Spgrid_dense,[size(Spgrid_dense),1])*(1./col_fact_dense)
                
                
                objfunc = np.zeros(size(c))
                for j in range(0,size(c)):
                    if c[j]>=0:
                        objfunc[j] = c[j]**(1-gammac)/(1-gammac)+ beta*Vpaux_dense[j]
                    else: 
                        objfunc[j] = (-(10**(5/gammac))**(1-gammac)/(1-gammac)) + beta*Vpaux_dense[j]
    
                amax = np.argmax(objfunc)
              
                a_opt[i]  = a_candidate
                sp_opt[i] = Spgrid_dense[amax]
                c_opt[i]  = c[amax]
            
            
    
    # figure(3)
    # plot(a_endo,Spgrid,a_opt,sp_opt)
    
    ## Borrowing constraint
    # Case 1: Spgrid(1) is global solution
    col_fact_1 = (par.col_fact * (Spgrid[0]<0) + 1 * (Spgrid[0]>=0))
    if isnan(sp_opt[1]) == 0:
    #     fprintf('Borrowing constraint - global \n')
        a_bc    = a_opt[0] # Any current level of savings below this should be constrained 
        
        S_bc = []
        
        #a_endo = c + Spgrid.T- dispinc
        for i in range(0,len(S)):
            if np.any(S[i]<a_bc):
                S_bc.append(S[i])      
        S_bc = np.array(S_bc)# Borrowing constrained
         
        c_bc    = (1+r_sav) * S_bc * (S_bc >= 0) + (1+r_debt) * S_bc * (S_bc < 0) + dispinc - Spgrid[0]*(1/col_fact_1)
        
    else: # Approximate a_bc
        pos = np.argmin(sp_opt)
        a_opt1    = a_opt[pos]
        
        if a_opt1<S[0]:
    #         fprintf('Borrowing constraint - local, but never binding \n')
            a_bc    = a_opt[pos]
            S_bc = []
            
            #a_endo = c + Spgrid.T- dispinc
            for i in range(0,len(S)):
                if np.any(S[i]<a_bc):
                    S_bc.append(S[i])      
            S_bc = np.array(S_bc)# Borrowing constrained
            
            c_bc    = (1+r_sav) * S_bc * (S_bc >= 0) + (1+r_debt) * S_bc * (S_bc < 0) + dispinc - Spgrid[0]*(1/col_fact_1)
            
        else:
    #         fprintf('Borrowing constraint - local & potentially binding \n')
            S_bc = S[S<a_opt1]
            
            if size(innop_prob)>1:
               inno3,Sp3  = np.meshgrid(INNO_pos,Spgrid)
               #Vpaux          = splVp(Sp3,inno3)*innop_prob.T
               Vpaux          = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)
            else:
               Vpaux          = splVp(Spgrid).T
          
            
            c = (np.repeat((1+r_sav) * S_bc * (S_bc >= 0) + (1+r_debt) * S_bc * (S_bc < 0),size(Spgrid.T)) +dispinc - np.repeat(Spgrid.T*(1/col_fact_1),size(S_bc)))
            c_k  = f_n * c
            
            objfunc = np.zeros(len(c))
            for j in range(0,len(c)):
                if c[j]>=0:
                    objfunc[j] = c[j]**(1-gammac)/(1-gammac) + beta*np.repeat(Vpaux,size(S_bc))[j]
                else: 
                    objfunc[j] = (-(10**(5/gammac))**(1-gammac)/(1-gammac)) + beta*np.repeat(Vpaux,size(S_bc))[j]
            
            
            amax = np.argmax(objfunc, axis =0)
            Sp_vfi   = Spgrid[amax]
            col_fact_vfi = (par.col_fact * (Sp_vfi<0) + 1 * (Sp_vfi>=0))
            c_bc    = (1+r_sav) * S_bc * (S_bc >= 0) + (1+r_debt) * S_bc * (S_bc < 0) + dispinc - Sp_vfi*(1./col_fact_vfi)
            

 ##########################################################################   
 ##################################################################  ##so fa exactly like matlab 
    # Discard local solutions and sort grid
    a_opt = a_opt[(isnan(a_opt)==0)]
    c_opt = c_opt[(isnan(c_opt)==0)]
    
    ord = np.argsort(a_opt)
    a_opt = np.sort(a_opt)
    c_opt = c_opt[ord]
    
    
    # Interpolate in originial grid S
    # (Since the borrowing limit is defined on strict inequality, all a_endo
    # should be included in interpolation)
    
    c_final  = approx_2d(np.append(S_bc, a_opt), np.append(c_bc, c_opt), S)
    
    sp_final = (1+r_sav)*S.T*(S.T>=0) + (1+r_debt)*S.T*(S.T<0) + dispinc - c_final 
    col_fact = (par.col_fact * (sp_final<0) + 1 * (sp_final>=0))
    sp_final = sp_final*col_fact # Back to original grid
    sp_final = sp_final * ((sp_final>=Spgrid[0]) +(sp_final<Spgrid[0]-1e-6)) + Spgrid[0] *(sp_final<Spgrid[0])*(sp_final>=Spgrid[0]-1e-6)
    
    ###if len(innop_prob)>1:
    if size(innop_prob)>1:
        inno3,Sp3       = np.meshgrid(INNO_pos,sp_final)
        Vpaux0          = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)
    else:
        Vpaux0          = splVp(sp_final)
###################################################CAREFUL THAT objfunc is defined differently for c-final >00 or c_final <0!!!!!!!!!!!!!!!!!
    objfunc0 = np.zeros(len(c_final))

    for j in range(0,len(c_final)):
        if c_final[j]>=0:
            objfunc0[j] = c_final[j]**(1-gammac)/(1-gammac)+ beta*Vpaux0[j]
            
        else: 
            objfunc0[j] = (-(10**(5/gammac))**(1-gammac)/(1-gammac)) + beta*Vpaux0[j]
    
   

    
    boundgrid = sum(sp_final>1.025*Spgrid[-1])
    ##############################################c_final still as good as in matlab
    # Check if there are slow decreases in C, i.e., not jumps
    difc = np.append(0, c_final[1::] - c_final[0:-1])
    inddecc = (difc<0)
    inddecc = (inddecc*((np.append(1, inddecc[0:-1])==1) + (np.append(inddecc[1::], 1) == 1))>0)
    #########################################################################so far ok then messed up
    inddecc[0:-2] = np.max([inddecc[0:-2],inddecc[2::]],axis =0)
    inddecc[2::] = np.max([inddecc[2::],  inddecc[0:-2]],axis = 0)
    inddecc[objfunc0<0] = 1
    inddecc[np.append(0, objfunc0[0:-1])<0] = 1 #inddedd 78 but objfunc is 80
        
    grid_prob = np.arange(0,len(c_final))
    do_interp = 1
    for i in (grid_prob[inddecc]):
        if do_interp == 1:
            curv         = 2
            Spgrid_dense_neg = -np.linspace(0,(-Spgrid[0])**(1/curv),300)**curv
            Spgrid_dense = np.append(Spgrid_dense_neg[::-1], np.linspace(1e-6**(1/curv),max(Spgrid[:])**(1/curv),1000)**curv)
            col_fact_dense = (par.col_fact * (Spgrid_dense<0) + 1 * (Spgrid_dense>=0))
            
            
            if size(innop_prob)>1:
                inno3, Sp3  = np.meshgrid(INNO_pos,Spgrid_dense)
                Vpaux_dense = np.dot(splVp(Spgrid_dense,INNO_pos).T,innop_prob)
            else:
                Vpaux_dense          = splVp(Spgrid_dense)
            
            do_interp    = 0
        
        c = (1+r_sav)*S[i]*(S[i]>=0) + (1+r_debt)*S[i]*(S[i]<0)  +dispinc - Spgrid_dense.T*(1./col_fact_dense)
        
        objfunc = np.zeros(len(c))

        for j in range(0,len(c)):
            if c[j]>=0:
                objfunc[j] = c[j]**(1-gammac)/(1-gammac)+ beta*Vpaux_dense[j]
                
            else: 
                objfunc[j] = (-(10**(5/gammac))**(1-gammac)/(1-gammac)) + beta*Vpaux_dense[j]

       
        #objfunc0 = c**(1-gammac)/(1-gammac) * (c>=0) + -(10**(5/gammac))**(1-gammac)/(1-gammac)*(c<0) + beta*Vpaux_dense
    
        amax = np.argmax(objfunc)
        sp_final[i] = Spgrid_dense[amax]
        c_final[i]  = c[amax] #######HERE cfinal[5:6] is wrong!!!!!!!!!!!!! becasue objfunc is  different and so in argmax (objfunc)
        
    # check for bad extrapolation
    c_max   = (1+r_sav)*S.T*(S.T>=0) + (1+r_debt)*S.T*(S.T<0)+dispinc-Spgrid[0]*(1/col_fact_1)
   
    c_final = ( c_final<= c_max) * c_final + (c_final > c_max) * c_max
    
    sp_final = (1+r_sav)*S.T*(S.T>=0) + (1+r_debt)*S.T*(S.T<0) + dispinc - c_final 
    col_fact = (par.col_fact * (sp_final<0) + 1 * (sp_final>=0))
    sp_final = sp_final*col_fact # Back to original grid
    
    if ((np.argmin(sp_final)-Spgrid[0])/np.abs(Spgrid[0]) > 0.01) and (np.argmin(sp_final)-Spgrid[0]<0):
        print('EGM error: savings are below debt limit - wrong extrapolation, min(s''): {min(sp_final)} \n',100*(np.argmin(sp_final)-Spgrid[0])/np.abs(Spgrid[0]))


    return c_final, sp_final, boundgrid


#np.dot(np.reshape(splVp(Sp3[:,0]),[1,len(Sp3[:,0])]).T,np.reshape(innop_prob,[1,len(innop_prob)]))

In [82]:
# -*- coding: utf-8 -*-
"""
Created on Mon Sep 12 14:10:17 2022

@author: Giorgia
"""
import numpy as np
import scipy.interpolate as spi
from scipy.interpolate import griddata
from scipy.interpolate import RegularGridInterpolator
from scipy.interpolate import LinearNDInterpolator
from scipy.interpolate import interpn
from scipy.interpolate import interp2d

def prob_fertility(par,options,Vp,Cp,j_pos):
    
# keyboard
# Solve household problem at fertility period: choice on number of children
    r_sav     = par.r_sav
    r_debt    = par.r_debt
    beta      = par.beta
    gammac    = par.gammac
    w         = par.w
    Lambda    = par.Lambda
    lambdan   = par.lambdan
    gamman    = par.gamman
    
    S         = par.grids[0][j_pos]
    EDUC      = par.educ
    
    AGE_PROF   = par.inc.age_prof
    INNO_pos   = par.inc.inno_pos
    INNO_posT = np.reshape(INNO_pos,[1,5])

    FE_pos     = par.inc.fe_pos
    
    N          = np.arange(0,4,1)
    
    Sp = np.zeros([len(INNO_pos),len(EDUC),len(N),len(S),len(FE_pos)])
    Sp[:] = np.nan
    C  = np.zeros([len(INNO_pos),len(EDUC),len(N),len(S),len(FE_pos)])
    C[:] = np.nan
    V  = np.zeros([len(INNO_pos),len(EDUC),len(N),len(S),len(FE_pos)])
    V[:] = np.nan
    Ck = np.zeros([len(INNO_pos),len(EDUC),len(N),len(S),len(FE_pos)])
    Ck[:] = np.nan
    Tp = np.zeros([len(INNO_pos),len(EDUC),len(N),len(S),len(FE_pos)])
    Tp[:] = np.nan
    boundgrid = np.zeros([len(INNO_pos),len(N),len(FE_pos),len(EDUC)])
    
    
    ## Without children and phi = 1
    if options.Fertility == 'Endo':
    # Only need to solve case without children in benchmark case where we allow them to choose endogenous fertility
            i_n   = 0
            
            for educ in range (0,len(EDUC)):
                
                S          = par.grids[educ][j_pos]
                Spgrid  = par.grids[educ][j_pos+1]
                
                FE        = par.inc.fe[educ]
                INNO      = par.inc.z_val[educ][j_pos,:]
                
                inno3 = np.repeat(INNO_posT,len(S),axis=0)
                INNOp_prob = par.inc.z_prob[educ][:,j_pos+1,:].T # Note I need j_pos + 1 here!
                
                r         = (r_sav * (Spgrid>=0) + r_debt * (Spgrid<0)).T
                rs        = (r_sav * (S>=0) + r_debt * (S<0))
                
                INNO2,S2  = np.meshgrid(INNO_pos,Spgrid)
                
                
                for ife in range (0,len(FE_pos)):
                    splVp = GlobalSpline2D(Spgrid, INNO_pos,Vp[:,educ,i_n,:,ife] )
                    for inno  in range (0,len(INNO_pos)):
                    
                        innop_prob = INNOp_prob[inno,:]
                        
                        h   = exp(AGE_PROF[educ][0][j_pos]) * exp(FE[ife])* exp(INNO[inno])
                        
                        Cpp = Cp[:,educ,i_n,:,ife].T
                        
                        if min(np.ravel(Cpp.any())<0 and abs(min(np.ravel(Cpp.any()))))>1e-6:
                            Cpp = max(Cpp,0)
                        ucp = beta*np.reshape(((1+r)),[80,])*np.dot(Cpp**(-gammac), np.reshape(innop_prob,[5,]))
                        
                        dispinc = w*h*(1-Lambda) + par.fert_trans[i_n]
                        
                        feasible = (dispinc + (1+rs)*S - Spgrid[0]>0)
                        
                        not_feasible = (dispinc + (1+rs)*S - Spgrid[0]<=0)
                        
                        C[inno,educ,i_n,feasible,ife],Sp[inno,educ,i_n,feasible,ife],boundgrid[inno,i_n,ife,educ] = EGM(par,ucp.T,dispinc,Spgrid,S[feasible])
                        Ck[inno,educ,i_n,feasible,ife] = np.zeros(len(C[inno,educ,i_n,feasible,ife]))
                        
                        Sp3 = np.repeat(np.reshape(Sp[inno,educ,i_n,feasible,ife],[80,1]),len(INNO_pos),axis = 1)
                        
                        vp = np.dot((splVp(Sp3[:,0],inno3[0,:]).T),innop_prob)
                        # for j in range(0,len(vp)):
                        #     if math.isnan(vp[j]):
                        #         vp[j] = -inf
                        # vp = np.sort(vp)
                        # for j in range(0,len(vp)):
                        #     if math.isinf(vp[j]):
                        #         vp[j] = nan

                        
                        V[inno,educ,i_n,feasible,ife] = C[inno,educ,i_n,feasible,ife]**(1-gammac)/(1-gammac) + beta*vp[feasible]
                        
                        C[inno,educ,i_n,not_feasible,ife] = 0;
                        V[inno,educ,i_n,not_feasible,ife] = -(10**(5/gammac))**(1-gammac)/(1-gammac)
                        Sp[inno,educ,i_n,not_feasible,ife]= Spgrid[0]

                        Tp[inno,educ,i_n,:,ife] = np.zeros(len(C[inno,educ,i_n,:,ife]))
            
    
    ## Case with Children
    if options.Fertility == 'Endo': #Only need to solve case without children in benchmark case where we allow them to choose endogenous fertility
            in_1    = 1
    elif options.Fertility == 'Exo':
            in_1    = 0

    
    for educ in range (0,len(EDUC)):
        INNOp_prob = par.inc.z_prob[educ][:,j_pos+1,:].T
        S          = par.grids[educ][j_pos]
        Spgrid  = par.grids[educ][j_pos+1]
    
        r         = (r_sav * (Spgrid>=0) + r_debt * (Spgrid<0)).T
        rs        = (r_sav * (S>=0) + r_debt * (S<0))
        
        inno3 = np.repeat(INNO_posT,len(S),axis=0)
        FE        = par.inc.fe[educ]
        INNO      = par.inc.z_val[educ][j_pos,:]
        INNO2,S2  = np.meshgrid(INNO_pos,Spgrid)

        for i_n in range(in_1,len(N)):
            n             = par.N[i_n]
            for ife in range (0,len(FE_pos)):
                splVp = GlobalSpline2D(S2[:,0], INNO2[0,:],Vp[:,educ,i_n,:,ife])
                for inno in range (0,len(INNO_pos)):
                    innop_prob      = INNOp_prob[inno,:]
                    
                    h   = exp(AGE_PROF[educ][0][j_pos]) * exp(FE[ife])* exp(INNO[inno])
                    
                    Cpp = Cp[:,educ,i_n,:,ife].T
                    if min(np.ravel(Cpp.any())<0 and abs(min(np.ravel(Cpp.any()))))>1e-6:
                        Cpp = max(Cpp,0)
                    ucp = beta*np.reshape(((1+r)),[80,1])*np.dot(Cpp**(-gammac), np.reshape(innop_prob,[5,1]))
                    
                    labor_inc = w*h*(1-Lambda)
                    
                    n_final = n* (par.fam_size/2)

                    ChildCost_0 = ChildCost(par,labor_inc,n_final,0,options)
                    ChildCost_1 = ChildCost(par,labor_inc,n_final,1,options)
                    ChildCost_opt = min(ChildCost_0,ChildCost_1)
                    Tp[inno,educ,i_n,:,ife]   = (ChildCost_1>ChildCost_0)
                    
                    dispinc = labor_inc- ChildCost_opt
                    feasible = (dispinc + (1+rs)*S - Spgrid[0]>0)
                    not_feasible = (dispinc + (1+rs)*S - Spgrid[0]<=0)
                    
                    if options.Ck == 'Yes':
                        
                        C[inno,educ,i_n,feasible,ife],Sp[inno,educ,i_n,feasible,ife],boundgrid[inno,i_n,ife,educ] = GEGM_withchild(par,ucp.T,dispinc,Spgrid,S[feasible],splVp,innop_prob,n_final) 

                        f_n     = (lambdan/n_final**(1-gamman))**(1/gammac)
                        Ck[inno,educ,i_n,feasible,ife]     = f_n * C[inno,educ,i_n,feasible,ife]
                        
                        Sp3 = np.repeat(np.reshape(Sp[inno,educ,i_n,:,ife],[len(feasible),1]),len(INNO_pos),axis = 1)                    
                        vp = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)
                        
                        #vp = np.dot((splVp(Sp3[:,0],inno3[0,:]).T),innop_prob)
                        # for j in range(0,len(vp)):
                        #     if math.isnan(vp[j]):
                        #         vp[j] = -inf
                        # vp = np.sort(vp)
                        # for j in range(0,len(vp)):
                        #     if math.isinf(vp[j]):
                        #         vp[j] = nan
                                
                                
                        V[inno,educ,i_n,feasible,ife] = (C[inno,educ,i_n,feasible,ife]**(1-gammac))/(1-gammac)*(C[inno,educ,i_n,feasible,ife]>=0) + lambdan* n_final**(gamman) * (Ck[inno,educ,i_n,feasible,ife]**(1-gammac))/(1-gammac)*(C[inno,educ,i_n,feasible,ife]>=0) -((10**(5/gammac))**(1-gammac))/(1-gammac)*(C[inno,educ,i_n,feasible,ife]<0) + beta*vp[feasible]
                        
                        # Fix bad extrapolation:
                        Sp[inno,educ,i_n,feasible,ife]  = Sp[inno,educ,i_n,feasible,ife]*(C[inno,educ,i_n,feasible,ife] >=0) + Spgrid[0]*(C[inno,educ,i_n,feasible,ife]<0)
                        C[inno,educ,i_n,feasible,ife]  = C[inno,educ,i_n,feasible,ife]*(C[inno,educ,i_n,feasible,ife] >= 0) + 0*(C[inno,educ,i_n,feasible,ife] <0)
                        Ck[inno,educ,i_n,feasible,ife] = Ck[inno,educ,i_n,feasible,ife]*(Ck[inno,educ,i_n,feasible,ife]>= 0) + 0*(Ck[inno,educ,i_n,feasible,ife]<0)
                        
                        C[inno,educ,i_n,not_feasible,ife] = 0
                        Ck[inno,educ,i_n,not_feasible,ife] = 0
                        V[inno,educ,i_n,not_feasible,ife] = -(10**(5/gammac))**(1-gammac)/(1-gammac)
                        Sp[inno,educ,i_n,not_feasible,ife]= Spgrid[0]
                            
                    if options.Ck == 'No':
                        C[inno,educ,i_n,feasible,ife],Sp[inno,educ,i_n,feasible,ife],boundgrid[inno,i_n,ife,educ]= GEGM(par,ucp.T,dispinc,Spgrid,S[feasible],splVp,innop_prob)
                        Ck[inno,educ,i_n,feasible,ife] =np.zeros(len(C[inno,educ,i_n,iphi,feasible,ife]))
                        Ck[:] = np.nan   
                        Sp3 = np.repeat(np.reshape(Sp[inno,educ,i_n,feasible,ife],[80,1]),len(INNO_pos),axis = 1)
                        
                        vp = np.dot((splVp(Sp3[:,0],inno3[0,:]).T),innop_prob)
                        # for j in range(0,len(vp)):
                        #     if math.isnan(vp[j]):
                        #         vp[j] = -inf
                        # vp = np.sort(vp)
                        # for j in range(0,len(vp)):
                        #     if math.isinf(vp[j]):
                        #         vp[j] = nan

                        
                        V[inno,educ,i_n,feasible,ife] = C[inno,educ,i_n,feasible,ife]**(1-gammac)/(1-gammac) + beta*vp[feasible]
                        
                        # Fix bad extrapolation:
                        Sp[inno,educ,i_n,feasible,ife]  = Sp[inno,educ,i_n,feasible,ife]*(C[inno,educ,i_n,feasible,ife] >=0) + Spgrid[0]*(C[inno,educ,i_n,feasible,ife]<0)
                        C[inno,educ,i_n,feasible,ife]  = C[inno,educ,i_n,feasible,ife]*(C[inno,educ,i_n,feasible,ife] >= 0) + 0*(C[inno,educ,i_n,feasible,ife] <0)
                        
                        C[inno,educ,i_n,not_feasible,ife] = 0
                        V[inno,educ,i_n,not_feasible,ife] = -(10**(5/gammac))**(1-gammac)/(1-gammac)
                        Sp[inno,educ,i_n,not_feasible,ife]= Spgrid[0]
    
    
    
    ## Search Grid: only with endogenous fertility
    
    V_2    = np.zeros([len(INNO_pos),len(EDUC),len(S),len(FE_pos)])
    V_2[:] = np.nan
    C_2    = np.zeros([len(INNO_pos),len(EDUC),len(S),len(FE_pos)])
    C_2[:] = np.nan
    Ck_2   = np.zeros([len(INNO_pos),len(EDUC),len(S),len(FE_pos)])
    Ck_2[:] = np.nan
    Sp_2   = np.zeros([len(INNO_pos),len(EDUC),len(S),len(FE_pos)])
    Sp_2[:] = np.nan
    Np_2 = np.zeros([len(INNO_pos),len(EDUC),len(S),len(FE_pos)])
    Np_2[:] = np.nan
    Tp_2   = np.zeros([len(INNO_pos),len(EDUC),len(S),len(FE_pos)])
    Tp_2[:] = np.nan
    
    posN = np.zeros([len(INNO_pos),len(EDUC),len(S),len(FE_pos)])
    
    if options.Fertility == 'Endo': #Only need to solve case without children in benchmark case where we allow them to choose endogenous fertility
            for i_s in range (0,len(S)):
                for ife in range (0,len(FE_pos)):    
                    for inno in range (0,len(INNO_pos)):
                        for educ in range (0,len(EDUC)):
                            for i_n in range (0,len(N)):
                                Vaux                           = np.squeeze(V[inno,educ,:,i_s,ife])     
                                # Max wrt N:
                                posauxN                        = np.argmax(Vaux)### otherwise it would be zero and in Matlab it is 1
                                posN[inno,educ,i_s,ife]        = int64(posauxN)
                                V_2[inno,educ,i_s,ife]     = V[inno,educ,int64(posN[inno,educ,i_s,ife]),i_s,ife]################understand!!!!!!!!!!!!!!!!!!!!!!!!!!
                                C_2[inno,educ,i_s,ife]     = C[inno,educ,int64(posN[inno,educ,i_s,ife]),i_s,ife]
                                Ck_2[inno,educ,i_s,ife]    = Ck[inno,educ,int64(posN[inno,educ,i_s,ife]),i_s,ife]
                                Sp_2[inno,educ,i_s,ife]    = Sp[inno,educ,int64(posN[inno,educ,i_s,ife]),i_s,ife]
                                Tp_2[inno,educ,i_s,ife]    = Tp[inno,educ,int64(posN[inno,educ,i_s,ife]),i_s,ife]
                                Np_2[inno,educ,i_s,ife]    = posN[inno,educ,i_s,ife]

    
    elif options.Fertility == 'Exo':
            for i_s in range (0,len(S)):
                for ife in range (0,len(FE_pos)):    
                    for inno in range (0,len(INNO_pos)):
                        for educ in range (0,len(EDUC)):
                            V_2[inno,educ,i_s,ife]  = V[inno,educ,0,i_s,ife]
                            C_2[inno,educ,i_s,ife]  = C[inno,educ,0,i_s,ife]
                            Ck_2[inno,educ,i_s,ife] = Ck[inno,educ,0,i_s,ife]
                            Sp_2[inno,educ,i_s,ife] = Sp[inno,educ,0,i_s,ife]
                            Tp_2[inno,educ,i_s,ife] = Tp[inno,educ,0,i_s,ife]
                            Np_2[inno,educ,i_s,ife] = 0
    
    
    errors = sum(boundgrid[:])/len(C[:])
    if options.timer_on == 'Y':
        if sum(boundgrid[:]) >= len(EDUC)*len(FE_pos)*len(INNO_pos)*len(N):
            print('j : {par.age(j_pos)}, Share of errors (increase grid) ={errors} \n')

    return V_2,C_2,Ck_2,Sp_2,Np_2,Tp_2


In [81]:
import numpy as np
import scipy.interpolate as spi
from scipy.interpolate import griddata
from scipy.interpolate import RegularGridInterpolator
from scipy.interpolate import LinearNDInterpolator
from scipy.interpolate import interpn
from scipy.interpolate import interp2d

def prob_work_young(par,options,Vp,Cp,j_pos):


    # Solve household problem when young
    r_sav     = par.r_sav
    r_debt    = par.r_debt
    beta      = par.beta
    gammac    = par.gammac
    w         = par.w
    Lambda    = par.Lambda
    
    S         = par.grids[0][j_pos]
    EDUC      = par.educ
    
    AGE_PROF   = par.inc.age_prof
    INNO_pos   = par.inc.inno_pos
    INNO_posT = np.reshape(INNO_pos,[1,5])

    FE_pos     = par.inc.fe_pos
    
    Sp = np.zeros([len(INNO_pos),len(EDUC),len(S),len(FE_pos)])
    C  = np.zeros([len(INNO_pos),len(EDUC),len(S),len(FE_pos)])
    V  = np.zeros([len(INNO_pos),len(EDUC),len(S),len(FE_pos)])
    boundgrid = np.zeros([len(INNO_pos),len(FE_pos),len(EDUC)])

    
    for educ in range (0,len(EDUC)):
        
        S          = par.grids[educ][j_pos]
        Spgrid  = par.grids[educ][j_pos+1]
        
        
        FE        = par.inc.fe[educ]
        INNO      = par.inc.z_val[educ][j_pos,:]
        
        inno3 = np.repeat(INNO_posT,len(S),axis=0)
        INNOp_prob = par.inc.z_prob[educ][:,j_pos+1,:].T # Note I need j_pos + 1 here!
        
        r         = (r_sav * (Spgrid>=0) + r_debt * (Spgrid<0)).T
        rs        = (r_sav * (S>=0) + r_debt * (S<0))
        
        INNO2,S2  = np.meshgrid(INNO_pos,Spgrid)
        
        for ife in range (0,len(FE_pos)):
            splVp = GlobalSpline2D(Spgrid, INNO_pos,Vp[:,educ,:,ife] )
            for inno  in range (0,len(INNO_pos)):
            
                innop_prob = INNOp_prob[inno,:]
                
                
                h   = exp(AGE_PROF[educ][0][j_pos]) * exp(FE[ife])* exp(INNO[inno])
                
                Cpp = Cp[:,educ,:,ife].T
                
                if min(np.ravel(Cpp.any())<0 and abs(min(np.ravel(Cpp.any()))))>1e-6:
                    Cpp = max(Cpp,0)
                ucp = beta*np.reshape(((1+r)),[80,1])*np.dot(Cpp**(-gammac), np.reshape(innop_prob,[size(innop_prob),1]))
                
                dispinc = w*h*(1-Lambda)+ par.init_trans[educ,j_pos]
                
                feasible = (dispinc + (1+rs)*S - Spgrid[0]>0)
                
                not_feasible = (dispinc + (1+rs)*S - Spgrid[0]<=0)
        
                
                C[inno,educ,feasible,ife],Sp[inno,educ,feasible,ife],boundgrid[inno,ife,educ] = GEGM(par,ucp.T,dispinc,Spgrid,S[feasible],splVp,innop_prob)

                #             EGM(par,ucp,dispinc,Spgrid,S);
                
                Sp3 = np.repeat(np.reshape(Sp[inno,educ,feasible,ife],[80,1]),len(INNO_pos),axis = 1)
                vp = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)
                
                V[inno,educ,feasible,ife] = C[inno,educ,feasible,ife]**(1-gammac)/(1-gammac) + beta*vp[feasible]

                
                C[inno,educ,not_feasible,ife] = 0;
                V[inno,educ,not_feasible,ife] = -(10**(5/gammac))**(1-gammac)/(1-gammac)
                Sp[inno,educ,not_feasible,ife]= Spgrid[0]

    
    errors = sum(boundgrid[:])/len(C[:])
    if options.timer_on == 'Y':
            if sum(boundgrid[:]) >= len(EDUC)*len(FE_pos)*len(INNO_pos):
                print('j : {par.age(j_pos)}, Share of errors (increase grid) ={errors} \n')

    return V,C,Sp


In [80]:
# -*- coding: utf-8 -*-
"""
Created on Fri Sep 16 13:13:07 2022

@author: Giorgia
"""
import numpy as np
import scipy.interpolate as spi

def prob_educ_hs_drop(par,options,Vp,Cp):
    # Solve household problem when it is HS dropout, for age 12 and 16
    # HS dropout:
    educ      = 0
    PSY    = par.psy_val_hs
    r_sav     = par.r_sav
    r_debt    = par.r_debt
    beta      = par.beta
    gammac    = par.gammac
    w         = par.w
    Lambda    = par.Lambda
    
    V       = {}
        
    for i in range (0, 1):
        a = [0]
        V[i]= [a]*3
   
    C       = {}
    # for i in range(0,3):
    #     V[i] = [[]]
        
    for i in range (0, 1):
        a = [0]
        C[i]= [a]*3
        
    Sp       = {}
    # for i in range(0,3):
    #     V[i] = [[]]
        
    for i in range (0, 1):
        a = [0]
        Sp[i]= [a]*3
    
    FE_pos     = par.inc.fe_pos
    AGE_PROF   = par.inc.age_prof
    INNO_pos   = par.inc.inno_pos
    INNO_posT = np.reshape(INNO_pos,[1,5])

    FE         = par.inc.fe[educ]
    
    Vpaux     = np.squeeze(Vp[:,educ,:,:])
    Cpaux     = np.squeeze(Cp[:,educ,:,:])
    
    for j_pos in range (par.Je2_pos,par.Je1_pos-2,-1):
        INNOp_prob = par.inc.z_prob[educ][:,j_pos+1,:].T
        S          = par.grids[educ][j_pos] # This grids do not change by education (at this age)
        inno3      = np.repeat(INNO_posT,len(S),axis=0)
        
        V2         = np.zeros([len(INNO_pos),len(S),len(FE_pos)])
        C2         = np.zeros([len(INNO_pos),len(S),len(FE_pos)])
        Sp2        = np.zeros([len(INNO_pos),len(S),len(FE_pos)])
        boundgrid  = np.zeros([len(INNO_pos),len(FE_pos)])
        
        Spgrid     = par.grids[educ][j_pos+1]
        r         = (r_sav * (Spgrid>=0) + r_debt * (Spgrid<0)).T
        rs        = (r_sav * (S>=0) + r_debt * (S<0))

        INNO2,S2  = np.meshgrid(INNO_pos,Spgrid) 
        
        INNO      = par.inc.z_val[educ][j_pos,:]
        
        for ife in range(0,len(FE_pos)):
            splVp = GlobalSpline2D(S2[:,0], INNO2[0,:], np.squeeze(Vpaux[:,:,ife])) ####bCAREFUL WITH INPUTS HERE!!!!!
            for inno  in range (0,len(INNO_pos)):
            
                innop_prob = INNOp_prob[inno,:]
                
                h   = exp(AGE_PROF[educ][0][j_pos]) * exp(FE[ife])* exp(INNO[inno])
                
                Cpp           = np.squeeze(Cpaux[:,:,ife]).T
                if min(np.ravel(Cpp.any())<0 and abs(min(np.ravel(Cpp.any()))))>1e-6:
                    Cpp  = max(Cpp,0);

                
                
                #ucp = beta*np.reshape(((1+r)),[len(r),])*np.dot(Cpp**(-gammac), np.reshape(innop_prob,[5,]))
                
                ucp = beta*np.reshape(((1+r)),[len(r),1])*np.dot(Cpp**(-gammac), np.reshape(innop_prob,[size(innop_prob),1]))
                #ucp = np.reshape(ucp,[size(ucp),1])  
                dispinc = w*h*(1-Lambda) + par.init_trans[educ][j_pos]
                
                feasible = (dispinc + (1+rs)*S - Spgrid[0]>0)
                
                not_feasible = (dispinc + (1+rs)*S - Spgrid[0]<=0)
                                
                C2[inno,feasible,ife],Sp2[inno,feasible,ife],boundgrid[inno,ife] = GEGM(par,ucp.T,dispinc,Spgrid,S[feasible],splVp,innop_prob)
                
                C2[inno,not_feasible,ife] = 0
                Sp2[inno,not_feasible,ife] = -Spgrid[0]
                
                Sp3 = np.repeat(np.reshape(Sp2[inno,feasible,ife],[len(Sp2[inno,feasible,ife]),1]),len(INNO_pos),axis = 1)
                vp = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)
                
                V2[inno,feasible,ife]=C2[inno,feasible,ife]**(1-gammac)/(1-gammac) + beta*vp[feasible]
                V2[inno,not_feasible,ife] = -(10**(5/gammac))**(1-gammac)/(1-gammac)

        
        errors = sum(boundgrid[:])/len(C2[:])
        if options.timer_on == 'Y':
                if sum(boundgrid[:]) >= len(FE_pos)*len(INNO_pos):
                    print('j : {par.age(j_pos)}, Share of errors (increase grid) ={errors} \n')

        
        V[0][j_pos - (par.Je1_pos-1)] = V2
        C[0][j_pos - (par.Je1_pos-1)] = C2
        Sp[0][j_pos - (par.Je1_pos-1)] = Sp2
        
        Vpaux = copy.deepcopy(V2)
        Cpaux = copy.deepcopy(C2)

    
    # Expectation about innovation
    V0         = np.zeros([len(S),len(FE_pos)])
    exp_prob   = np.squeeze(par.inc.z_prob[educ][:,j_pos,0])
    for i_s in range(0,len(S)):
        for fe in range(0,len(FE_pos)):
            vp        = np.squeeze(V2[:,i_s,fe])
            V0[i_s,fe] = np.dot(np.reshape(vp,[1,len(vp)]),exp_prob)

    return V0,V,C,Sp 


In [79]:
# -*- coding: utf-8 -*-
"""
Created on Sat Sep 17 12:57:07 2022

@author: Giorgia
"""
import numpy.matlib
def prob_educ_hs_grad(par,options,Vp,Cp):
    
# Solve household problem when it is HS Graduate, for age 12 and 16

# HS graduate:
    educ      = 1
    PSY       = par.psy_val_hs
    r_sav     = par.r_sav
    r_debt    = par.r_debt
    beta      = par.beta
    gammac    = par.gammac
    w         = par.w
    Lambda    = par.Lambda
    
    # for i in range (0, 1):
    #     a = [0]
    #     V[i]= [a]*3
    
    V       = {}  
        
    for i in range (0, 1):
        a = [0]
        V[i]= [a]*3
        V[0][0] = [a]*len(PSY)
########################################CAN'T FIGURE OUT    
    C       = {}  
        
    for i in range (0, 1):
        a = [0]
        C[i]= [a]*3
        C[0][0] = [a]*len(PSY)
        
    # C =  {}
    # for i in range (1, 3):
    #     a = [0]
    #     C[0] = [a] * 100
    #     C[i]= [a]*3
#############################################
        
    Sp       = {} 
        
    for i in range (0, 1):
        a = [0]
        Sp[i]= [a]*3
        Sp[0][0] = [a]*len(PSY)
        
    # for i in range (0, 1):
    #     a = [0]
    #     Sp[i]= [a]*3
        
    FE_pos     = par.inc.fe_pos
    AGE_PROF   = par.inc.age_prof
    INNO_pos   = par.inc.inno_pos
    INNO_posT = np.reshape(INNO_pos,[1,5])
    
    FE         = par.inc.fe[educ]
    
    Vpaux     = np.squeeze(Vp[:,educ,:,:])
    Cpaux     = np.squeeze(Cp[:,educ,:,:])
    
    
    for j_pos in range (par.Je2_pos,par.Je1_pos-1,-1):
        INNOp_prob = par.inc.z_prob[educ][:,j_pos+1,:].T
        S          = par.grids[educ][j_pos] # This grids do not change by education (at this age)
        inno3      = np.repeat(INNO_posT,len(S),axis=0)
        
        V2         = np.zeros([len(INNO_pos),len(S),len(FE_pos)])
        C2         = np.zeros([len(INNO_pos),len(S),len(FE_pos)])
        Sp2        = np.zeros([len(INNO_pos),len(S),len(FE_pos)])
        boundgrid  = np.zeros([len(INNO_pos),len(FE_pos)])
        
        Spgrid     = par.grids[educ][j_pos+1]
        r         = (r_sav * (Spgrid>=0) + r_debt * (Spgrid<0)).T
        rs        = (r_sav * (S>=0) + r_debt * (S<0))

        INNO2,S2  = np.meshgrid(INNO_pos,Spgrid) 
        
        INNO      = par.inc.z_val[educ][j_pos,:]
        
        for ife in range(0,len(FE_pos)):
            #splVp = spi.interp2d(S2, INNO2, np.squeeze(Vpaux[:,:,ife]) )
            splVp = GlobalSpline2D(S2[:,0], INNO2[0,:], np.squeeze(Vpaux[:,:,ife]))
            for inno  in range (0,len(INNO_pos)):
            
                innop_prob = INNOp_prob[inno,:]
                
                h   = exp(AGE_PROF[educ][0][j_pos]) * exp(FE[ife])* exp(INNO[inno])
                
                Cpp           = np.squeeze(Cpaux[:,:,ife]).T
                if min(np.ravel(Cpp.any())<0 and abs(min(np.ravel(Cpp.any()))))>1e-6:
                    Cpp  = max(Cpp,0)

                ucp = beta*np.reshape(((1+r)),[len(r),1])*np.dot(Cpp**(-gammac), np.reshape(innop_prob,[5,1]))
                dispinc = w*h*(1-Lambda) + par.init_trans[educ][j_pos]
                
                feasible = (dispinc + (1+rs)*S - Spgrid[0]>0)
                
                not_feasible = (dispinc + (1+rs)*S - Spgrid[0]<=0)
                                
                C2[inno,feasible,ife],Sp2[inno,feasible,ife],boundgrid[inno,ife] = GEGM(par,ucp.T,dispinc,Spgrid,S[feasible],splVp,innop_prob)
                
                C2[inno,not_feasible,ife] = 0
                Sp2[inno,not_feasible,ife] = -Spgrid[0]
                
                Sp3 = np.repeat(np.reshape(Sp2[inno,feasible,ife],[len(Sp2[inno,feasible,ife]),1]),len(INNO_pos),axis = 1)
                vp = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,innop_prob)
                
                V2[inno,feasible,ife]=C2[inno,feasible,ife]**(1-gammac)/(1-gammac) + beta*vp[feasible]
                V2[inno,not_feasible,ife] = -(10**(5/gammac))**(1-gammac)/(1-gammac)
    
        errors = sum(boundgrid[:])/len(C2[:])
        if options.timer_on == 'Y':
                if sum(boundgrid[:]) >= len(FE_pos)*len(INNO_pos):
                    print('j : {par.age(j_pos)}, Share of errors (increase grid) ={errors} \n')
        
        V[0][j_pos - (par.Je1_pos-1)] = V2
        C[0][j_pos - (par.Je1_pos-1)] = C2
        Sp[0][j_pos - (par.Je1_pos-1)] = Sp2
        
        Vpaux = V2
        Cpaux = C2

   

    ## Age 16 (age of HS)
    # Expectation about innovation
    exp_prob   = np.squeeze(par.inc.z_prob[educ][:,j_pos,0])
    
    j_pos = par.Je1_pos-1
        
    S          = par.grids[educ][j_pos] # This grids do not change by education (at this age)
    inno3      = np.repeat(INNO_posT,len(S),axis=0)
    
    V2         = np.zeros([len(S),len(FE_pos)])
    C2         = np.zeros([len(S),len(FE_pos)])
    Sp2        = np.zeros([len(S),len(FE_pos)])
    boundgrid  = np.zeros([len(FE_pos)])
    
    Spgrid     = par.grids[educ][j_pos+1]
    r         = (r_sav * (Spgrid>=0) + r_debt * (Spgrid<0)).T
    rs        = (r_sav * (S>=0) + r_debt * (S<0))

    INNO2,S2  = np.meshgrid(INNO_pos,Spgrid) 
    
    INNO      = par.inc.z_val[educ][j_pos,:]
    
    
    for ife in range(0,len(FE_pos)):
        splVp = GlobalSpline2D(S2[:,0], INNO2[0,:], np.squeeze(Vpaux[:,:,ife]))
            
        h   = exp(AGE_PROF[educ][0][j_pos]) * exp(FE[ife])* exp(INNO[inno])
        
        Cpp           = np.squeeze(Cpaux[:,:,ife]).T
        if min(np.ravel(Cpp.any())<0 and abs(min(np.ravel(Cpp.any()))))>1e-6:
            Cpp  = max(Cpp,0)
        
        ucp = beta*np.reshape(((1+r)),[len(r),1])*np.dot(Cpp**(-gammac), np.reshape(exp_prob,[5,1]))
        dispinc = -par.pe1 + par.init_trans[educ][j_pos]
        
        feasible = (dispinc + (1+rs)*S - Spgrid[0]>0)
        
        not_feasible = (dispinc + (1+rs)*S - Spgrid[0]<=0)
        
        C2[feasible,ife],Sp2[feasible,ife],boundgrid[ife] = GEGM(par,ucp.T,dispinc,Spgrid,S[feasible],splVp,exp_prob)
        
        C2[not_feasible,ife] = 0
        Sp2[not_feasible,ife] = 0
        
        Sp3 = np.repeat(np.reshape(Sp2[:,ife],[len(Sp2[:,ife]),1]),len(INNO_pos),axis = 1)
        vp = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,exp_prob)
        
        V2[feasible,ife]=C2[feasible,ife]**(1-gammac)/(1-gammac) + beta*vp[feasible]
        V2[not_feasible,ife] = -(10**(5/gammac))**(1-gammac)/(1-gammac)
    
    if j_pos >= par.Je1_pos: #only case j_pos = 10,9
        V[0][j_pos - (par.Je1_pos-1)] = V2
        C[0][j_pos - (par.Je1_pos-1)] = C2
        Sp[0][j_pos - (par.Je1_pos-1)] = Sp2
    else: #case j_pos = 8
                                    
        for i in range(0, len(PSY) ):    
            V[0][j_pos - (par.Je1_pos-1)][i] = V2 - np.reshape(np.repeat(np.repeat(PSY[i],size(V2,0)),size(V2,1)),[size(V2,0),size(V2,1)])
            C[0][j_pos - (par.Je1_pos-1)][i] =  C2
            Sp[0][j_pos - (par.Je1_pos-1)][i] =  Sp2

    
    Vpaux = V2
    Cpaux = C2
    
    return V,C, Sp

In [78]:
# -*- coding: utf-8 -*-
"""
Created on Sat Sep 17 16:53:16 2022

@author: Giorgia
"""
import types, copy
def prob_educ_co_grad(par,options,Vp,Cp):
    # Solve household problem when it is College Graduate, for age 12 and 16
    # College graduate:
    educ      = 2
    PSY       = par.psy_val_col
    r_sav     = par.r_sav
    beta      = par.beta
    gammac    = par.gammac
    Lambda    = par.Lambda
    
    V       = {}  
        
    for i in range (0, 1):
        a = [0]
        V[i]= [a]*3
        V[0][0] = [a]*len(PSY)
    
    C       = {}  
        
    for i in range (0, 1):
        a = [0]
        C[i]= [a]*3
        C[0][0] = [a]*len(PSY)
        
    # C =  {}
    # for i in range (1, 3):
    #     a = [0]
    #     C[0] = [a] * 100
    #     C[i]= [a]*3
#############################################
        
    Sp       = {} 
        
    for i in range (0, 1):
        a = [0]
        Sp[i]= [a]*3
        Sp[0][0] = [a]*len(PSY)
        
        

    FE_pos     = par.inc.fe_pos
    AGE_PROF   = par.inc.age_prof
    INNO_pos   = par.inc.inno_pos
    INNO_posT = np.reshape(INNO_pos,[1,5])

    FE         = par.inc.fe[educ]
    
    Vpaux     = np.squeeze(Vp[:,educ,:,:])
    Cpaux     = np.squeeze(Cp[:,educ,:,:])
    
    
    # Expectation about innovation
    j_pos      = par.Je2_pos+1
    #exp_prob   = np.reshape(np.squeeze(par.inc.z_prob[educ][:,j_pos,0]),[1,len(np.squeeze(par.inc.z_prob[educ][:,j_pos,0]))]) #############here we determine id xp_prob is empy or not which will determine Vaux in GEGM_college
    exp_prob   = np.squeeze(par.inc.z_prob[educ][:,j_pos,0])
    ## Age 14-20
    for j_pos in range (par.Je2_pos,par.Je1_pos-2,-1):
        INNOp_prob = par.inc.z_prob[educ][:,j_pos+1,:].T
        S          = par.grids[educ][j_pos] # This grids do not change by education (at this age)
        Spgrid     = par.grids[educ][j_pos+1]
        V2         = np.zeros([len(S),len(FE_pos)])
        C2         = np.zeros([len(S),len(FE_pos)])
        Sp2        = np.zeros([len(S),len(FE_pos)])
        boundgrid  = np.zeros([len(FE_pos)])
        
    
        if j_pos   == par.Je2_pos:
            r_debt_today        = 0
            r_debt_tomorrow     = copy.deepcopy(par.r_debt)
            col_fact_1          = (par.col_fact * (Spgrid[0]<0) + 1 * (Spgrid[0]>=0))
            par_temp            = copy.deepcopy(par)
            par_temp.r_debt     = 0
            FE                  = par.inc.fe[1] #Skills before getting educated
        elif j_pos   == par.Je2_pos-1:
            r_debt_today        = copy.deepcopy(par.r_debt)
            r_debt_tomorrow     = 0
            col_fact_1          = 1
            par_temp            = copy.deepcopy(par)
            par_temp.col_fact   = 1
            FE                  = par.inc.fe[1] #Skills before getting educated
        else:
            r_debt_today        = copy.deepcopy(par.r_debt)
            r_debt_tomorrow     = copy.deepcopy(par.r_debt)
            FE                  = par.inc.fe[0] #Skills before getting educated
        
        # r = np.zeros(len(Spgrid))
        # for i in range(0,len(Spgrid)):
        #     if Spgrid [i]>= 0:
        #         r[i] = r_sav
        #     else:
        #         r[i] = r_debt_tomorrow
                
        r         = (r_sav * (Spgrid>=0) + r_debt_tomorrow * (Spgrid<0)).T #( should be trnasposed but it is not) currently working but keep in  mind!!!
        rs        = (r_sav * (S>=0) + r_debt_today * (S<0))
        
        if j_pos > par.Je2_pos-1:
            inno3      = np.repeat(INNO_posT,len(S),axis=0)
            INNO2,S2  = np.meshgrid(INNO_pos,Spgrid) 

        
        for ife in range (0,len(FE_pos)):
            if j_pos > par.Je2_pos-1:##########################################################I used >par.Je2_pos-1 instead of > as in Matlab because all j_pos are shifted down to 1
                splVp = GlobalSpline2D(Spgrid, INNO_pos,np.squeeze(Vpaux[:,:,ife]) )
                Cpp           = np.squeeze(Cpaux[:,:,ife]).T  ###################################################################################### 
                if min(np.ravel(Cpp.any())<0 and abs(min(np.ravel(Cpp.any()))))>1e-6:
                    Cpp  = max(Cpp,0)
                        
                ucp = beta*np.reshape(((1+r)),[len(r),1])*np.dot(Cpp**(-gammac), np.reshape(exp_prob,[size(exp_prob),1]))
                
            else:
                splVp   = spi.interp1d(Spgrid,np.squeeze(Vpaux[:,ife]), fill_value="extrapolate")
                Cpp           = np.squeeze(Cpaux[:,ife]) ###################################################################################### 
                
                if min(np.ravel(Cpp.any())<0 and abs(min(np.ravel(Cpp.any()))))>1e-6:
                    Cpp  = max(Cpp,0)
                        
                ucp = beta*np.reshape(((1+r)),[len(r),])*Cpp**(-gammac)*exp_prob
                ucp = np.reshape(ucp,[size(ucp),1])                
            
            h   = exp(AGE_PROF[educ][0][j_pos]) * exp(FE[ife])
            
            # if ndim(Cpaux) == 3:
            #     Cpp           = np.squeeze(Cpaux[:,:,ife]).T
               
            # else:
            #     Cpp           = Cpaux[:,ife].T
           
            ############################################
            if j_pos >= par.Je2_pos-1:
                
                dispinc = par.w_college*h*(1-Lambda) - par.pe2 + par.init_trans[educ][j_pos]
            else:
                
                dispinc = -par.pe1 + par.init_trans[educ][j_pos]
            
            feasible = (dispinc + (1+rs)*S - Spgrid[0]*(1/col_fact_1)>0)
            
            not_feasible = (dispinc + (1+rs)*S - Spgrid[0]*(1/col_fact_1)<=0)
            
            
            if j_pos >= par.Je2_pos-1:###################also here I made adjustements which I need to this of as it is > instead of >= despite being par.Je2_pos-1 instead of par.Je2_pos
                C2[feasible,ife],Sp2[feasible,ife],boundgrid[ife] = GEGM_college(par_temp,ucp.T,dispinc,Spgrid,S[feasible],splVp,exp_prob)
                C2[not_feasible,ife]  = 0
                Sp2[not_feasible,ife] = Spgrid[0]
            else:
                C2[feasible,ife],Sp2[feasible,ife],boundgrid[ife] = GEGM(par,ucp.T,dispinc,Spgrid,S[feasible],splVp,exp_prob)
                C2[not_feasible,ife]  = 0
                Sp2[not_feasible,ife] = Spgrid[0]

            
            if j_pos > par.Je2_pos-1:
                Sp3 = np.repeat(np.reshape(Sp2[:,ife],[len(Sp2[:,ife]),1]),len(INNO_pos),axis = 1)
                vp = np.dot(splVp(Sp3[:,0],inno3[0,:]).T,exp_prob)
            else:
                vp          = splVp(Sp2[:,ife])

            
            V2[feasible,ife]=C2[feasible,ife]**(1-gammac)/(1-gammac) + beta*vp[feasible]
            V2[not_feasible,ife] = -(10**(5/gammac))**(1-gammac)/(1-gammac)

        
    
        if j_pos >= par.Je1_pos: # case j_pos = 10,9 I switch to >= and leave par.Je1_pos becasue we want to get '0'
            V[0][j_pos - (par.Je1_pos-1)] = V2
            C[0][j_pos - (par.Je1_pos-1)] = C2
            Sp[0][j_pos - (par.Je1_pos-1)] = Sp2
        else: #case j_pos = 8
                                         
            for i in range(0, len(PSY) ):    
                V[0][j_pos - (par.Je1_pos-1)][i] = V2 - np.reshape(np.repeat(np.repeat(PSY[i],size(V2,0)),size(V2,1)),[size(V2,0),size(V2,1)])
                C[0][j_pos - (par.Je1_pos-1)][i] =  C2
                Sp[0][j_pos - (par.Je1_pos-1)][i] =  Sp2
         
            
        Vpaux = V2
        Cpaux = C2
        exp_prob = 1

    return V,C,Sp

In [77]:
# -*- coding: utf-8 -*-
"""
Created on Fri Sep 16 12:47:17 2022

@author: Giorgia
"""
import numpy as np 

def prob_educ(par,options,Vp,Cp):
# Solve education choice

    # Age 16 (age of HS) 
    # Ve        = {}
    # for i in range(0,3):
    #     Ve[i] = [[],[],[]]
 ##########or   
    Ve =  {}
    for i in range (0, 3):
        a = [0]
        Ve[i]= [a]*3
        
    Ce =  {}
    for i in range (0, 3):
        a = [0]
        Ce[i]= [a]*3
        
    Se =  {}
    for i in range (0, 3):
        a = [0]
        Se[i]= [a]*3
    
            
    j_pos     = par.Je1_pos-1
    S         = par.grids[0][j_pos]
    FE_pos     = par.inc.fe_pos
    PSY       = par.psy_val_hs
    
    V0 = np.zeros([len(PSY),len(S),len(FE_pos)])
           
    tau = np.zeros([len(PSY),len(S),len(FE_pos)]) 
    
    ## Case 1: HS Dropout
    educ = 0
    Vhsd,Veaux,Ceaux,Seaux = prob_educ_hs_drop(par,options,Vp,Cp) ##ok<*ASGLU>
    
    for jp in range (0,3):
        Ve[educ][jp] = Veaux[0][jp]
        Ce[educ][jp] = Ceaux[0][jp]
        Se[educ][jp] = Seaux[0][jp]

    ## Case 2: HS Graduate
    educ = 1
    Veaux,Ceaux,Seaux = prob_educ_hs_grad(par,options,Vp,Cp)
    for jp in range (0,3):
        Ve[educ][jp] = Veaux[0][jp]
        Ce[educ][jp] = Ceaux[0][jp]
        Se[educ][jp] = Seaux[0][jp]
    
    ## Case 3: College Graduate
    educ = 2
    
    Veaux,Ceaux,Seaux = prob_educ_co_grad(par,options,Vp,Cp)
    for jp in range (0,3):
        Ve[educ][jp] = Veaux[0][jp]
        Ce[educ][jp] = Ceaux[0][jp]
        Se[educ][jp] = Seaux[0][jp]
    
    ## Optimal education:
    for ipsy in range (0,len(PSY)):
        for i_s in range (0,len(S)):
            for ife in range (0,len(FE_pos)):
                oo = Ce[1][0][99][i_s,ife]
                v_hsd = Vhsd[i_s][ife]
                v_hsg = Ve[1][0][ipsy][i_s,ife]
                v_cg  = Ve[2][0][ipsy][i_s,ife]
                
                v_aux = [v_hsd, v_hsg, v_cg]
                V0[ipsy,i_s,ife] = np.max(v_aux)
                tau[ipsy,i_s,ife] = np.argmax(v_aux)

    
    
    return Ve,Ce,Se,V0,tau

In [112]:
# -*- coding: utf-8 -*-
"""
Created on Wed Aug 10 13:35:40 2022

@author: Giorgia
"""

from types import SimpleNamespace
import time
import itertools as it
import numpy as np
from scipy import optimize
from scipy.interpolate import RegularGridInterpolator
from scipy.io import loadmat
import os
import types, copy


os.chdir('/Users/Giorgia/Dropbox/ReplicationProject/DairuchPaper/DairuchFiles/run_generic')

guess_ = loadmat('/Users/Giorgia/Dropbox/ReplicationProject/DairuchPaper/DairuchFiles/run_generic/guess.mat')

##type(guess['guess']),guess['guess'].shape
##type(annots['guess'][0][0]),annots['guess'][0][0].shape
guess.tau = guess_['guess'][0][0]['tau']
guess.Vc0 = guess_['guess'][0][0]['Vc0']

#guess.keys()

def tic():
    #Homemade version of matlab tic and toc functions
    import time
    global startTime_for_tictoc
    startTime_for_tictoc = time.time()

def toc():
    import time
    if 'startTime_for_tictoc' in globals():
        print("Elapsed time is " + str(time.time() - startTime_for_tictoc) + " seconds.")
    else:
        print("Toc: start time not set")
tic()
toc()

options.start_Calibrate = 'N'

options.timer_on = 'Y'

def solve_model(par,options):
##Solve model - April 2015
  
    r_sav = par.r_sav
    w = par.w
# Notes:
# Time length: 2 years

    if options.start_Calibrate == 'N':
	# Load initial guess for tau and Vc0
        filename = str('guess')
        file = str(filename + '.mat')
        file_path = r'/Users/Giorgia/Dropbox/ReplicationProject/DairuchPaper/DairuchFiles/run_generic/'+file
        exist = os.path.isfile(file_path)

        if exist:
            guess_ = loadmat(file)
            #guess.keys()
            #tau = guess['guess'][0][0]['tau']
            #Vc0 = guess['guess'][0][0]['Vc0']
            #Vc0 = np.reshape(Vc0,[10,20,20])###################wrong!!!!!
            Vc0 = guess.Vc0

        else:
            Vc0   = np.repeat(np.zeros([1,1]),1)

    elif options.start_Calibrate == 'Y':
		## Load initial guess for tau and Vc0 from HPC
        Vc0   = options.guess.Vc0


# If guess is of different size of current grid start new guess
    Vc0_aux  = np.zeros([len(par.psy_val_hs),len(par.PHI),par.N_fe])
    if Vc0.ndim != Vc0_aux.ndim:
        Vc0  = np.zeros([len(par.psy_val_hs),len(par.PHI),par.N_fe])
    elif sum(len(Vc0[:,0,0]) - len(Vc0_aux[:,0,0]) + len(Vc0[0,:,0]) - len(Vc0_aux[0,:,0]) + len(Vc0[0,0,:]) - len(Vc0_aux[0,0,:])) != 0:########### Vc0 is (20,20,10), Vc0_aux is (100,5,20). I want to get -75=  
        Vc0  = np.zeros([len(par.psy_val_hs),len(par.PHI),par.N_fe])
############################################################################

## Empty matrix
    V   = {}
    for i in range (0, par.Jd_pos):
        V[i] = [[]]
    
    C   = {}
    for i in range (0, par.Jd_pos):
        C[i] = [[]]
    Ck   = {}
    for i in range (0, par.Jd_pos):
        Ck[i] = [[]]
        
    S   = {}
    for i in range (0, par.Jd_pos):
        S[i] = [[]]
        
    Tp   = {}
    for i in range (0, par.Jd_pos):
        Tp[i] = [[]]
    
    ## Dead: V_Jd = 0
    V[par.Jd_pos-1] = np.zeros([len(par.educ),int(par.Ls[-1]),par.N_fe])
    C[par.Jd_pos-1] = np.zeros([len(par.educ),int(par.Ls[-1]),par.N_fe])
    S[par.Jd_pos-1] = np.zeros([len(par.educ),int(par.Ls[-1]),par.N_fe])

    ## Retirement: for ages j=Jr:Jd-1
    for j in range(par.Jd_pos-2,par.Jr_pos-2,-1):
        V[j],C[j],S[j] = prob_ret(par,options,V[j+1],C[j+1],j)
        if options.timer_on == 'Y':
                print(f'j: {par.age[j]}, time: {toc()} sec\n')
    
    ## Working: for age j=Jr-1: 
    j = par.Jr_pos-2
    V[j],C[j],S[j]  = prob_workT(par,options,V[j+1],C[j+1],j);
    if options.timer_on == 'Y':
            print(f'j: {par.age[j]}, time: {toc()} sec\n')

   ## Working old: for ages j=Jc+Je1 : Jr -2
    for j in range (par.Jr_pos-3,(par.Jc_pos + par.Je1_pos-3),-1):
        V[j],C[j],S[j] = prob_work_old(par,options,V[j+1],C[j+1],j)
        if options.timer_on == 'Y':
                print(f'j: {par.age[j]}, time: {toc()} sec\n')
   
    ## Iterate over years of educ and Vchild0
    iter_ = 0
    distV = 1
    while(distV > options.tolV and iter_ <= options.maxiter):
        ticiter = tic
        ## Transfer money to children, age j = Jc+Je1
        j = par.Jc_pos+par.Je1_pos-3
        V[j],C[j],Ck[j],S[j],PHIp,Tp[j] = prob_work_trans(par,options,V[j+1],C[j+1],Vc0,j)
        if options.timer_on == 'Y':
            print(f'j: {par.age[j]}, time: {toc()} sec\n')  
            
    ## Working with Children at home: for ages j=Jc+1:Jc+Je1-3: age  = 32
        for j in range(par.Jc_pos+par.Je1_pos-4,par.Jc_pos-1,-1):
            V[j],C[j],Ck[j],S[j],Tp[j] = prob_work_with_child(par,options,V[j+1],C[j+1],j)
            if options.timer_on == 'Y':
                print(f'j: {par.age[j]}, time: {toc()} sec\n')  
##################################################################
##############################################################THINGS ARE LOOKING GOOD BUT FOR VERY NEGATIVE NUMBERS IN FIRST ROWS BOTH C AND V ARE WORNG. THE APROXIMATION MESSES THINGS UP. gegm USES splVp AS INPUT SO ALSO C AND S (OUTPUT OF GEGM) ARE WORNG
    ## Fertility: for age j=Jc
        j=par.Jc_pos-1
        V[j],C[j],Ck[j],S[j],Np,Tp[j] = prob_fertility(par,options,V[j+1],C[j+1],j)
        if options.timer_on == 'Y':
            print(f'j: {par.age[j]}, time: {toc()} sec\n')  
        
        ## Work young: for ages j=Je2+1:Jc-1
        for j in range(par.Jc_pos-2,par.Je2_pos,-1):
            V[j],C[j],S[j] = prob_work_young(par,options,V[j+1],C[j+1],j)
            if options.timer_on == 'Y':
                print(f'j: {par.age[j]}, time: {toc()} sec\n')
        
        ## Solve Education choice
        j = par.Je2_pos
        Ve,Ce,Se,Vc1,tau1 = prob_educ(par,options,V[j+1],C[j+1])
        if options.timer_on == 'Y':
            print(f'j: {par.age[j]}, time: {toc()} sec\n, iter:{iter_}, distV: {distV}')
        
        
        difV  = np.abs(Vc1-Vc0)
        distV = np.max(difV[:])
        tau0  = tau1
    
    #    pace    = 0.25;
        pace    = 0.9
        Vc0 = pace*Vc1+(1-pace)*Vc0
        
    
        iter_ = iter_+1
        #if options.model_iter_on == 'Y':
           # fprintf('HH iter %03i, norm Vchild : %9.4f, tfs = %03.0f sec\n', iter, distV,toc(ticiter));

        
    #     fprintf(' \n')

    
    if (distV>options.tolV) == 1:
        flags.converge = 'N'
    else:
        flags.converge = 'Y'

    
    # keyboard
    model.pol = SimpleNamespace()
    pol = model.pol
    ## Output
    # Policy functions:
    pol.C         = copy.deepcopy(C)
    pol.Ce        = copy.deepcopy(Ce)
    pol.Ck        = copy.deepcopy(Ck)
    pol.S         = copy.deepcopy(S)
    pol.Se     	  = copy.deepcopy(Se)
    pol.V         = copy.deepcopy(V)
    pol.Ve        = copy.deepcopy(Ve)
    pol.Vc0       = copy.deepcopy(Vc0)
    pol.Np        = copy.deepcopy(Np)
    pol.PHIp      = copy.deepcopy(PHIp)
    pol.Tp        = copy.deepcopy(Tp)
    pol.tau0      = copy.deepcopy(tau1)
    pol.tau       = copy.deepcopy(tau1)
    
    # Initial guess
    guess.Vc0    = Vc0
    
    # flags
    flags.iter_ = iter_
    
    return pol, par, guess, flags

            



Elapsed time is 0.0 seconds.


In [113]:
"""Run solve_model """
from types import SimpleNamespace
import time
import itertools as it
import numpy as np
from scipy import optimize
from scipy.interpolate import RegularGridInterpolator
from scipy.io import loadmat
import os
import types, copy


os.chdir('/Users/Giorgia/Dropbox/ReplicationProject/DairuchPaper/DairuchFiles/run_generic')

guess_ = loadmat('/Users/Giorgia/Dropbox/ReplicationProject/DairuchPaper/DairuchFiles/run_generic/guess.mat')

##type(guess['guess']),guess['guess'].shape
##type(annots['guess'][0][0]),annots['guess'][0][0].shape
guess.tau = guess_['guess'][0][0]['tau']
guess.Vc0 = guess_['guess'][0][0]['Vc0']

#guess.keys()

def tic():
    #Homemade version of matlab tic and toc functions
    import time
    global startTime_for_tictoc
    startTime_for_tictoc = time.time()

def toc():
    import time
    if 'startTime_for_tictoc' in globals():
        print("Elapsed time is " + str(time.time() - startTime_for_tictoc) + " seconds.")
    else:
        print("Toc: start time not set")
tic()
toc()

options.start_Calibrate = 'N'

options.timer_on = 'Y'

r_sav = par.r_sav
w = par.w
# Notes:
# Time length: 2 years

if options.start_Calibrate == 'N':
# Load initial guess for tau and Vc0
    filename = str('guess')
    file = str(filename + '.mat')
    file_path = r'/Users/Giorgia/Dropbox/ReplicationProject/DairuchPaper/DairuchFiles/run_generic/'+file
    exist = os.path.isfile(file_path)

    if exist:
        guess_ = loadmat(file)
        #guess.keys()
        #tau = guess['guess'][0][0]['tau']
        #Vc0 = guess['guess'][0][0]['Vc0']
        #Vc0 = np.reshape(Vc0,[10,20,20])###################wrong!!!!!
        Vc0 = guess.Vc0

    else:
        Vc0   = np.repeat(np.zeros([1,1]),1)

elif options.start_Calibrate == 'Y':
    ## Load initial guess for tau and Vc0 from HPC
    Vc0   = options.guess.Vc0


# If guess is of different size of current grid start new guess
Vc0_aux  = np.zeros([len(par.psy_val_hs),len(par.PHI),par.N_fe])
if Vc0.ndim != Vc0_aux.ndim:
    Vc0  = np.zeros([len(par.psy_val_hs),len(par.PHI),par.N_fe])
elif sum(len(Vc0[:,0,0]) - len(Vc0_aux[:,0,0]) + len(Vc0[0,:,0]) - len(Vc0_aux[0,:,0]) + len(Vc0[0,0,:]) - len(Vc0_aux[0,0,:])) != 0:########### Vc0 is (20,20,10), Vc0_aux is (100,5,20). I want to get -75=  
    Vc0  = np.zeros([len(par.psy_val_hs),len(par.PHI),par.N_fe])
############################################################################

## Empty matrix
V   = {}
for i in range (0, par.Jd_pos):
    V[i] = [[]]

C   = {}
for i in range (0, par.Jd_pos):
    C[i] = [[]]
Ck   = {}
for i in range (0, par.Jd_pos):
    Ck[i] = [[]]

S   = {}
for i in range (0, par.Jd_pos):
    S[i] = [[]]

Tp   = {}
for i in range (0, par.Jd_pos):
    Tp[i] = [[]]

## Dead: V_Jd = 0
V[par.Jd_pos-1] = np.zeros([len(par.educ),int(par.Ls[-1]),par.N_fe])
C[par.Jd_pos-1] = np.zeros([len(par.educ),int(par.Ls[-1]),par.N_fe])
S[par.Jd_pos-1] = np.zeros([len(par.educ),int(par.Ls[-1]),par.N_fe])

## Retirement: for ages j=Jr:Jd-1
for j in range(par.Jd_pos-2,par.Jr_pos-2,-1):
    V[j],C[j],S[j] = prob_ret(par,options,V[j+1],C[j+1],j)
    if options.timer_on == 'Y':
            print(f'j: {par.age[j]}, time: {toc()} sec\n')

## Working: for age j=Jr-1: 
j = par.Jr_pos-2
V[j],C[j],S[j]  = prob_workT(par,options,V[j+1],C[j+1],j);
if options.timer_on == 'Y':
        print(f'j: {par.age[j]}, time: {toc()} sec\n')

## Working old: for ages j=Jc+Je1 : Jr -2
for j in range (par.Jr_pos-3,(par.Jc_pos + par.Je1_pos-3),-1):
    V[j],C[j],S[j] = prob_work_old(par,options,V[j+1],C[j+1],j)
    if options.timer_on == 'Y':
            print(f'j: {par.age[j]}, time: {toc()} sec\n')

## Iterate over years of educ and Vchild0
iter_ = 0
distV = 1
while(distV > options.tolV and iter_ <= options.maxiter):
    ticiter = tic
    ## Transfer money to children, age j = Jc+Je1
    j = par.Jc_pos+par.Je1_pos-3
    V[j],C[j],Ck[j],S[j],PHIp,Tp[j] = prob_work_trans(par,options,V[j+1],C[j+1],Vc0,j)
    if options.timer_on == 'Y':
        print(f'j: {par.age[j]}, time: {toc()} sec\n')  

## Working with Children at home: for ages j=Jc+1:Jc+Je1-3: age  = 32
    for j in range(par.Jc_pos+par.Je1_pos-4,par.Jc_pos-1,-1):
        V[j],C[j],Ck[j],S[j],Tp[j] = prob_work_with_child(par,options,V[j+1],C[j+1],j)
        if options.timer_on == 'Y':
            print(f'j: {par.age[j]}, time: {toc()} sec\n')  
##################################################################
##############################################################THINGS ARE LOOKING GOOD BUT FOR VERY NEGATIVE NUMBERS IN FIRST ROWS BOTH C AND V ARE WORNG. THE APROXIMATION MESSES THINGS UP. gegm USES splVp AS INPUT SO ALSO C AND S (OUTPUT OF GEGM) ARE WORNG
## Fertility: for age j=Jc
    j=par.Jc_pos-1
    V[j],C[j],Ck[j],S[j],Np,Tp[j] = prob_fertility(par,options,V[j+1],C[j+1],j)
    if options.timer_on == 'Y':
        print(f'j: {par.age[j]}, time: {toc()} sec\n')  

    ## Work young: for ages j=Je2+1:Jc-1
    for j in range(par.Jc_pos-2,par.Je2_pos,-1):
        V[j],C[j],S[j] = prob_work_young(par,options,V[j+1],C[j+1],j)
        if options.timer_on == 'Y':
            print(f'j: {par.age[j]}, time: {toc()} sec\n')

    ## Solve Education choice
    j = par.Je2_pos
    Ve,Ce,Se,Vc1,tau1 = prob_educ(par,options,V[j+1],C[j+1])
    if options.timer_on == 'Y':
        print(f'j: {par.age[j]}, time: {toc()} sec\n, iter:{iter_}, distV: {distV}')


    difV  = np.abs(Vc1-Vc0)
    distV = np.max(difV[:])
    tau0  = tau1

#    pace    = 0.25;
    pace    = 0.9
    Vc0 = pace*Vc1+(1-pace)*Vc0


    iter_ = iter_+1
    #if options.model_iter_on == 'Y':
       # fprintf('HH iter %03i, norm Vchild : %9.4f, tfs = %03.0f sec\n', iter, distV,toc(ticiter));


#     fprintf(' \n')


if (distV>options.tolV) == 1:
    flags.converge = 'N'
else:
    flags.converge = 'Y'


# keyboard
model.pol = SimpleNamespace()
pol = model.pol
## Output
# Policy functions:
pol.C         = copy.deepcopy(C)
pol.Ce        = copy.deepcopy(Ce)
pol.Ck        = copy.deepcopy(Ck)
pol.S         = copy.deepcopy(S)
pol.Se     	  = copy.deepcopy(Se)
pol.V         = copy.deepcopy(V)
pol.Ve        = copy.deepcopy(Ve)
pol.Vc0       = copy.deepcopy(Vc0)
pol.Np        = copy.deepcopy(Np)
pol.PHIp      = copy.deepcopy(PHIp)
pol.Tp        = copy.deepcopy(Tp)
pol.tau0      = copy.deepcopy(tau1)
pol.tau       = copy.deepcopy(tau1)

# Initial guess
guess.Vc0    = Vc0

# flags
flags.iter_ = iter_

Elapsed time is 0.0 seconds.
Elapsed time is 0.0019927024841308594 seconds.
j: 78.0, time: None sec

Elapsed time is 0.01967334747314453 seconds.
j: 76.0, time: None sec

Elapsed time is 0.0341801643371582 seconds.
j: 74.0, time: None sec

Elapsed time is 0.04713106155395508 seconds.
j: 72.0, time: None sec

Elapsed time is 0.06135296821594238 seconds.
j: 70.0, time: None sec

Elapsed time is 0.07431387901306152 seconds.
j: 68.0, time: None sec

Elapsed time is 0.08878755569458008 seconds.
j: 66.0, time: None sec

Elapsed time is 0.14794921875 seconds.
j: 64.0, time: None sec



C:\Users\Giorgia\AppData\Local\Temp\ipykernel_118012\499513969.py:102: RuntimeWarning: divide by zero encountered in power
  ucp = beta*np.reshape(((1+r)),[80,])*np.dot(Cpp**(-gammac), np.reshape(innop_prob,[5,]))


Elapsed time is 0.832944393157959 seconds.
j: 62.0, time: None sec

Elapsed time is 1.5202398300170898 seconds.
j: 60.0, time: None sec

Elapsed time is 2.174287796020508 seconds.
j: 58.0, time: None sec

Elapsed time is 2.7951760292053223 seconds.
j: 56.0, time: None sec

Elapsed time is 3.46525502204895 seconds.
j: 54.0, time: None sec

Elapsed time is 4.08674693107605 seconds.
j: 52.0, time: None sec

Elapsed time is 4.732452869415283 seconds.
j: 50.0, time: None sec

Elapsed time is 5.383139610290527 seconds.
j: 48.0, time: None sec

Elapsed time is 5.999464750289917 seconds.
j: 46.0, time: None sec

Elapsed time is 6.6285998821258545 seconds.
j: 44.0, time: None sec



C:\Users\Giorgia\AppData\Local\Temp\ipykernel_118012\506952869.py:45: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(arrays[k][:, idx[k]] for k in range(n))


Elapsed time is 54.24649500846863 seconds.
j: 42.0, time: None sec



C:\Users\Giorgia\AppData\Local\Temp\ipykernel_118012\2610672.py:136: RuntimeWarning: divide by zero encountered in power
  ucp = beta*np.reshape(((1+r)),[80,1])*np.dot(Cpp**(-gammac), np.reshape(innop_prob,[5,1]))
C:\Users\Giorgia\AppData\Local\Temp\ipykernel_118012\1918662882.py:38: RuntimeWarning: invalid value encountered in subtract
  difmin = np.append(ucp[:,0:-1]- ucp[:,1::],0) #wrong
C:\Users\Giorgia\AppData\Local\Temp\ipykernel_118012\1918662882.py:41: RuntimeWarning: invalid value encountered in subtract
  difmax = np.append(0, ucp[:,1::] - ucp[:,0:-1]) #wrong


Elapsed time is 78.76339054107666 seconds.
j: 40.0, time: None sec

Elapsed time is 124.10642766952515 seconds.
j: 38.0, time: None sec

Elapsed time is 174.9248969554901 seconds.
j: 36.0, time: None sec

Elapsed time is 229.27889251708984 seconds.
j: 34.0, time: None sec

Elapsed time is 289.24540758132935 seconds.
j: 32.0, time: None sec

Elapsed time is 351.66835379600525 seconds.
j: 30.0, time: None sec



C:\Users\Giorgia\AppData\Local\Temp\ipykernel_118012\1065135521.py:150: RuntimeWarning: divide by zero encountered in power
  ucp = beta*np.reshape(((1+r)),[80,1])*np.dot(Cpp**(-gammac), np.reshape(innop_prob,[5,1]))


Elapsed time is 412.3359136581421 seconds.
j: 28.0, time: None sec

Elapsed time is 430.551922082901 seconds.
j: 26.0, time: None sec

Elapsed time is 451.5718159675598 seconds.
j: 24.0, time: None sec

Elapsed time is 467.5695056915283 seconds.
j: 22.0, time: None sec

Elapsed time is 478.97005915641785 seconds.
j: 20.0, time: None sec
, iter:0, distV: 1
Elapsed time is 526.472540140152 seconds.
j: 42.0, time: None sec

Elapsed time is 572.1892495155334 seconds.
j: 40.0, time: None sec

Elapsed time is 617.3774421215057 seconds.
j: 38.0, time: None sec

Elapsed time is 666.8095231056213 seconds.
j: 36.0, time: None sec

Elapsed time is 719.3797860145569 seconds.
j: 34.0, time: None sec

Elapsed time is 774.769612789154 seconds.
j: 32.0, time: None sec

Elapsed time is 832.2231628894806 seconds.
j: 30.0, time: None sec

Elapsed time is 892.7313134670258 seconds.
j: 28.0, time: None sec

Elapsed time is 902.6199786663055 seconds.
j: 26.0, time: None sec

Elapsed time is 907.207701921463

KeyboardInterrupt: 

In [62]:
# -*- coding: utf-8 -*-
"""solve_ergodic_distribution1
"""#https://github.com/aprsa/ndpolator/blob/main/ndpolator/ndpolator.py
from scipy.interpolate import LinearNDInterpolator
from scipy.interpolate import interpn
import types, copy
def solve_ergodic_distribution1(par,pol,options):
# Interpolate policy functions into dense grid

    ##%% Distribute policy functions
    S           = copy.deepcopy(pol.S)
    Se          = copy.deepcopy(pol.Se)
    C           = copy.deepcopy(pol.C)
    Ce          = copy.deepcopy(pol.Ce)
    Ck          = copy.deepcopy(pol.Ck)
    Np          = copy.deepcopy(pol.Np)
    PHIp        = copy.deepcopy(pol.PHIp)
    Tp          = copy.deepcopy(pol.Tp)
    tau0        = copy.deepcopy(pol.tau0)
    
    #del pol
    
    
   ## Interpolate policy functions in a dense grid
    # Parameters
    EDUC                            = par.educ
    N                               = par.N
    PSY                             = par.psy_val_hs
    FE_pos                          = par.inc.fe_pos
    FE_pos                          = np.array(FE_pos)
    INNO_pos                        = par.inc.inno_pos
    INNO_pos                        = np.array(INNO_pos)
    
    # Arrays for new policies
    S_dense           = {}
    for i in range (0, par.Jd_pos):
        S_dense[i] = [[]]
    
    

    Se_dense                  = {}
    for i in range(0,3):
         Se_dense[i] = [[],[],[]]
   
    C_dense           = {}
    for i in range (0, par.Jd_pos):
        C_dense[i] = [[]]
   
    Ck_dense           = {}
    for i in range (0, par.Jd_pos):
        Ck_dense[i] = [[]]
    
    Ce_dense                  = {}
    for i in range(0,3):
         Ce_dense[i] = [[],[],[]]   
    # for i in range (0, 3):
    #     a = [0]
    #     Ce_dense[i]= [a]*3
    
    Tp_dense           = {}
    for i in range (0, par.Jd_pos):
        Tp_dense[i] = [[]]
    
    ## 1. Create dense grid
    curv                            = 3
    
    par.grids_dense                 = {}
    for i in range (0, len(EDUC)):
        a = [0]
        par.grids_dense[i]= [a]*par.Jd_pos ##################may need to transpose
    
    for educ in range (0, len(EDUC)):
        par.grids_dense[educ][par.Je1_pos-1] = par.grids[educ][par.Je1_pos-1]

    n_2_s                           = 100
    
    
    for j_pos in range (par.Je1_pos,par.Jd_pos):
        for educ in range (0, len(EDUC)):
            S0                          = par.grids[educ][j_pos]
            s_min                       = abs(np.min(S0))
            s_max                       = abs(np.max(S0))
            par.grids_dense[educ][j_pos] = np.linspace(-s_min**(1/curv),s_max**(1/curv),n_2_s)**curv

    
    ## 2. Interpolate policy function of savings for education
    educ  = 0
    for t in range(0,3):
        S_dense_aux      = par.grids_dense[educ][par.Je1_pos+t-1]
        S_orig           = par.grids[educ][par.Je1_pos+t-1]
        Sp_max           = np.max(par.grids_dense[educ][par.Je1_pos+t])
    
    
        S_pol            = Se[educ][t]
        C_pol            = Ce[educ][t]
        if np.max(S_pol[:]) > Sp_max:### probably will need to se for i in S_pol, S_pol[i] > Sp_max
            j_pos        = par.Je1_pos+t-1
            print(f'Attention: age = {par.age[j_pos]}, educ = {educ}, extrapolation in ergodic distribution, max extrap = {log(np.max(S_pol[:])/Sp_max)}')
            
            S_pol        = np.where(S_pol>Sp_max, Sp_max, S_pol)
     
        Sp_min           = np.min(par.grids_dense[educ][par.Je1_pos+t])
        if np.min(S_pol[:]) < Sp_min:##### or any??--- no , min and max should be ok
            j_pos        = par.Je1_pos+t-1
            print(f'Attention: age ={par.age(j_pos)}, educ = {educ}, extrapolation in ergodic distribution, min extrap = {log(np.min(S_pol[:])/Sp_min)}')
            S_pol        = np.where(S_pol<Sp_min, Sp_min, S_pol)##########################check!!
            
        pol = np.zeros([len(INNO_pos),len(S_dense_aux),len(FE_pos)])
        for i in range (0, len(INNO_pos)):
            for j in range(0, len(S_dense_aux)):
                for k in range (0, len(FE_pos)):
                    pol[i,j,k] = interp_3d(INNO_pos,S_orig,FE_pos,S_pol,INNO_pos[i],S_dense_aux[j],FE_pos[k])
                    
        Se_dense[educ][t] = np.reshape(pol,[len(INNO_pos),len(S_dense_aux),len(FE_pos)])   ### probably no need to reshape         

        
        pol = np.zeros([len(INNO_pos),len(S_dense_aux),len(FE_pos)])
        for i in range (0, len(INNO_pos)):
            for j in range(0, len(S_dense_aux)):
                for k in range (0, len(FE_pos)):
                    pol[i,j,k] = interp_3d(INNO_pos,S_orig,FE_pos,C_pol,INNO_pos[i],S_dense_aux[j],FE_pos[k])
        Ce_dense[educ][t] = np.reshape(pol,[len(INNO_pos),len(S_dense_aux),len(FE_pos)])  
    
        
    
    educ  = 1
    for t in range(0,3):
        S_dense_aux      = par.grids_dense[educ][par.Je1_pos+t-1]
        S_orig           = par.grids[educ][par.Je1_pos+t-1]
        Sp_max           = np.max(par.grids_dense[educ][par.Je1_pos+t])
        
        S_pol            = Se[educ][t]
        C_pol            = Ce[educ][t]
        if np.max(np.ravel(S_pol[:])) > Sp_max:
            j_pos        = par.Je1_pos+t-1
            print(f'Attention: age = {par.age[j_pos]}, educ = {educ}, extrapolation in ergodic distribution, max extrap = {log(np.max(S_pol[:])/Sp_max)}')
            S_pol        = np.where(S_pol>Sp_max, Sp_max, S_pol)
        
        Sp_min           = min(par.grids_dense[educ][par.Je1_pos+t])
        if min(np.ravel(S_pol[:])) < Sp_min:
            j_pos        = par.Je1_pos+t-1;
            print(f'Attention: age ={par.age[j_pos]}, educ = {educ}, extrapolation in ergodic distribution, min extrap = {log(np.min(S_pol[:])/Sp_min)}')
            S_pol        = np.where(S_pol<Sp_min, Sp_min, S_pol)################until here
        
        if t <= 0:
            psy_orig         = PSY
            S1,FE1,PSY1    = np.meshgrid(S_orig,FE_pos,psy_orig)
            S2,FE2,PSY2   = np.meshgrid(S_dense_aux,FE_pos,psy_orig)
            
            pol = np.zeros([len(psy_orig),len(S_dense_aux),len(FE_pos)])
            for i in range (0, len(psy_orig)):
                for j in range(0, len(S_dense_aux)):
                    for k in range (0, len(FE_pos)):
                        pol[i,j,k] = interp_3d(psy_orig,S_orig,FE_pos,S_pol,psy_orig[i],S_dense_aux[j],FE_pos[k])
                        
            Se_dense[educ][t] = pol#np.reshape(pol,[len(INNO_pos),len(S_dense_aux),len(FE_pos)])
            #Se_dense[educ][t][:,:,:]    = reshape(pol(S2,FE2,PSY2), length(S_dense_aux),length(FE_pos),length(psy_orig))
            
            
            pol = np.zeros([len(psy_orig),len(S_dense_aux),len(FE_pos)])
            for i in range (0, len(psy_orig)):
                for j in range(0, len(S_dense_aux)):
                    for k in range (0, len(FE_pos)):
                        pol[i,j,k] = interp_3d(psy_orig,S_orig,FE_pos,C_pol,psy_orig[i],S_dense_aux[j],FE_pos[k])
            
            Ce_dense[educ][t] = pol
            
        else:
            S1,FE1,INNO1   = np.meshgrid(S_orig,FE_pos,INNO_pos)
            S2,FE2,INNO2   = np.meshgrid(S_dense_aux,FE_pos,INNO_pos)
            
            pol = np.zeros([len(INNO_pos),len(S_dense_aux),len(FE_pos)])
            for i in range (0, len(INNO_pos)):
                for j in range(0, len(S_dense_aux)):
                    for k in range (0, len(FE_pos)):
                        pol[i,j,k] = interp_3d(INNO_pos,S_orig,FE_pos,S_pol,INNO_pos[i],S_dense_aux[j],FE_pos[k])
            Se_dense[educ][t] = pol
            
            pol = np.zeros([len(INNO_pos),len(S_dense_aux),len(FE_pos)])
            for i in range (0, len(INNO_pos)):
                for j in range(0, len(S_dense_aux)):
                    for k in range (0, len(FE_pos)):
                        pol[i,j,k] = interp_3d(INNO_pos,S_orig,FE_pos,C_pol,INNO_pos[i],S_dense_aux[j],FE_pos[k])
            Ce_dense[educ][t] = pol

        
    educ  = 2
    for t in range(0,3):
        S_dense_aux      = par.grids_dense[educ][par.Je1_pos+t-1]
        S_orig           = par.grids[educ][par.Je1_pos+t-1]
        Sp_max           = np.max(par.grids_dense[educ][par.Je1_pos+t])
        
        psy_orig             = PSY
        
        S_pol            = Se[educ][t]
        C_pol            = Ce[educ][t]
        
        if np.max(np.ravel(S_pol[:])) > Sp_max:
            j_pos        = par.Je1_pos+t-1
            print(f'Attention: age = {par.age[j_pos]}, educ = {educ}, extrapolation in ergodic distribution, max extrap = {log(np.max(S_pol[:])/Sp_max)}')
            S_pol        = np.where(S_pol>Sp_max, Sp_max, S_pol)
        
        Sp_min           = np.min(par.grids_dense[educ][par.Je1_pos+t])
        if np.min(np.ravel(S_pol[:])) < Sp_min:
            j_pos        = par.Je1_pos+t-1;
            print(f'Attention: age ={par.age[j_pos]}, educ = {educ}, extrapolation in ergodic distribution, min extrap = {log(np.min(S_pol[:])/Sp_min)}')
            S_pol = np.where(S_pol<Sp_min, Sp_min, S_pol)
    
        if t <= 0:
            psy_orig         = PSY
            S1,FE1,PSY1    = np.meshgrid(S_orig,FE_pos,psy_orig)
            S2,FE2,PSY2   = np.meshgrid(S_dense_aux,FE_pos,psy_orig)
            
            pol = np.zeros([len(psy_orig),len(S_dense_aux),len(FE_pos)])
            for i in range (0, len(psy_orig)):
                for j in range(0, len(S_dense_aux)):
                    for k in range (0, len(FE_pos)):
                        pol[i,j,k] = interp_3d(psy_orig,S_orig,FE_pos,S_pol,psy_orig[i],S_dense_aux[j],FE_pos[k])
                        
            Se_dense[educ][t] = pol#np.reshape(pol,[len(INNO_pos),len(S_dense_aux),len(FE_pos)])
            #Se_dense[educ][t][:,:,:]    = reshape(pol(S2,FE2,PSY2), length(S_dense_aux),length(FE_pos),length(psy_orig))
            
            
            pol = np.zeros([len(psy_orig),len(S_dense_aux),len(FE_pos)])
            for i in range (0, len(psy_orig)):
                for j in range(0, len(S_dense_aux)):
                    for k in range (0, len(FE_pos)):
                        pol[i,j,k] = interp_3d(psy_orig,S_orig,FE_pos,C_pol,psy_orig[i],S_dense_aux[j],FE_pos[k])
            
            Ce_dense[educ][t] = pol
            
            
        else:
            # S1,FE1   = np.meshgrid(S_orig,FE_pos)
            # S2,FE2   = np.meshgrid(S_dense_aux)
            pol = np.zeros([len(S_dense_aux),len(FE_pos)])
            
            for j in range(0, len(S_dense_aux)):
                for k in range (0, len(FE_pos)):
                    pol[j,k] = interp_2d(S_orig,FE_pos,S_pol,S_dense_aux[j],FE_pos[k])
            
            Se_dense[educ][t]= pol
            
            
            pol = np.zeros([len(S_dense_aux),len(FE_pos)])
            for j in range(0, len(S_dense_aux)):
                for k in range (0, len(FE_pos)):
                    pol[j,k] = interp_2d(S_orig,FE_pos,C_pol,S_dense_aux[j],FE_pos[k])
            
            Ce_dense[educ][t] = pol
        
        #del Se
    
   ## 3. Interpolate policy function of savings for working young
    for j_pos in range (par.Je2_pos+1,par.Jc_pos-1):
        S_dense[j_pos]        = np.zeros([len(INNO_pos),len(EDUC),len(par.grids_dense[0][j_pos]),len(FE_pos)])
        S_dense[j_pos][:] = np.nan
        C_dense[j_pos]        = np.zeros([len(INNO_pos),len(EDUC),len(par.grids_dense[0][j_pos]),len(FE_pos)])
        C_dense[j_pos][:] = np.nan

        for educ in range (0,len(EDUC)):
            
            S_dense_aux      = par.grids_dense[educ][j_pos]
            S_orig           = par.grids[educ][j_pos]#### to transpose
            
            Sp_max           = np.max(par.grids_dense[educ][j_pos+1])
            S_pol            = np.squeeze(S[j_pos][:,educ,:,:])
            C_pol            = np.squeeze(C[j_pos][:,educ,:,:])
            
            if np.max(np.ravel(S_pol[:])) > Sp_max:
                print(f'Attention: age = {par.age[j_pos]}, educ = {educ}, extrapolation in ergodic distribution, max extrap = {log(np.max(S_pol[:])/Sp_max)}')
                S_pol        = np.where(S_pol>Sp_max, Sp_max, S_pol)
    
            
            Sp_min           = np.min(par.grids_dense[educ][j_pos+1])
            if np.min(np.ravel(S_pol[:])) > Sp_min:
                print(f'Attention: age = {par.age[j_pos]}, educ = {educ}, extrapolation in ergodic distribution, min extrap = {log(np.min(S_pol[:])/Sp_min)}')
                S_pol = np.where(S_pol<Sp_min, Sp_min, S_pol)
    
            
            # S1,FE1,INNO1   = np.meshgrid(S_orig,FE_pos,INNO_pos)
            # S2,FE2,INNO2   = np.meshgrid(S_dense_aux,FE_pos,INNO_pos)
            
            pol = np.zeros([len(INNO_pos),len(S_dense_aux),len(FE_pos)])
            for i in range (0, len(INNO_pos)):
                for j in range(0, len(S_dense_aux)):
                    for k in range (0, len(FE_pos)):
                        pol[i,j,k] = interp_3d(INNO_pos,S_orig,FE_pos,S_pol,INNO_pos[i],S_dense_aux[j],FE_pos[k])
                        
                        
            S_dense[j_pos][:,educ,:,:]  = pol
            
            pol = np.zeros([len(INNO_pos),len(S_dense_aux),len(FE_pos)])
            for i in range (0, len(INNO_pos)):
                for j in range(0, len(S_dense_aux)):
                    for k in range (0, len(FE_pos)):
                        pol[i,j,k] = interp_3d(INNO_pos,S_orig,FE_pos,C_pol,INNO_pos[i],S_dense_aux[j],FE_pos[k])
            
            C_dense[j_pos][:,educ,:,:]  = pol
        

    
    ## 4. Interpolate policy function of savings for fertility
    j_pos                   = par.Jc_pos-1
    S_dense[j_pos]          = np.zeros([len(INNO_pos),len(EDUC),len(par.grids_dense[0][j_pos]),len(FE_pos)])
    S_dense[j_pos][:]       = np.nan
    C_dense[j_pos]          = np.zeros([len(INNO_pos),len(EDUC),len(par.grids_dense[0][j_pos]),len(FE_pos)])
    C_dense[j_pos][:]       = np.nan
    Ck_dense[j_pos]         = np.zeros([len(INNO_pos),len(EDUC),len(par.grids_dense[0][j_pos]),len(FE_pos)])
    Ck_dense[j_pos][:]      = np.nan
    Np_dense                = np.zeros([len(INNO_pos),len(EDUC),len(par.grids_dense[0][j_pos]),len(FE_pos)])
    Np_dense[:]             = np.nan
    Tp_dense[j_pos]         = np.zeros([len(INNO_pos),len(EDUC),len(par.grids_dense[0][j_pos]),len(FE_pos)])
    Tp_dense[j_pos][:]        = np.nan
    
    for educ in range(0,len(EDUC)):      
        S_dense_aux         = par.grids_dense[educ][j_pos]
        S_orig              = par.grids[educ][j_pos]
        Sp_max              = np.max(par.grids_dense[educ][j_pos+1])
    
        S_pol               = np.squeeze(S[j_pos][:,educ,:,:])
        C_pol               = np.squeeze(C[j_pos][:,educ,:,:])
        Ck_pol              = np.squeeze(Ck[j_pos][:,educ,:,:])
        
        if np.max(np.ravel(S_pol[:])) > Sp_max:
            print(f'Attention: age = {par.age[j_pos]}, educ = {educ}, extrapolation in ergodic distribution, min extrap = {log(np.min(S_pol[:])/Sp_min)}')
            S_pol           = np.where(S_pol>Sp_max, Sp_max, S_pol)
        
            Sp_min          = np.min(par.grids_dense[educ][j_pos+1])
        
        if np.min(S_pol[:]) > Sp_min:
            print(f'Attention: age = {par.age[j_pos]}, educ = {educ}, extrapolation in ergodic distribution, min extrap = {log(np.min(S_pol[:])/Sp_min)}')
            S_pol           = np.where(S_pol<Sp_min, Sp_min, S_pol)
            
        pol = np.zeros([len(INNO_pos),len(S_dense_aux),len(FE_pos)])
        for i in range (0, len(INNO_pos)):
            for j in range(0, len(S_dense_aux)):
                for k in range (0, len(FE_pos)):
                    pol[i,j,k] = interp_3d(INNO_pos,S_orig,FE_pos,S_pol,INNO_pos[i],S_dense_aux[j],FE_pos[k])
                    
                    
        S_dense[j_pos][:,educ,:,:]  = pol
        
        pol = np.zeros([len(INNO_pos),len(S_dense_aux),len(FE_pos)])
        for i in range (0, len(INNO_pos)):
            for j in range(0, len(S_dense_aux)):
                for k in range (0, len(FE_pos)):
                    pol[i,j,k] = interp_3d(INNO_pos,S_orig,FE_pos,C_pol,INNO_pos[i],S_dense_aux[j],FE_pos[k])
        
        C_dense[j_pos][:,educ,:,:]  = pol
        
        
        pol = np.zeros([len(INNO_pos),len(S_dense_aux),len(FE_pos)])
        for i in range (0, len(INNO_pos)):
            for j in range(0, len(S_dense_aux)):
                for k in range (0, len(FE_pos)):
                    pol[i,j,k] = interp_3d(INNO_pos,S_orig,FE_pos,Ck_pol,INNO_pos[i],S_dense_aux[j],FE_pos[k])
        
        Ck_dense[j_pos][:,educ,:,:]  = pol
        
        Np_pol                      = np.squeeze(Np[:,educ,:,:])
        pol = np.zeros([len(INNO_pos),len(S_dense_aux),len(FE_pos)])
        for i in range (0, len(INNO_pos)):
            for j in range(0, len(S_dense_aux)):
                for k in range (0, len(FE_pos)):
                    points = (INNO_pos,S_orig,FE_pos)
                    point = np.array([INNO_pos[i],S_dense_aux[j],FE_pos[k]])
                    pol[i,j,k] = int(interpn(points,Np_pol,point,method = "nearest"))###################################careful to check if results are the same, here I use nearest neighboorhood extrapolation, and int() to round numbers, but matlba may want to have a nearest neightborhood interpolation
        
        
        Np_dense[:,educ,:,:]        = pol
        
        Tp_pol                      = np.squeeze(Tp[j_pos][:,educ,:,:])
        pol = np.zeros([len(INNO_pos),len(S_dense_aux),len(FE_pos)])
        for i in range (0, len(INNO_pos)):
            for j in range(0, len(S_dense_aux)):
                for k in range (0, len(FE_pos)):
                    points = (INNO_pos,S_orig,FE_pos)
                    point = np.array([INNO_pos[i],S_dense_aux[j],FE_pos[k]])
                    pol[i,j,k] = (interpn(points,Tp_pol,point,method = "nearest"))###################################careful to check if results are the same, here I use nearest neighboorhood extrapolation, and int() to round numbers, but matlba may want to have a nearest neightborhood interpolation
                    
        Tp_dense[j_pos][:,educ,:,:]  = pol
        
        ## 5. Interpolate policy function of savings for work with child
    for j_pos  in range  (par.Jc_pos,par.Jc_pos+par.Je1_pos-3):
        
        S_dense[j_pos]          = np.zeros([len(INNO_pos),len(EDUC),len(N),len(par.grids_dense[0][j_pos]),len(FE_pos)])
        S_dense[j_pos][:]       = np.nan
        C_dense[j_pos]          = np.zeros([len(INNO_pos),len(EDUC),len(N),len(par.grids_dense[0][j_pos]),len(FE_pos)])
        C_dense[j_pos][:]       = np.nan
        Ck_dense[j_pos]         = np.zeros([len(INNO_pos),len(EDUC),len(N),len(par.grids_dense[0][j_pos]),len(FE_pos)])
        Ck_dense[j_pos][:]      = np.nan
        Tp_dense[j_pos]         = np.zeros([len(INNO_pos),len(EDUC),len(N),len(par.grids_dense[0][j_pos]),len(FE_pos)])
        Tp_dense[j_pos][:]        = np.nan
        
        for educ in range(0,len(EDUC)):
            S_dense_aux       = par.grids_dense[educ][j_pos]
            S_orig            = par.grids[educ][j_pos]
            Sp_max            = np.max(par.grids_dense[educ][j_pos+1])
                
            S_pol             = np.squeeze(S[j_pos][:,educ,:,:,:])
            C_pol             = np.squeeze(C[j_pos][:,educ,:,:,:])
            Ck_pol            = np.squeeze(Ck[j_pos][:,educ,:,:,:])
            Tp_pol            = np.squeeze(Tp[j_pos][:,educ,:,:,:])
            
            if np.max(ravel(S_pol[:])) > Sp_max:
                print(f'Attention: age = {par.age[j_pos]}, educ = {educ}, extrapolation in ergodic distribution, min extrap = {log(np.max(S_pol[:])/Sp_max)}')
                S_pol           = np.where(S_pol>Sp_max, Sp_max, S_pol)
            
                Sp_min           = np.min(par.grids_dense[educ][j_pos+1])
            if np.min(np.ravel(S_pol[:])) > Sp_min:
                print(f'Attention: age = {par.age[j_pos]}, educ = {educ}, extrapolation in ergodic distribution, min extrap = {log(np.min(S_pol[:])/Sp_min)}')
                S_pol           = np.where(S_pol<Sp_min, Sp_min, S_pol)
        
            if options.Fertility == 'Endo':
                    
                pol = np.zeros([len(INNO_pos),len(N),len(S_dense_aux),len(FE_pos)])
                for i in range (0, len(INNO_pos)):
                    for n in range (0,len(N)):
                        for j in range(0, len(S_dense_aux)):
                            for k in range (0, len(FE_pos)):
                                pol[i,n,j,k] = interp_4d(INNO_pos,N,S_orig,FE_pos,S_pol,INNO_pos[i],N[n],S_dense_aux[j],FE_pos[k])
                            
                            
                S_dense[j_pos][:,educ,:,:,:]  = pol
                
                pol = np.zeros([len(INNO_pos),len(N),len(S_dense_aux),len(FE_pos)])
                for i in range (0, len(INNO_pos)):
                    for n in range (0,len(N)):
                        for j in range(0, len(S_dense_aux)):
                            for k in range (0, len(FE_pos)):
                                pol[i,n,j,k] = interp_4d(INNO_pos,N,S_orig,FE_pos,C_pol,INNO_pos[i],N[n],S_dense_aux[j],FE_pos[k])
                            
                C_dense[j_pos][:,educ,:,:,:]   = pol
                
                pol = np.zeros([len(INNO_pos),len(N),len(S_dense_aux),len(FE_pos)])
                for i in range (0, len(INNO_pos)):
                    for n in range (0,len(N)):
                        for j in range(0, len(S_dense_aux)):
                            for k in range (0, len(FE_pos)):
                                pol[i,n,j,k] = interp_4d(INNO_pos,N,S_orig,FE_pos,Ck_pol,INNO_pos[i],N[n],S_dense_aux[j],FE_pos[k])

                Ck_dense[j_pos][:,educ,:,:,:]  = pol
                
                pol = np.zeros([len(INNO_pos),len(N),len(S_dense_aux),len(FE_pos)])
                for i in range (0, len(INNO_pos)):
                    for n in range (0,len(N)):
                        for j in range(0, len(S_dense_aux)):
                            for k in range (0, len(FE_pos)):
                                points = (INNO_pos,N,S_orig,FE_pos)
                                point = np.array([INNO_pos[i],N[n],S_dense_aux[j],FE_pos[k]])
                                pol[i,n,j,k] = (interpn(points,Tp_pol,point,method = "nearest"))
                             
                Tp_dense[j_pos][:,educ,:,:,:]  = pol
             
            elif options.Fertility == 'Exo':  ##########################################################################     
                pol = np.zeros([len(INNO_pos),len(S_dense_aux),len(FE_pos)])
                for i in range (0, len(INNO_pos)):
                    for j in range(0, len(S_dense_aux)):
                        for k in range (0, len(FE_pos)):
                            pol[i,j,k] = interp_3d(INNO_pos,S_orig,FE_pos,S_pol,INNO_pos[i],S_dense_aux[j],FE_pos[k])
                S_dense[j_pos][:,educ,0,:,:]  = pol
                
                pol = np.zeros([len(INNO_pos),len(S_dense_aux),len(FE_pos)])
                for i in range (0, len(INNO_pos)):
                    for j in range(0, len(S_dense_aux)):
                        for k in range (0, len(FE_pos)):
                            pol[i,j,k] = interp_3d(INNO_pos,S_orig,FE_pos,C_pol,INNO_pos[i],S_dense_aux[j],FE_pos[k])
                C_dense[j_pos][:,educ,0,:,:] = pol
                
                pol = np.zeros([len(INNO_pos),len(S_dense_aux),len(FE_pos)])
                for i in range (0, len(INNO_pos)):
                    for j in range(0, len(S_dense_aux)):
                        for k in range (0, len(FE_pos)):
                            pol[i,j,k] = interp_3d(INNO_pos,S_orig,FE_pos,Ck_pol,INNO_pos[i],S_dense_aux[j],FE_pos[k])
                Ck_dense[j_pos][:,educ,0,:,:]   = pol
                
                Tp_pol                      = np.squeeze(Tp[j_pos][:,educ,:,:])
                
                pol = np.zeros([len(INNO_pos),len(S_dense_aux),len(FE_pos)])
                for i in range (0, len(INNO_pos)):
                    for j in range(0, len(S_dense_aux)):
                        for k in range (0, len(FE_pos)):
                            points = (INNO_pos,S_orig,FE_pos)
                            point = np.array([INNO_pos[i],S_dense_aux[j],FE_pos[k]])
                            pol[i,j,k] = (interpn(points,Tp_pol,point,method = "nearest"))
                            
                Tp_dense[j_pos][:,educ,0,:,:]   = reshape(pol(S2,FE2,INNO2),length(S_dense_aux),length(FE_pos),1,length(INNO_pos))
                    
        
            ## 6. Interpolate policy function of savings for work + transfers to child
            j_pos                       = par.Jc_pos+par.Je1_pos-3
            
            S_dense[j_pos]          = np.zeros([len(INNO_pos),len(EDUC),len(N),len(par.grids_dense[0][j_pos]),len(FE_pos)])
            S_dense[j_pos][:]       = np.nan
            C_dense[j_pos]          = np.zeros([len(INNO_pos),len(EDUC),len(N),len(par.grids_dense[0][j_pos]),len(FE_pos)])
            C_dense[j_pos][:]       = np.nan
            Ck_dense[j_pos]         = np.zeros([len(INNO_pos),len(EDUC),len(N),len(par.grids_dense[0][j_pos]),len(FE_pos)])
            Ck_dense[j_pos][:]      = np.nan
            Tp_dense[j_pos]         = np.zeros([len(INNO_pos),len(EDUC),len(N),len(par.grids_dense[0][j_pos]),len(FE_pos)])
            Tp_dense[j_pos][:]      = np.nan
            PHIp_dense              = np.zeros([len(INNO_pos),len(EDUC),len(N),len(par.grids_dense[0][j_pos]),len(FE_pos)])
            PHIp_dense[:]           = np.nan
            
            for educ in range (0,len(EDUC)):
                S_dense_aux     = par.grids_dense[educ][j_pos]
                S_orig          = par.grids[educ][j_pos]
                Sp_max          = np.max(par.grids_dense[educ][j_pos+1])
                
            
                S_pol           = np.squeeze(S[j_pos][:,educ,:,:,:])
                C_pol           = np.squeeze(C[j_pos][:,educ,:,:,:])
                Ck_pol          = np.squeeze(Ck[j_pos][:,educ,:,:,:])
                Tp_pol          = np.squeeze(Tp[j_pos][:,educ,:,:,:])
                PHIp_pol        = np.squeeze(PHIp[:,educ,:,:,:])
                
                #     aux               = reshape(1*(H_dense_aux<= CUTOFFS(educ,1)) + ...
                #         2.*(H_dense_aux> CUTOFFS(educ,1)).*(H_dense_aux<= CUTOFFS(educ,2)) +...
                #         3.*(H_dense_aux> CUTOFFS(educ,2)),1,length(H_dense_aux));
                #     Grs_dense{j_pos}(:,:,educ,:)  = repmat(aux,length(par.grids_dense{1,j_pos}),1,1,length(N));

    
                if np.max(ravel(S_pol[:])) > Sp_max:
                    print(f'Attention: age = {par.age[j_pos]}, educ = {educ}, extrapolation in ergodic distribution, min extrap = {log(np.max(S_pol[:])/Sp_max)}')
                    S_pol           = np.where(S_pol>Sp_max, Sp_max, S_pol)
                
                    Sp_min           = np.min(par.grids_dense[educ][j_pos+1])
                if np.min(np.ravel(S_pol[:])) > Sp_min:
                    print(f'Attention: age = {par.age[j_pos]}, educ = {educ}, extrapolation in ergodic distribution, min extrap = {log(np.min(S_pol[:])/Sp_min)}')
                    S_pol           = np.where(S_pol<Sp_min, Sp_min, S_pol)
    

                if options.Fertility == 'Endo':
                    
                    pol = np.zeros([len(INNO_pos),len(N),len(S_dense_aux),len(FE_pos)])
                    for i in range (0, len(INNO_pos)):
                        for n in range (0,len(N)):
                            for j in range(0, len(S_dense_aux)):
                                for k in range (0, len(FE_pos)):
                                    pol[i,n,j,k] = interp_4d(INNO_pos,N,S_orig,FE_pos,S_pol,INNO_pos[i],N[n],S_dense_aux[j],FE_pos[k])
                    S_dense[j_pos][:,educ,:,:,:]  = pol
                    
                    pol = np.zeros([len(INNO_pos),len(N),len(S_dense_aux),len(FE_pos)])
                    for i in range (0, len(INNO_pos)):
                        for n in range (0,len(N)):
                            for j in range(0, len(S_dense_aux)):
                                for k in range (0, len(FE_pos)):
                                    pol[i,n,j,k] = interp_4d(INNO_pos,N,S_orig,FE_pos,C_pol,INNO_pos[i],N[n],S_dense_aux[j],FE_pos[k])
                                
                    C_dense[j_pos][:,educ,:,:,:]   = pol
                    
                    pol = np.zeros([len(INNO_pos),len(N),len(S_dense_aux),len(FE_pos)])
                    for i in range (0, len(INNO_pos)):
                        for n in range (0,len(N)):
                            for j in range(0, len(S_dense_aux)):
                                for k in range (0, len(FE_pos)):
                                    pol[i,n,j,k] = interp_4d(INNO_pos,N,S_orig,FE_pos,Ck_pol,INNO_pos[i],N[n],S_dense_aux[j],FE_pos[k])

                    Ck_dense[j_pos][:,educ,:,:,:]  = pol
                    
                    pol = np.zeros([len(INNO_pos),len(N),len(S_dense_aux),len(FE_pos)])
                    for i in range (0, len(INNO_pos)):
                        for n in range (0,len(N)):
                            for j in range(0, len(S_dense_aux)):
                                for k in range (0, len(FE_pos)):
                                    points = (INNO_pos,N,S_orig,FE_pos)
                                    point = np.array([INNO_pos[i],N[n],S_dense_aux[j],FE_pos[k]])
                                    pol[i,n,j,k] = (interpn(points,Tp_pol,point,method = "nearest"))
                                 
                    Tp_dense[j_pos][:,educ,:,:,:]  = pol
                    
                    pol = np.zeros([len(INNO_pos),len(N),len(S_dense_aux),len(FE_pos)])
                    for i in range (0, len(INNO_pos)):
                        for n in range (0,len(N)):
                            for j in range(0, len(S_dense_aux)):
                                for k in range (0, len(FE_pos)):
                                    points = (INNO_pos,N,S_orig,FE_pos)
                                    point = np.array([INNO_pos[i],N[n],S_dense_aux[j],FE_pos[k]])
                                    pol[i,n,j,k] = (interpn(points,PHIp_pol,point,method = "nearest"))
                                 
                    PHIp_dense[:,educ,:,:,:]      = pol
                        
                elif options.Fertility == 'Exo':
                    pol = np.zeros([len(INNO_pos),len(S_dense_aux),len(FE_pos)])
                    for i in range (0, len(INNO_pos)):
                        for j in range(0, len(S_dense_aux)):
                            for k in range (0, len(FE_pos)):
                                pol[i,j,k] = interp_3d(INNO_pos,S_orig,FE_pos,S_pol,INNO_pos[i],S_dense_aux[j],FE_pos[k])
                    S_dense[j_pos][:,educ,0,:,:]  = pol
                    
                    pol = np.zeros([len(INNO_pos),len(S_dense_aux),len(FE_pos)])
                    for i in range (0, len(INNO_pos)):
                        for j in range(0, len(S_dense_aux)):
                            for k in range (0, len(FE_pos)):
                                pol[i,j,k] = interp_3d(INNO_pos,S_orig,FE_pos,C_pol,INNO_pos[i],S_dense_aux[j],FE_pos[k])
                    C_dense[j_pos][:,educ,0,:,:] = pol
                    
                    pol = np.zeros([len(INNO_pos),len(S_dense_aux),len(FE_pos)])
                    for i in range (0, len(INNO_pos)):
                        for j in range(0, len(S_dense_aux)):
                            for k in range (0, len(FE_pos)):
                                pol[i,j,k] = interp_3d(INNO_pos,S_orig,FE_pos,Ck_pol,INNO_pos[i],S_dense_aux[j],FE_pos[k])
                    Ck_dense[j_pos][:,educ,0,:,:]   = pol
                    
                    Tp_pol                      = np.squeeze(Tp[j_pos][:,educ,:,:])
                    
                    pol = np.zeros([len(INNO_pos),len(S_dense_aux),len(FE_pos)])
                    for i in range (0, len(INNO_pos)):
                        for j in range(0, len(S_dense_aux)):
                            for k in range (0, len(FE_pos)):
                                points = (INNO_pos,S_orig,FE_pos)
                                point = np.array([INNO_pos[i],S_dense_aux[j],FE_pos[k]])
                                pol[i,j,k] = (interpn(points,Tp_pol,point,method = "nearest"))
                                
                    Tp_dense[j_pos][:,educ,0,:,:]   = pol                   

                    pol                         = np.zeros([len(INNO_pos),len(S_dense_aux),len(FE_pos)])
                    for i in range (0, len(INNO_pos)):
                        for j in range(0, len(S_dense_aux)):
                            for k in range (0, len(FE_pos)):
                                points = (INNO_pos,S_orig,FE_pos)
                                point = np.array([INNO_pos[i],S_dense_aux[j],FE_pos[k]])
                                pol[i,j,k] = (interpn(points,PHIp_pol,point,method = "nearest"))
                                
                    PHIp_dense[:,educ,0,:,:]        = pol

        
        ## 6. Interpolate policy function of savings for work old
        for j_pos       in range (par.Jc_pos+par.Je1_pos-2, par.Jr_pos-1):
            S_dense[j_pos]          = np.zeros([len(INNO_pos),len(EDUC),len(par.grids_dense[0][j_pos]),len(FE_pos)])
            S_dense[j_pos][:]       = np.nan
            C_dense[j_pos]          = np.zeros([len(INNO_pos),len(EDUC),len(par.grids_dense[0][j_pos]),len(FE_pos)])
            C_dense[j_pos][:]       = np.nan
            for educ in range  (0,len(EDUC)):
                S_dense_aux = par.grids_dense[educ][j_pos]
                S_orig      = par.grids[educ][j_pos]
                Sp_max      = np.max(par.grids_dense[educ][j_pos+1])
            
                S_pol       = np.squeeze(S[j_pos][:,educ,:,:])
                C_pol       = np.squeeze(C[j_pos][:,educ,:,:])
                
                if np.max(ravel(S_pol[:])) > Sp_max:
                    print(f'Attention: age = {par.age[j_pos]}, educ = {educ}, extrapolation in ergodic distribution, min extrap = {log(np.max(S_pol[:])/Sp_max)}')
                    S_pol           = np.where(S_pol>Sp_max, Sp_max, S_pol)
                
                    Sp_min           = np.min(par.grids_dense[educ][j_pos+1])
                if np.min(np.ravel(S_pol[:])) > Sp_min:
                    print(f'Attention: age = {par.age[j_pos]}, educ = {educ}, extrapolation in ergodic distribution, min extrap = {log(np.min(S_pol[:])/Sp_min)}')
                    S_pol           = np.where(S_pol<Sp_min, Sp_min, S_pol)
                                  
                    pol = np.zeros([len(INNO_pos),len(S_dense_aux),len(FE_pos)])
                    for i in range (0, len(INNO_pos)):
                        for j in range(0, len(S_dense_aux)):
                            for k in range (0, len(FE_pos)):
                                pol[i,j,k] = interp_3d(INNO_pos,S_orig,FE_pos,S_pol,INNO_pos[i],S_dense_aux[j],FE_pos[k])
                                
                    S_dense[j_pos][:,educ,:,:] = pol        

                    
                    pol = np.zeros([len(INNO_pos),len(S_dense_aux),len(FE_pos)])
                    for i in range (0, len(INNO_pos)):
                        for j in range(0, len(S_dense_aux)):
                            for k in range (0, len(FE_pos)):
                                pol[i,j,k] = interp_3d(INNO_pos,S_orig,FE_pos,C_pol,INNO_pos[i],S_dense_aux[j],FE_pos[k])
                    C_dense[j_pos][:,educ,:,:] = pol 


        
        ## 8. Retirement
        for j_pos       in range (par.Jr_pos-1,par.Jd_pos):
            S_dense[j_pos]          = np.zeros([len(EDUC),len(par.grids_dense[0][j_pos]),len(FE_pos)])
            S_dense[j_pos][:]       = np.nan
            C_dense[j_pos]          = np.zeros([len(EDUC),len(par.grids_dense[0][j_pos]),len(FE_pos)])
            C_dense[j_pos][:]       = np.nan
            
            for educ in range (0,len(EDUC)):
                S_dense_aux = par.grids_dense[educ][j_pos]
                S_orig      = par.grids[educ][j_pos]
                j_pos_1     = min(j_pos,par.Jd_pos-1)
                Sp_max      = np.max(par.grids_dense[educ][j_pos_1])
                S_pol       = np.squeeze(S[j_pos][educ,:,:])
                C_pol       = np.squeeze(C[j_pos][educ,:,:])
                
                if np.max(ravel(S_pol[:])) > Sp_max:
                    print(f'Attention: age = {par.age[j_pos]}, educ = {educ}, extrapolation in ergodic distribution, min extrap = {log(np.max(S_pol[:])/Sp_max)}')
                    S_pol           = np.where(S_pol>Sp_max, Sp_max, S_pol)
                
                    Sp_min           = np.min(par.grids_dense[educ][j_pos+1])
                if np.min(np.ravel(S_pol[:])) > Sp_min:
                    print(f'Attention: age = {par.age[j_pos]}, educ = {educ}, extrapolation in ergodic distribution, min extrap = {log(np.min(S_pol[:])/Sp_min)}')
                    S_pol           = np.where(S_pol<Sp_min, Sp_min, S_pol)
                             
                pol = np.zeros([len(S_dense_aux),len(FE_pos)])
                
                for j in range(0, len(S_dense_aux)):
                    for k in range (0, len(FE_pos)):
                        pol[j,k] = interp_2d(S_orig,FE_pos,S_pol,S_dense_aux[j],FE_pos[k])
                S_dense[j_pos][educ,:,:]  = pol
                
                pol = np.zeros([len(S_dense_aux),len(FE_pos)])
            
                for j in range(0, len(S_dense_aux)):
                    for k in range (0, len(FE_pos)):
                        pol[j,k] = interp_2d(S_orig,FE_pos,C_pol,S_dense_aux[j],FE_pos[k])
                C_dense[j_pos][educ,:,:]  = pol
            
            #del S
        
        ## Save interpolated policy functions
        
        # 9. Rename grid vectors
        par2                        = copy.deepcopy(par)
        del par2.grids
        del par2.grids_dense
        par2.grids                  = copy.deepcopy(par.grids_dense)
        
        # Policy functions
        Se                          = copy.deepcopy(Se_dense)
        S                           = copy.deepcopy(S_dense)
        Ce                          = copy.deepcopy(Ce_dense)
        C                           = copy.deepcopy(C_dense)
        Ck                          = copy.deepcopy(Ck_dense)
        Np                          = copy.deepcopy(Np_dense)
        PHIp                        = copy.deepcopy(PHIp_dense)
        # Grs                       = Grs_dense
        pol_dense.Se                = copy.deepcopy(Se)
        pol_dense.S                 = copy.deepcopy(S)
        pol_dense.Ce                = copy.deepcopy(Ce)
        pol_dense.C                 = copy.deepcopy(C)
        pol_dense.Ck                = copy.deepcopy(Ck)
        pol_dense.Np                = copy.deepcopy(Np)
        pol_dense.PHIp              = copy.deepcopy(PHIp)
        pol_dense.Tp                = copy.deepcopy(Tp_dense)
        #pol_dense.Grs               = Grs
        pol_dense.tau0              = copy.deepcopy(tau0)
    
    return pol_dense,par,par2

In [114]:
"""CE tools for Julia"""


const BASE_TYPES = [:spli, :cheb, :lin]
const ABSR_MAP = Dict(
    :none => Direct(),
    :direct => Direct(),
    :tensor => Tensor(),
    :expanded => Expanded(),
)
get_bformat(b::T) where T<:BasisMatrix{Direct} = :direct
get_bformat(b::T) where T<:BasisMatrix{Expanded} = :expanded
get_bformat(b::T) where T<:BasisMatrix{Tensor} = :tensor

function to_dict(bm::BasisMatrix)
    B = Dict{Symbol, Any}()
    B[:order] = bm.order
    B[:format] = get_bformat(bm)
    B[:vals] = bm.vals
    B
end

function bm_from_dict(B::Dict)
    arr_type = eltype(B[:vals])
    bm = BasisMatrix{typeof(ABSR_MAP[B[:format]]),arr_type}(B[:order], B[:vals])
    bm
end

base_exists(s::Symbol) = s in BASE_TYPES

basedef(s::Symbol, args...) =
    s == :spli ? splidef(args...) :
    s == :cheb ? chebdef(args...) :
    s == :lin  ? lindef(args...)  :
    error("somehow you snuck through here you 👺")

basenode(s::Symbol, args...) =
    s == :spli ? splinode(args...) :
    s == :cheb ? chebnode(args...) :
    s == :lin  ? linnode(args...)  :
    error("somehow you snuck through here you 👺")

BasisMatrices.evalbase(s::Symbol, args...) =
    s == :spli ? splibase(args...) :
    s == :cheb ? chebbase(args...) :
    s == :lin  ? linbase(args...)  :
    error("somehow you snuck through here you 👺")

# Helper function
function squeeze_trail(x::AbstractArray)
    sz = size(x)
    squeezers = Int[]
    n = length(sz)
    for i=n:-1:1
        if sz[i] == 1
            push!(squeezers, i)
        else
            break
        end
    end
    squeeze(x, tuple(squeezers...))
end


# ---------------------------- #
# Generic translated functions #
# ---------------------------- #

# from fundef.m  -- DONE
function fundef(foo...)
    d = length(foo)  # 89
    n = zeros(Int, d)  # 93
    b = zeros(d)  # 94
    a = zeros(d)  # 95
    p = Array{Any}(undef, d)  # 96
    _params = Array{BasisMatrices.BasisParams}(undef, d)

    basetype = Array{Symbol}(undef, d)
    for j=1:d
        basetype[j] = foo[j][1]  # 99
        !(base_exists(basetype[j])) && error("Unknown basis $(foo[j][1])")
        n[j], a[j], b[j], p[j], _params[j] = basedef(basetype[j], foo[j][2:end]...)  # 124
    end

    # package output. Lines 143-150
    g = Dict{Symbol, Any}()
    g[:d] = d
    g[:n] = n
    g[:a] = a
    g[:b] = b
    g[:basetype] = basetype
    g[:params] = p
    g[:_basis_params] = _params
    g[:_basis] = Basis(_params...)
    g
end

# fundefn.m
function fundefn(basistype::Symbol, n, a, b, order=3)
    d = length(n)
    length(a) != d && error("a must be same dimension as n")
    length(b) != d && error("b must be same dimension as n")
    any(a .> b) && error("left endpoints must be less than right endpoints")
    any(n .< 2) && error("n(i) must be greater than 1")

    params = Array{Any}(undef, 1, d)
    if basistype == :cheb
        for i=1:d params[i] = Any[:cheb, n[i], a[i], b[i]] end
    elseif basistype == :spli
        for i=1:d params[i] = Any[:spli, [a[i], b[i]], n[i]-order+1, order] end
    elseif basistype == :lin
        for i=1:d params[i] = Any[:lin, [a[i], b[i]], n[i]] end
    end

    fundef(params...)
end

# funnode.m -- DONE
funnode(basis::Dict) = nodes(basis[:_basis])

# funbase.m -- DONE
function funbase(basis::Dict, x=funnode(basis)[1], order=fill(0, 1, basis[:d]))
    BasisMatrix(basis[:_basis], Expanded(), x, order).vals[1]
end

# funbasex.m -- DONE
function funbasex(basis::Dict{Symbol}, x=funnode(basis)[1], order=0,
                  bformat::Symbol=:none)
    to_dict(BasisMatrix(basis[:_basis], ABSR_MAP[bformat], x, order))
end

# funfitf.m -- DONE
funfitf(basis, f::Function, args...) = funfitf(basis[:_basis], f, args...)

# funfitxy.m -- DONE
function funfitxy(basis, x, y)
    c, bm = funfitxy(basis[:_basis], x, y)
    return c, to_dict(bm)
end

# funeval.m -- DONE
function funeval(c, basis::Dict, B, _order=0)
    isempty(c) && error("missing basis coefficients")
    order = BasisMatrices._check_order(basis[:d], _order)

    if isa(B, Dict)  # B is a basis structure
        bm = bm_from_dict(B)
        y = funeval(c, bm, order)
        return y, B
    else
        bm = BasisMatrix(basis[:_basis], B, order)
        y = funeval(c, bm, bm.order)
        return y, to_dict(bm)
    end
end

# fund.m
function fund(c, basis, x, hess_opt)
    # TODO: come back when I need this. I think I should probably do something
    #       like what optim does and write functions `f`, `fg!` and `fgh!` to
    #       replicate this for the type of basis instead of this function
    nothing
end

# funbconv.m  -- DONE
function funbconv(b::Dict, order=fill(0, 1, size(b[:order], 2)),
                  format::Symbol=:expanded)
    bm = bm_from_dict(b)
    new_bm = convert(typeof(ABSR_MAP[format]), bm, order)
    to_dict(new_bm)
end


SyntaxError: invalid syntax (3835526551.py, line 4)

In [ ]:
"""trans_draw"""

def trans_draw(par,mu0,tau0,Q_ergo_in):
# From mu0 to mup
# keyboard
# Choice: education
    j_pos     = par.Je1_pos-1
    S         = par.grids[0][j_pos]
    FE_pos    = par.inc.fe_pos
    EDUC      = par.educ
    PSY       = par.psy_val_hs
    
    
    ## 1. Choice of educ
    mu0int          = np.zeros([len(EDUC),len(PSY),len(S),len(FE_pos)])
    mu0int[0,:,:,:] = mu0    # initial distribution in extended grid, position 1
    mu0int_vec      = mu0int[:]
    
    if Q_ergo_in.created == 'N':

            # extend policy for education tau
            tau0p         = np.ones([len(EDUC),len(PSY),len(S),len(FE_pos)])
            tau0p[0,:,:,:]  = tau0;
            tau0int_vec   = tau0p[:]
    
            # create transition matrix of educ: linear interpolant for policy of education
            fspaceergeduc   = fundef({'spli',EDUC,0,1})
            Qeduc           = funbas(fspaceergeduc,tau0int_vec)
    
            # create transition matrix of fixed states: Qfixed
            Qfixed              = np.kron(np.ones([len(EDUC),1]),scipy.sparse.identity(len(S)*len(FE_pos)*len(PSY)))###?????????
    
            # create aggregate transition: Q
            Q   = dprod(Qeduc,Qfixed)
            Q_ergo_out.mat1 = Q
            Q_ergo_out.created = 'Y'
    if Q_ergo_in.created == 'Y':
            Q   = Q_ergo_in.mat1
            Q_ergo_out = Q_ergo_in
    # new distribution: 
    mu_int_vec      = Q.T * mu0int_vec
    muint             = reshape(mu_int_vec,length(S),length(FE_pos),length(PSY),length(EDUC));
    
    ## 2. draw of innovation z0
    # load distribution of z0 conditional on education
    z_prob     = par.inc.z_prob
    z0_prob    = [z_prob[0][0][par.Je1_pos-1].T, z_prob[1][0][par.Je2_pos-1].T, z_prob[2][0][par.Je2_pos+1].T]      # (educ,prob)
    INNO_pos   = par.inc.inno_pos
    NZ0        = len(INNO_pos)         
    
    # vectorize distribution and policy
    mu0int_vec    = muint[:]
    
    if Q_ergo_in.created == 'N':
        # create transition matrix of Z0 conditional on education
        Qz                = np.kron(z0_prob,np.ones([len(EDUC),len(PSY),len(S),len(FE_pos)]))

        # create transition matrix of fixed states: Qfixed
        Qfixed              = scipy.sparse.identity(len(S)*len(FE_pos)*len(PSY)*len(EDUC))

        # create aggregate transition: Q
        Q   = dprod(Qz,Qfixed) 
        Q_ergo_out.mat2 = Q
        Q_ergo_out.created = 'Y'
    
   if Q_ergo_in.created == 'Y':
        Q   = Q_ergo_in.mat2
        Q_ergo_out = Q_ergo_in

    
    # new distribution: 
   mup_vec = Q.T * mu0int_vec
   mup     = np.reshape(mup_vec,[NZ0,len(EDUC),len(PSY),len(S),len(FE_pos)])
    

   return mup, Q_ergo_out 

In [ ]:
""""""

In [ ]:
"""solve ergodic_distribution2"""


#https://github.com/QuantEcon/BasisMatrices.jl



from ast import literal_eval
def solve_ergodic_distribution2(par2,pol_dense,options):
# Find ergodic distribution across cohorts
    if options.timer_on == 'Y':
        print(f'\n Find ergodic distribution across cohorts \n \n')


# Compute demographic transition
# Find ergodic distribution across cohorts
###################################### OUTPUT:#####################################
#% mu{j}(a) is the pdf of state a at age j (i.e. at the begining of age j)
#################################################################################

## distribute policy functions
# from varname import varname
# def function():
#     return varname()

# func = function()
    pol_mat = copy.deepcopy(pol_dense.__dict__)##### this will be useful!!! converse SImpleNameSPce to dictionary
    pol_mat = pol_mat.keys()
    for i in range (0,len(pol_mat)):
        eval([literal_eval(pol_mat[i]) '= pol_dense.(cell2mat(pol_mat(i)));']);
    #######################################need to finish but eval is ugly
    #del pol_dense
    
    
    # Empty cell for measures at each age
    mu        =  {}
    
    for i in range (0, par2.Jd_pos-1):
        mu[i] = [[]]
    
    Q_ergo =  {}
    for i in range (0, par2.Jd_pos-1):
        Q_ergo[i] = [[]]

    for jj in range (0,par2.Jd_pos-1):#structure array 
        Q_ergo[jj].created = 'N'

    
    
    ## mu{1}: guess
    
    # Load initial guess for tau and Vc0

    os.chdir('/Users/Giorgia/Dropbox/ReplicationProject/DairuchPaper/DairuchFiles/run_generic')        
    filename = str('dist_ss')
    mu = str(filename + '.mat')
    file_path = r'/Users/Giorgia/Dropbox/ReplicationProject/DairuchPaper/DairuchFiles/run_generic/'+mu
    exist = os.path.isfile(file_path)

    if exist:
        guess_ = loadmat(mu)
        j_pos = par2.Je1_pos-2
        guess.mu = copy.deepcopy(guess_["mu"])
        for j in range (0,3):
            guess.mu[j][0] = guess.mu[j][0].T
            for i in range(0,10):
                guess.mu[j][0][i] = guess.mu[j][0][i].T
        j = 3######################################################careful here that dimensions 3 and 10 are inverted here (in matlab (10,3))
        guess.mu[j][0] = guess.mu[j][0].T
        for i in range (0,3):
            guess.mu[j][0][i] = np.reshape(guess.mu[j][0][i],[10,20,20])
            for k in range (0,10):
                guess.mu[j][0][i][k] = guess.mu[j][0][i][k].T
        j = 4# (3, 10, 100, 100) vs (100   100    10     3)
        guess.mu[j][0] = guess.mu[j][0].T
        for k in range (0,10):
            guess.mu[j][0][i][k] = guess.mu[j][0][i][k].T
                
        for j in range (5,8):# (3, 100, 100)  vs 100   100     3
            guess.mu[j][0] = guess.mu[j][0].T
            for i in range(0,3):
                guess.mu[j][0][i] = guess.mu[j][0][i].T
        for j in range (8,10):# (4, 3, 100, 100)  vs 100   100     3     4
            guess.mu[j][0] = guess.mu[j][0].T
            # for i in range(0,3):
            #     guess.mu[j][0][i] = guess.mu[j][0][i].T
        j = 10 # (3, 20, 4, 3, 100, 100) vs 100   100     3     4    20     3 ######################gotta fix this
        guess.mu[j][0] = guess.mu[j][0].T
        
        muc   = guess.mu[j_pos][0]
    
    else:
        muc = np.zeros([2,2])

    j_pos     = par2.Je1_pos-1
    S_grid    = par2.grids[0][j_pos]
    FE_pos    = par2.inc.fe_pos
    PSY       = par2.psy_val_hs
    
    # If guess is of different size of current grid start new guess
    muc_aux = np.zeros([len(PSY),len(S_grid),len(FE_pos)])
    if muc.ndim != muc_aux.ndim:
        # start with a guess for distribution over (s,educ)
        # Guess 1: 'symmetric': one obs for each grid point (s,educ)
        muc    = np.zeros([len(PSY),len(S_grid),len(FE_pos)])
        nn     = len(PSY)*len(S_grid)*len(FE_pos) #size of grid to get probability measure
        muc[:] = 1/nn
    elif np.sum(np.array(np.shape(muc)) - np.array(np.shape(muc_aux))) != 0:
        # start with a guess for distribution over (s,educ)
        # Guess 1: 'symmetric': one obs for each grid point (s,educ)
        muc    = np.zeros([len(PSY),len(S_grid),len(FE_pos)])
        nn     = len(PSY)*len(S_grid)*len(FE_pos)#size of grid to get probability measure
        muc[:] = 1/nn

    
    pop = np.zeros([options.maxiterD,1])
    cohort = 0
    dif = 1
    
    if options.timer_on == 'Y':
        print(f'cohort: {cohort}, dif:{dif}, pop:{np.sum(muc[:])}, time ={toc}')

    while ((dif>options.tolD ) and cohort <=options.maxiterD):
        cohort = cohort + 1
        j_pos      = par2.Je1_pos-2
        mu[j_pos] = muc # update initial distribution for cohort
        
        ## Age j=Je1: new states are education and innovation
        j_pos = par2.Je1_pos-1
        [mu[j_pos],Q_ergo[j_pos-1]] = trans_draw(par2,mu[j_pos-1],tau0,Q_ergo[j_pos-1])#### here the shit starts
        
        if options.timer_on == 'Y':
            tot = sum(mu{j_pos}(:))
            print(f'age:{par2.age(j_pos)}, pop:{tot}, time:{toc} \n')

        
        ## Age j=Je1+1:Je2+3
        for j_pos in range(par2.Je1_pos,par2.Je2_pos + 2):
            mu[j_pos],Q_ergo[j_pos-1] = trans_educ(par2,mu[j_pos-1],Se[0][j_pos - par2.Je1_pos],Se[1][j_pos - par2.Je1_pos],Se[2][j_pos - par2.Je1_pos],j_pos,Q_ergo[j_pos-1])
            # Only need to pass policy functions given education, iid shock does
            # not matter once we condition on education choice
            if options.timer_on == 'Y':
                tot = np.sum(mu[j_pos][:])
                print(f'age:{par2.age(j_pos)}, pop:{toc}, time: {toc} \n')

    #     mu{j_pos}                = squeeze(sum(mu{j_pos},3)); % Remove Psychic Cost
     
        
       ## Age j=Je2 + 2: Jc
        for j_pos in range(par2.Je2_pos+2,par2.Jc_pos):
            mu[j_pos],Q_ergo[j_pos-1] = trans_work(par2,mu[j_pos-1],S[j_pos-1],j_pos,Q_ergo[j_pos-1])
            
            if options.timer_on == 'Y':
                tot = np.sum(mu[j_pos][:])
                print(f'age:{par2.age(j_pos)}, pop:{toc}, time: {toc} \n')

        
        ## Age Jc
        j_pos = par2.Jc_pos
        fert_pol = Np
        mu[j_pos],Q_ergo[j_pos-1] = trans_fertility(par2,mu[j_pos-1],S[j_pos-1],fert_pol,j_pos,Q_ergo[j_pos-1],options)
        switch options.timer_on
            case {'Y'}
                tot = sum(mu{j_pos}(:));
                fprintf('age:%i, pop:%1.2f, time: %3.1f sec \n',par2.age(j_pos),tot,toc)
        
        ## Age Jc+1
        for j_pos in range(par2.Jc_pos+1,par2.Jc_pos+par2.Je1_pos-2):
            mu[j_pos],Q_ergo[j_pos-1] = trans_work_with_child(par2,mu[j_pos-1],S[j_pos-1],j_pos,Q_ergo[j_pos-1])
            if options.timer_on == 'Y':
                tot = np.sum(mu[j_pos][:])
                print(f'age:{par2.age(j_pos)}, pop:{toc}, time: {toc} \n')
        
        ## Age Jc+2: phi transfer + new cohort
        j_pos       = par2.Jc_pos+par2.Je1_pos-2
        #     j_pos_child = par2.Je1_pos;
        phi_pol = PHIp
        mu[j_pos],Q_ergo[j_pos-1] = trans_work_trans(par2,mu[j_pos-1],S[j_pos-1],j_pos,Q_ergo[j_pos-1])
        if options.timer_on == 'Y':
            tot = np.sum(mu[j_pos][:])
            print(f'age:{par2.age(j_pos)}, pop:{toc}, time: {toc} \n')
        
        pop_final  = pop[cohort]
        j_0_pos = par2.Je1_pos-2
        dif = np.max(abs(mu[j_0_pos][:]-muc[:]))
        # Describe muc
        mu[0]      = muc
        mu[1]      = muc
        
        if options.timer_on == 'Y':
            tot = np.sum(mu[j_pos][:])
            print(f'cohort:{cohort}, pop:{pop(cohort)}, time: {toc} \n')

    return mu,mu_ige0,muc,pop_final,Q_ergo

[[7.45412218e-01 2.27259908e-01 2.59824687e-02 1.32024789e-03
  2.51571719e-05]
 [5.68149769e-02 7.58403453e-01 1.71435117e-01 1.30163915e-02
  3.30061972e-04]
 [4.33041145e-03 1.14290078e-01 7.62759021e-01 1.14290078e-01
  4.33041145e-03]
 [3.30061972e-04 1.30163915e-02 1.71435117e-01 7.58403453e-01
  5.68149769e-02]
 [2.51571719e-05 1.32024789e-03 2.59824687e-02 2.27259908e-01
  7.45412218e-01]]
[[7.52379252e-01 2.21859768e-01 2.45330512e-02 1.20570777e-03
  2.22210113e-05]
 [5.54649420e-02 7.64645778e-01 1.67299107e-01 1.22887466e-02
  3.01426942e-04]
 [4.08884187e-03 1.11532738e-01 7.68756841e-01 1.11532738e-01
  4.08884187e-03]
 [3.01426942e-04 1.22887466e-02 1.67299107e-01 7.64645778e-01
  5.54649420e-02]
 [2.22210113e-05 1.20570777e-03 2.45330512e-02 2.21859768e-01
  7.52379252e-01]]
[[7.57462866e-01 2.17886646e-01 2.35034246e-02 1.12680603e-03
  2.02580877e-05]
 [5.44716614e-02 7.69214578e-01 1.64260089e-01 1.17719704e-02
  2.81701508e-04]
 [3.91723744e-03 1.09506726e-01 7.7315

namespace(inno_pos=[0, 1, 2, 3, 4],
          fe_pos=range(0, 5),
          age_prof={0: [array([0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
                            0.       , 0.       , 0.       , 0.0639914, 0.1235652, 0.1787214,
                            0.22946  , 0.275781 , 0.3176844, 0.3551702, 0.3882384, 0.416889 ,
                            0.441122 , 0.4609374, 0.4763352, 0.4873154, 0.493878 , 0.496023 ,
                            0.4937504, 0.4870602, 0.4759524, 0.460427 , 0.440484 , 0.4161234,
                            0.3873452, 0.3541494, 0.316536 , 0.274505 ])],
                    1: [array([0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
                            0.       , 0.       , 0.       , 0.0838808, 0.161912 , 0.2340936,
                            0.3004256, 0.360908 , 0.4155408, 0.464324 , 0.5072576, 0.5443416,
                            0.575576 , 0.6009608, 0.620496 , 0.6341816, 0.6420176, 0.644004 ,
       

In [64]:
C



NameError: name 'C' is not defined